## <span style="color:royalblue">Capstone: Cleaning and Transformation Notebook 1 by Puneet Sran</span>

#### <span style="color:purple">Please note that the cleaning and transformation of data is split into two notebooks as otherwise, a single notebook would be too long. At the end of this notebook, I save the work performed in this notebook as a new .csv file, which I then open in Part 2.</span>

In [1]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Read in data
ok_cupid_df = pd.read_csv('data/okcupid_profiles.csv')
ok_cupid_df.columns

Index(['age', 'status', 'sex', 'orientation', 'body_type', 'diet', 'drinks',
       'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'pets', 'religion', 'sign',
       'smokes', 'speaks', 'essay0', 'essay1', 'essay2', 'essay3', 'essay4',
       'essay5', 'essay6', 'essay7', 'essay8', 'essay9'],
      dtype='object')

In [4]:
# Check shape
ok_cupid_df.shape

(59946, 31)

In [5]:
ok_cupid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [6]:
ok_cupid_df.head(2)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN


In [7]:
# fill income column containing -1 with null (to determine null values)
ok_cupid_df.loc[ok_cupid_df['income'] == -1, 'income'] = np.nan

In [8]:
# check percentage of null values
round(ok_cupid_df.isna().sum()/ok_cupid_df.shape[0]*100,2)

age             0.00
status          0.00
sex             0.00
orientation     0.00
body_type       8.83
diet           40.69
drinks          4.98
drugs          23.49
education      11.06
ethnicity       9.48
height          0.01
income         80.81
job            13.68
last_online     0.00
location        0.00
offspring      59.32
pets           33.23
religion       33.74
sign           18.44
smokes          9.19
speaks          0.08
essay0          9.15
essay1         12.63
essay2         16.08
essay3         19.14
essay4         17.58
essay5         18.10
essay6         22.97
essay7         20.77
essay8         32.07
essay9         21.02
dtype: float64

In [9]:
# refill income null back with -1
ok_cupid_df['income'] = ok_cupid_df['income'].fillna(-1)

## <span style="color:royalblue">Fill null values:</span>
### <span style="color:royalblue">Drugs:</span>

In [10]:
ok_cupid_df['drugs'] = ok_cupid_df['drugs'].fillna('unknown_drugs')
ok_cupid_df['drugs'].value_counts()

never            37724
unknown_drugs    14080
sometimes         7732
often              410
Name: drugs, dtype: int64

In [11]:
# find index where drug value is unknown an drop it from rows
index_drugs = ok_cupid_df[ok_cupid_df['drugs'] == 'unknown_drugs'].index
ok_cupid_df.drop(index_drugs, inplace = True)

### <span style="color:royalblue">Diet:</span>

In [12]:
# fill null diet with unknown
ok_cupid_df['diet'] = ok_cupid_df['diet'].fillna('unknown_diet')
ok_cupid_df['diet'].value_counts()

unknown_diet           18560
mostly anything        13105
anything                4805
strictly anything       3653
mostly vegetarian       2530
mostly other             785
strictly vegetarian      698
vegetarian               450
strictly other           333
other                    257
mostly vegan             252
strictly vegan           182
vegan                     96
mostly kosher             70
mostly halal              41
strictly halal            15
strictly kosher           15
halal                     10
kosher                     9
Name: diet, dtype: int64

#### <span style="color:royalblue">Fill `unknown_diet` based on essay columns:</span>

<span style="color:royalblue">I spent quite a bit of time exploring the essay columns one day and it occured to me that I can possibly fill other features if I can find that information from the essay columns. I started with a simple search which led me to try and find more values and enhance my search.</span>

<span style="color:royalblue">I did read majority of these essays to determine if the users diet was X.</span>

In [13]:
# use .loc to find specific essay string and fill 'diet' column with appropriate diet
ok_cupid_df.loc[ok_cupid_df['essay0'] == "im looking for someone to share some raging adhd. im a self motivated and light hearted superhero who enjoy's riding my bike everywhere and eating every goddamn thing i can.  im looking for someone to go adventuring with. i enjoy blind drunken adventures sometimes but you dont have to be a drinker. no vegans, i will eat anything... including people... especially hipsters. im not really a nerd (i don't play magic cards/excessive videogames) but i can like nerdy girls.  i just got this account, so gimmie some time to write down more shenanigans that are important  if u make chiptunes hit me the fuck up! i wanna make some!  i am awesome, eccentric, and energetic", 'diet'] = 'strictly anything'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "rabid bibliophile, humorless feminist (that's a joke), eternal student. i like to write poetry on people, bake (vegan) cupcakes, make art and dress-up.  i identify as queer but my choices here are limited so i chose bisexual.  i am quiet, empathetic, and geeky", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "cooking all types of foods, caring for others, being very social and outgoing. i think i am really good at horseback riding it is one of my true passions even though i have not gone in a long time. singing in the shower is always a good time....lol.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay2'] == 'writing and listening. in addition, i am an amazing troubleshooter. i thoroughly enjoy (and therefore have gotten good at) making people laugh, acting, bicycling, juggling, copy editing, computers... and lots more. - why does okcupid rate me "less energetic" than other guys? that\'s kind of weird if you know me. i\'m, like, chatty. plus i manufacture heat really well. i save on heating costs!  also, i\'ve recently learned that i\'m a good cook, mainly because i can follow a recipe. ask me about my unerring chicken & dumplings or my stupid good pot pie. it\'s so easy when it\'s easy!', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "i'm a musician at heart and i'm great at playing the drums. i also love to cook, especially butter chicken.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "eating chicken wings, shooting guns(recreational lol not at people) i'm actually pretty good at a lot if stuff it's just those first two are probably the most relevant lol :-) if you get to know me youll find out ;-) i'd say im a pretty interesting individual, but i know most of you lady's are basically looking at my photos and only that and couldn't give a dam about what i'm bout besides whether or not i'm making money..", 'diet'] = 'anything'

essay4_diet_string = ok_cupid_df[ok_cupid_df['diet'] == 'unknown_diet']['essay4'].tolist()[2]
essay4_diet_string

ok_cupid_df.loc[ok_cupid_df['essay4'] == essay4_diet_string, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "one of my hobbies is making sushi.  overall i like most foods. one of my personal rules is to try things at least once, so i get to experience a lot of variety.  books: i have enjoyed harry potter series, da vinci code, and what i wish i knew when i was 20. i don't read as much as i would like to, but when i do i enjoy it.  i am open with music, i can find good music in any genre, even country, one can find good in even the worst. for the most part, i enjoy techno/dance, 80's rock (queen, bon jovi, scorpions), classic/jazz (sinatra, elvis, big bad voodoo daddy, red elvises), and current top charts. one of my favorite new discovery's have been apple trees & tangerines.  movies: i enjoy watching movies in theaters. one of my favorite is lord of war but the list includes; ferris bueler, sean connery movies (the rock, finding forester, hunt for red october, bond), indiana jones trilogy, quentin terantino flicks (pulp fiction, reservoir dogs, inglorious bastards, etc.). leonardo dicaprio has put himself in awesome roles since titanic, catch me if you can and the aviator. the brothers bloom. .", 'diet'] = 'anything'

In [14]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "where to begin. yikes.  *funk/soul/rnb: jamiroquai, stevie wonder, earth wind and fire, chaka khan, incognito, brand new heavies, the meters, d'angelo, erykah badu *jazz: marcus miller, weather report, jaco pastorius, avishai cohen, esperanza spalding, mike stern, jeff beck *hip hop: the roots, wu-tang, tupac, e-40, outkast, dre, talib kweli-*drum n bass/ electronica: calibre, london elektricity, logistics, nu-tone, break, goldie, *rock: soundgarden, alice n chains, red hot chili peppers, jimi hendrix, tommy bolin, peter frampton, led zeppelin, stevie ray vaughn, the police, u2, in flames, deftones, soilwork, megadeth, pantera, at the gates.....this list could take up the whole profile...lol.  food: sushi, udon, chicken tikka masala, pad thai, red, green, yellow curries, a great burger, great ny pizza, gyros/shwarma/ mediterranean.  authors/ books: to name just a few: ken follet, heinlen, margaret atwood, tolkein, frank herbert, krishna murti, eckhart tolle, hesse,", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i\'m not so good at these lists of books, movies and food because i have such wide tastes, which frankly don\'t matter too much to me... and lately i\'ve been too busy moving, gig-ing and touring to relax.  but here are some generalizations:  i mostly read books. i vary between old classics to feed my brain, non-fiction and escape-ist fiction novels. i loved anna karenina, lolita, the last of the mohicans. i love well reasearched sience/social-science books, like "guns germs and steel." occasionally i need a fun escape. i have been a big tom robbins and vonnegut fan in the past. i\'m finally reading barbara kingsolver\'s books - i\'m wondering what took me so long to discover her.  movies: i tend to like comedies, and some science fiction, and deep movies that make you think. i don\'t like horror movies (blech!)  tv: i don\'t usually watch it (no cable.) but i really like: the mighty boosch, and saturday night live, and the daily show. i\'m open to learning more about what\'s good on tv - but i\'ve been out of tv culture for so long that i feel like a luditie.  music: yes! i like a lot of styles. lately i\'ve been really enjoying: arcade fire, regina specktor, joanna newsome, vampire weekend, iron & wine, the decemberists, kings of conveinience, ... i love discovering new things! even though i\'m mostly an indie and folksy music fan, i have a soft spot for good electronic music. especially if it makes me want to dance. but i also appreciate really spacey ambient music. like i said, i have wide tastes. if it\'s a well crafted piece of music - i\'ll probably like it - that includes rap, hip-hop, country and rock.  i like getting new music from many of my friends whom are musicians.  food: i\'m not very picky. my only real aversion are green bell peppers. and i don\'t eat a lot of meat, but i\'m not vegetarian - i tend to stay away from beef. (i have this funny philosophy that if the animal is too big for me to feel comfortable killing it, slaughtering it, and consuming it, then maybe i shouldn\'t eat it. chicken and lamb are good - yum!) that said: i like thai, indian, italian, sushi, ethiopian, ... you name it!', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: fantasy, sci-fi... i just finished the song of ice and fire books (game of thrones) and i can't recommend that enough!! looking for the next book to read. any ideas?  music: first love was metal, then came drum & bass and now i'm into dubstep. but that's not all, i love bjork, amon tobin, trance, alternative, even some hip hop. i go to live shows at least every month or two.  tv shows: seinfeld, south park, metalocalypse, sparticus, game of thrones, wilfred, flight of the concords, daily show, colbert report, conan (both of them).  food: mexican, authentic chinese food, sushi, thai, mitchell's ice cream, my own cooking.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: 1491. omnivore's dilemma. the amazing adventures of kavalier and clay. the princess bride. the necessary revolution. dubliners. the phantom tollbooth. the anatomy coloring book. on the origin of species.  music: live shows! local bands to fall in love with. new stuff. old stuff. arcade fire. beatles. malajube. louis armstrong. decemberists. phoenix. spoon. bach.  food: healthy and only a little not healthy. pizza. blueberries. mexican. salvadoran. spicy asian. indian. mission tacos. i make a mean pasta sauce. and a real man can admit his addiction to milk and cookies. just sayin'.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "some favorite authors poets and playwrights are:  stephen dunn, milan kundera, tony hoagland, miranda july, cormac mccarthy, tom robbins, george bernard shaw, samual beckett, leonard cohen and anne lamott  music:  leonard cohen, tom waits, bright eyes, pulp, radiohead, m ward, gillian welch, regina spektor, depeche mode, the national, okkervil river, ani difranco, the blow, cat power, elliott smith, atmosphere, jay z, johnny cash, modest mouse, jolie holland, feist, sole, smog, arcade fire, sage francis...  guilty pleasures:  comics. i used to spend all my paper route money on the little bastards. not so much anymore but i still have a soft spot for people in tights. huge x-men and batman fan. when i was 12 years old michelle pfeiffer came along in that leather suite and catapulted me into manhood (whatever way you want to take that is probably correct)  anything vampire is also locked away in the box-o-shame. i love buffy the vampire slayer - actually, i kind of geek out on most of the stuff that joss whedon has done.  food:  i love food, all food, but especially seafood. and i know i should say that i cook because people like it when you say that, but i don't much. someday though.", 'diet'] = 'anything'

In [15]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: the unbearable lightness of being, milan kundera. girl with the dragon tattoo, stieg larssen (such a page turner). three cups of tea, greg mortenson. the spirit catches you and you fall down- anne fadiman the little prince, antoine de saint-exupry  movies: american beauty. fried green tomatoes. amalie.  music: all kinds. sting, adele, black eyed peas, van morrison, lenny kravitz, red hot chili peppers, alicia keys, train, lauryn hill, plain white t's.  food: in general, anything fresh and healthy. i like unique flavors. love sushi and indian. ice cream and chocolate are my weaknesses.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "anime - fairy tail and one piece are the ones i'm currently watching.  shows - simpsons, south park, family guy, and american dad.  food - chocolate and steak  music- jpop jrock rap and some hip hop  movie - my absolute fav is shaolin soccer. if you understand cantonese you must watch it. crows zero is an awesome high school fighting movie set in japan.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "wow, they grouped a lot of items together in this one!  my favorite author is chuck palahniuk, invisible monsters will always be a favorite.  i love pretty much all movies. i like movies that make me think and movies i don't really have to think about. comedies, dramas, horror. and i have a strange addiction to zombie movies (double tap)  as for tv, i really like sitcoms. sillier the better. modern family, archer, southpark, etc. but tv is a lot like movies, i really enjoy it all, except reality tv :-p  and food, well, that's not as hard. i love all food. i eat anything and am obsessed with trying new things (i work for a company that specializes in food, i have to stay up on what's new and tasty in the city).", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "authors: rumi, george lipsitz, barack obama, amy tan, robert frost, allen ginsberg, stannard, kerouac, tom wolfe, steinbeck, terence mckenna, poe, chopra, ferlinghetti, maya angelo, timothy leary, ken kesey, thoreau, hunter thompson, hafiz, r. crumb, martin luther king, jr.  movies: being there w/peter sellers and koyaanisqatsi, coen brothers and david lynch kind of stuff, o brother! some tv (rather streaming online) has entered my realm after years off the tube: jon stewart, house md, true blood, the mentalist. stuff i can watch online, at my convenience and commercial-free.  music on my latest mix: joules graves, b. w. stevenson, si*s, nina simone, ani difranco, bill evans, syd barrett, yes, linda ronstadt, pink floyd, the pixies, neil young, miles davis, chuck mangione, mark farina, baby gramps, anne murray, erykah badu, cabaret diosa, dead prez, oliver nelson, the doors, bebe, trillian green, leon redbone, lhasa de sela, john coltrane, coco de mer, michael franti, the police, jimi hendrix, alanis, steely dan, putumayo latin lounge, al di meola, stevie wonder, violent femmes, portishead, astrud gilberto, massive attack, janis, tiny tim, elo, csn&y, elp, moody blues, trip hop and down-tempo ambient, psychedelia and punk, classical and bebop jazz, fusion, flamenco ...... and yes, disco.  foodies: thai, japanese, mexican, indian, chinese, veg, raw... anything healthy, vibrant, organic. i gravitate towards vegetarian, not so much into dairy and meat at present (except fish). if it is organic i might try anything!  dark chocolate and red wine are just as sensuous as mango in my book...", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'in all of these categories, i like variety...i rarely "don\'t like" things, which means that everything has its entertainment value.... books = ones that inspire me to question and hopefully expand and improve who i am, or ones that just thoroughly entertain movies = everything from commercial to independent, horror to documentary music = everything from dance/rap to country to classical to r&b/soul food = everything form sushi to pasta, veggies to burgers did you notice a theme? extra points, if you can relate.', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i really love movies, so i have a lot of favorites. some of the old standbys are the wedding singer, almost famous, the fisher king, never been kissed, tap (with gregory hines, who was an amazing dancer!), instinct, regarding henry, remember the titans, and there are lots more that i cannot remember off the top of my head. i cruise around the movies channels on the weekend and watch whatever sounds good to me at the time.  and with music i could go on and on because i have my favorites in every style. but i grew up listening to r&b so that is my first love, and i always try to keep up with what is current, so i listen to kmel and reserve cds from the library so i can have new songs on my ipod.  comfort food that soothes the soul, that's what i like the best. give me cheese, sauces, pumpkin pie with whipped cream, boysenberry cobbler, smothered pork chops, mashed potatoes with real gravy, shrimp scampi, enchiladas verde, flank steak with caper sauce, buttermilk fried chicken, eggs benedict, and pretty much anything that paula dean is cooking today works for me! my mom was irish and came from a family of 12 kids, and she could cook absolutely anything and make it taste wonderful...:)", 'diet'] = 'anything'

In [16]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: les liaisons dangereuses by choderlos de laclos, influence by cialdini, the story of b by daniel quinn.  movies: amadeus, fight club, secretary, troy, the princess bride  music: mozart, verdi, "burning man thumpy thumpy" music, 80\'s pop, and whatever my dj friends may be mashing up at the time  food: favorite is mexican, but love pretty much anything. especially if it at one time breathed & had a soul... :) if you\'re a vegan, move along.', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "tv: buffy, sherlock, stargate atlantis, the x-files, twin peaks, daria, good eats, reading rainbow, clarissa explains it all. classics of our time! also, mail call.  books: science fiction, fantasy, mystery, and trashy romance, not necessarily in that order. most recently, i've been reading the dresden files,  girls to the front: the true story of the riot grrrl revolution, and the magician king, in no particular order.  music: electroclash, electro, pop, post-punk, avant-garde jazz, singer-songwriter stuff, musicals, and... i like a lot of music.  food: mostly vegetarian except for sushi and cheeseburgers. (i don't eat meat on a regular basis, but it's okay if you do.) i really enjoy vegetarian and vegan cooking, as well as cooking creatively for friends with dietary restrictions.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: life of pi, jitterbug perfume, the alchemist, an imaginary life, lamb, lust lizard of melancholy cove, the four agreements and for "guilty pleasure" most of dean koontz\'s books.  movies: i love movies, and can appreciate things in just about every movie. some of my favorite movies are: the princess bride (all-time favorite), the matrix trilogy, love actually, saving private ryan, gladiator, the usual suspects and the lord of the rings trilogy.  music: almost anything. my ipod bounces amongst the likes of audio slave, rachmaninoff, paul simon, nickelback, harry connick jr, journey, abba, phantom of the opera ... it\'s quite a surreal experience.  food: i\'ll eat just about anything now, though i was a vegan for 2.5 years. ironically i was on atkin\'s for the 2 year previous to being vegan. among my favorites are pepperoni pizza, thai food, sushi, chili and just about every dessert ever created (especially ones with lots of chocolate).', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'partial to history (durant, tuchman), some classic lit (english & russian) , mix of contemporary genres just for fun, e.g., "spymaster\'s lady" ;) i could get really pretentious here with some heavy lifting (i.e., math, technical), i\'ll pass on that.  movies... good question... but directors, or titles? let\'s go with directors: wong kar wai, lina wertmuller and liliana cavani for starters. maybe a swing through shainberg as well.  titles... vickie christina barcelona! gotta love it. only woody allen could get so much mileage out of a 1 song soundtrack. there\'s so much more: up, casino royale, conan the barbarian (dialog blows but the body language is pretty good!), king fu hustle, steven chow rocks it. room with a view. i could go on and on.  food: discerning but not picky. i\'m cool with everything from bk to gary danko. yes, the coffee-rubbed pork shoulder at range is often a little too salty, and yes, i like it anyway. you\'re free to treat me to french laundry or whats-her-name\'s place in berkeley. i thought myth was a little overrated... but the food and service were good. quince is pretty cool, and not at all snooty if you use the manners you were raised with. i\'ve done the raw thing in santa monica... it\'s ok. vegan is fine. good vegan is even better. in general, my philosophy is that food is good to eat.  music: pretty much anything in context. i\'ve seen everything from kathleen battle at a revival to the who doing tommy (whole thing) live in concert. so, whatever, it\'s all good. that being said, i tend to gravitate toward jazz genres along the miles in a silent way sessions, really early blues influenced rock (sabbath first album, heep), beethoven symphonies and piano work (beethoven would have *definitely* been a metal head), deep house, psychill, ambient and downtempo for the more contemporary sounds. i\'m cool with trance (van buuren, schulz, wisternoff, van dyke) in moderation.  as i think about it... i\'m playing mostly stuff along the lines of thievery corp, morphine, ben watt and zero 7 at the moment.  actually, i\'m mostly listening to groovera\'s low mercury on itunes. bought some good music from their playlist.', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books i love to read, i love books. there are too many to name, but here are a few: brave new world, evolutions rainbow, the neverending story, jitterbug perfume, wild dreams of a new beginnings, the future of life, middlesex.  movies... i use to love them, but now i can't seem to sit through an entire 2+ hour show.  music just got adele's new album and i can't stop listening to it! i love ryan adams, the be good tanyas, mirah, radiohead, bowie (to name a few).  food i love food. i have been a vegan for the last 10 years, so vegan food :)", 'diet'] = 'strictly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'music: let\'s just say there was a time when i could have djed the biblical flood, so to name everything.......help. (we\'re assuming it was 40 days and 40 nights)  books: contemporary poetry, anything wave publishes.....current reads as of now, iq84, many masters many lives, omnivore\'s dilemma and all sorts of contemporary poetry. i usually have a current \'to be read\' stack.  movies: i usually will watch anything, but can be snobby sometimes. i will openly admit to seeing \'insert anything\' the other night with an old girlfriend and surprisingly enjoyed sucking down a monster sprite with a mouth full of popcorn.  food: well, i obviously like popcorn and......!!! recently i\'ve become mostly vegan which is somewhat surprising. i totally used to be a cheeseburger, milk and egg head. what happen? i started hanging with vegans and.....i read several books, watched a series of relevant documentaries and watched how my body felt after changing my diet. i couldn\'t help but change. "mostly\' is a key word and i will eat anything made/offered with love (within reason). --i will not eat your florescent cheetos unless i\'m deeply under the influence.  t.v i\'ll be slightly impressed if you know: *absolutely fabulous *strangers with candy but the truth is....i don\'t plug into zombie trances too often.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i mostly read non-fiction, though if you\'ve got a good fiction recommendation, i\'d love to hear it. just finished "kindred" by octavia butler. now reading i may not get there with you. strong influences: power of myth, criminal of poverty: growing up homeless in america, redefining relationships (i\'m not a polyamorist, but this book respectfully engages taboos that i\'ve taken for granted. even so, monogamy is what i\'m seeking), why we can\'t wait, dharma, color, and culture: new voices in western buddhism (this isn\'t really a favorite book of mine, but if you know it and relate, highfive!), non-violent communication (can\'t say i\'m a regular practitioner, but i try when i can, or at least when i have to), people\'s history of the u.s. (zinn), michael eric dyson, etc...  i won\'t sit through any movie, but some of my favorites have ranged from musicals (colma, once, resurrection) to comedies (hitch, for what it\'s worth/dave chappelle, 40 yr old virgin, superbad, kung fu hustle) and etc (kill bill, you can\'t be neutral on a moving train).  music: lotsa pandora these days...current stations: handsome boy modeling school, booka shade and then erykah, alicia, jay-z, marketa irglova, etc, etc... podcast\'s too, especially if they make me laugh...  as far as food goes, 99% of what i eat could be called vegan, but i\'m not a vegan. i\'m a omnivegavore. maybe.', 'diet'] = 'mostly vegan'

In [17]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "depending on my mood, i'll read anything from cereal boxes, trivia of all sorts, the dictionary, book jackets, news articles, obits, fiction, nonfiction, billboards...you get the picture.  these are a few of the books i've liked a lot: song of solomon, toni morrison the liar's club, mary karr light in august, william faulkner atonement, ian mcewan the namesake, jhumpa lahiri  my fav from 2011: the talent code by daniel coyle  movies: i love movies. i'll even watch a corny old bad one if it's got a thing or two going for it: an actor i dig, a city i love or know well, a half-way decent plot, a great scene. a film has to be ber bloodlusty for me not to watch it or just bloody bad.  films i've liked & loved: no country for old men, days of heaven, traffic, love, actually, imagine me & you, moulin rouge, boys don't cry, 21 grams, a room with a view, howard's end, diner, mona lisa  and after two episodes of amc's the killing, i think i'm going to have to add that to my list of fav tv shows. and also i'm digging this new bbc show, luther with idris elba. oh, idris...  music: i have broad musical tastes. i dig some classical tunes, jazz, r&b, blues, movie soundtracks, rock n' roll, even a few country tunes here and there. i can't say that i have favorites, but i definitely have music moods. for the past few months i've been writing to nick cave & warren ellis's soundtrack for the assassination of jesse james by the coward robert ford. --man, i'm a sucker for a fiddle.  food: i love asian food of all types, especially thai. actually, anything that's got lots of vegetables, no dairy, and plenty of fish is right up my alley. but i'm not a vegan, pescatarian...whatever. in the past few months i've turned back to fowl and lamb (sorry can't do the cow, pig thing) to level out the aminos & the iron. all, i can say is wow, what a difference a chicken makes.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i have so many books in my house and in my brain. i love anything by dostoevsky, virginia woolf, steinbeck, sandra cisneros, tom robbins...there are so many. i am a vegan. i love good music, of all kind.", 'diet'] = 'strictly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i learn a lot about life, the world, and human nature form literature, so the list of those to whom i am particularly indebted is long. my favorite authors/books: * thomas mann (my absolute favorite novel is "the magic mountain," but then there is also "the holy sinner") * a.s. byatt * nadine gordimer * j.m. coetzee (especially the animal questions) * kafka * henry james * goethe\'s "faust" and some others * sophocles * aeschylus * flannery o\'connor * "the little prince" * eugene o\'neill * joyce carol oates * "the god of small things" * plus nietzsche, kant, hobbes, vicki hearne, and numerous other philosophers * a lot of cultural and biological anthropology (my favorite among these: "nuclear rites" or "the symbolic species: the coevolution of language and the brain")  my favorite directors/movies: * tarkovsky (especially "andrei rublev") * bresson * kurosawa (especially "the lower depths" and "the idiot") * bergman * everything involving david attenborough * dreyer * schlondorff * ozu * mizoguchi\'s "ugetsu" * truffaut\'s "the 400 blows" * christopher guest * wes anderson * "harold and maude" * "breakfast at tiffany\'s"  my favorite music is harder to specify. i love bach and early polyphonic music, and sometimes very much enjoy brahms, beethoven, or wagner. i also like, but in a trivial way, a lot of pop music from my youth, due to its ability to reactivate those old feelings.  my absolute favorite food is pizza (though it is almost impossible to find a good one in the bay area), followed by cuisines with lots of vegetables, and sometimes tending toward macrobiotic. when i am not searching for good pizza, i am a mostly-vegan ethical vegetarian, so i prefer vegetable-oriented ethnic cuisines. middle eastern/mediterranean food is probably my favorite among these, since i can never get too much hummus. i also enjoy cooking, and try to make everything as organic and locally grown as possible.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: fiction - exit to eden, god of small things, the bell jar, the electric kool-aid acid test. non-fiction - embracing the beloved, destiny of souls, true love, power of now, a general theory of love. also, poetry - hafiz, e.e. cummings and sylvia plath.  movies: i'll go see anything if it's in 3-d. not a movie critic, pretty impressed with any full-length film, on principle. steer clear of guns, action flicks, and slasher-movies. faves are annie hall, leaving las vegas, lost in translation, synecdoche new york, and some old kid classics like the last unicorn, the neverending story, dark crystal and labyrinth.  music: joni mitchell, david bowie, cat stevens, john denver, mazzy star, butch walker, venus in furs, x ray spex, simian mobile disco, gold chains, junior boys, yeah yeah yeahs, daft punk, the knife, the faint, isolation years, elliot smith, ida, into another, brand new, coheed & cambria, audioslave, metallica, black sabbath, aerosmith, scorpions, iron maiden, skid row, beau nasty. not at all ironic, any of these.  food: i'm pretty much vegan and try to eat healthy. besides the aforementioned predilection for orange soda. i could eat thai food every day. burritos, too.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: anything by octavia butler, a language older than words, geek love, middlesex  movies: lots of dorky tv shows; xena, lost, bsg, and movies that make you feel good but that dont perpetuate lots of fucked up shit, and possibly secretary (but i would love to talk with you about how the role of consent effected yr feelings toward the movie)  music: hip hop, some metal, -- an eclectic list, really. on my ipod you might find the coup, regina spektor, missy elliot, some metal/rock, tupac, and erika badu.  food: anything vegan. i looove food.', 'diet'] = 'vegan'

In [18]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: nature and madness (paul shepard), beyond boundaries (barbara noske), 100 years of solitude (gabriel garcia marquez), a language older than words (derrick jensen), oryx and crake (margaret atwood), the world without us (alan weisman), this list could go on for quite some time...  movies: closer, chasing amy, v for vendetta, american beauty, fried green tomatoes, living out loud, ratatouille, the planet earth series, wall-e, network, the fifth element, b-grade horror films.  music: genre-wise i enjoy electronic, pop, instrumental, alternative, rnb, and pretty much anything with a good beat. specifically, i enjoy ratatat, zero 7, air, thievery corporation, all american rejects, postal service, imogen heap, and frou frou. i'm also on a major lily allen and jason mraz kick right now.  food: i reside somewhere in between the worlds of vegetarian and vegan. as for types of cooking, i love chinese, indian, ethiopian and thai (you can never have too much pad thai). i also kind of have a love affair with rice vinegar and soy sauce. there's a gallon of soy sauce in the kitchen to prove it. oh, and the smell of sesame seed oil gets me hot.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "reading favs: naked and more, nancy drew, kite/runner, corrections, city of thieves, marching powder, thousand splendid suns, a long way down, hope for the flowers, bukowski, the catcher in the rye, where the wild things are, still life with woodpecker, bust mag, fips (blog), cookbooks; urban vegan, i am grateful-cafe gratitude cookbook, design books, construction books, national geographic, apartment therapy blog, nytimes/nymag...  to name a few: neon indian, animal collective, arcade fire, roots, morning benders, lcd soundsystem, nerd, rip slyme, de la soul, spacemen 3, mike relm, tribe called quest, stooges, tipper, the black keys, mos def, autechre, phoenix, orb, orbital, aphex twin, sufjan stevens, buke & gass, deerhoof, -ziq, yeah yeah yeahs, joy division, velvet underground, underworld, squarepusher, amon tobin, sts9, silversun pickups, the rapture, ratatat, pixies, nina simone, nkob, helio sequence, neal diamond, modest mouse, black keys, little jackie, lily allen, led zeppelin, beck, jt, junior boys, the smiths, flaming lips, bowie, ccr, coco rosie, broken social scene, bjrk, atlas sound, ave d., beach house.  food: almond butter, raspberry jam, plain yogurt, oatmeal, bananas, mangos, chunky peanut butter, fish tacos, burritos, green curry, rice noodles, palak paneer, tikka masala, vermicelli noodles with imperial rolls, shrimp spring rolls, almonds, yogurt balls, cranberries, wasabi peas/crackers, humus, coconut milk, sriracha sauce, yum, tamari.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm really good at reading the first half of a lot of books (damn add). authors i love: salman rushdie, malcolm gladwell, bill bryson, pearl s. buck, rohinton mistry, vonnegut, steinbeck.  the music i like is expansive. girl singers, hip hop, led zeppelin, bluegrass, loud and fast. bands with lots of instruments and lots of energy. anything i can dance to or sing along with. i don't have a problem with music snobs, but i really can't hang in conversation centered on the latest underground stuff that only a few supercool people have ever heard of.  i'm somewhere between vegetarian and vegan, but entirely avoid imposing my opinions on others. i love food. also on the list, good beer and cheap whiskey.  i haven't had an actual tv for years, but let's be honest... computers are the new tv. i'll get as hooked on crack as the next girl. if you know what i mean.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: no short answer possible - i moved boxes and boxes and boxes of books across the country - and have just kept buying more since. i tend to love science fiction & fantasy, but also read business books & a very eclectic mix of other books. lately i\'ve been catching up on graphic novels and even starting to buy some comics as they are published, but this changes often. i always have at least one or two books in my bag with me at all times. plus usually a few weeks of new yorkers.  movies: until the end of the world is my favorite film of all time. i\'m a serious film fan - though i don\'t get out to as many films as i would like to any year (my fantasy is to "do" a serious film festival at least once a year and watch dozens of films...). other films i really love: most of charlie chaplin\'s works (which i was lucky enough to project on film while in college), most films by wim wenders, the princess bride (can quote it practically from memory), serenity (really this list can go on and on and on - there are about 10 films out at the moment i really want to see and many more coming later this year - 2009 has been a great year for film)  music: i have very eclectic tastes. if kcrw (santa monica public radio) plays it i\'ll probably like it. current favs include: school of seven bells, moby, kinky, m.i.a., dj shadow, zee avi but my tastes and collection are pretty varied - i occasionally enjoy playing dj (well programming an itunes playlist) at friend\'s parties). to give this some context, my itunes library has about 14,000 tracks (seriously) many of which i haven\'t heard since ripping them but still my tastes are very diverse. i love live music - but don\'t get out to it as much as i\'d like - whether an opera, rock, or a singer songwriter in a cafe.  food: okay how do i answer this? short version - i write a food blog. long version: sf: serious coffee places (ritual, four barrel, coffee bar, epicenter cafe and of course blue bottle); serious foodie hotspots (ferry building & farmer\'s markets, mission st food, zuni cafe, radio africa kitchen, sebu) but i\'m also often found trying small hole in wall places in far out neighborhoods of sf (and other cities i visit)  continuing on the food topic - i love nearly all types & styles of food - if done well, preferably with seasonal ingredients & local producers. but i\'m equally passionate about great street food as i am about a great tasting menu from a fantastic chef.  my own cooking lately tends to be very seasonal, lots of vegetable dishes (including many vegan) usually with meat from my local butcher\'s shop (or the farmer\'s market) i don\'t cook as often as i would like - mostly because cooking is for me a social act - i much prefer cooking for others than just for myself (and yes, i\'ve occasionally thought about being even more serious about cooking).', 'diet'] = 'anything'

In [19]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: salman rushdie, v. woolf, short stories and whatnot published by mcsweeney's, tim wise, i've been reading a lot of non-fiction recently, which is ok  movies: i own a few movies. just the ones i think are worth watching over and over again: i heart huckabees, the science of sleep, shortbus, the holy mountain & other jorodowsky (dude is so weird), kill bill vols 1 & 2 (these were weirdly a gift i gave to the guy in the pic of me clambering who went to the peace corps, so he gave them back... take that sentence structure!). charlie kauffman is insane and a genius, michel gondry is a genius and insane.  music: this is hard, not sure why. albums: dark side of the moon (pink floyd, but you should know that), z (my morning jacket), yoshimi battles the pink robots (the flaming lips); genres: rock (as evidenced above, all types), a smattering of hip-hop, general top 40 bs (i'm not ashamed to admit this); a bit more of alt-country and ben folds, bluegrass: john hartford. musics you've never heard of (or you have): wahfffles (rip), the guntown kid, j-stiles (a nickname i've given someone i've never met, but for some reason feel i at liberty to nickname)  food: i love food. i'm trying to get better at cooking it, but i don't have a lot of patience. formerly a vegetarian/vegan, i'm enjoying all foods now, but usually don't spend a lot of money on meaty fare (old habits die hard)", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "the books i like are chic lit, mostly. i am taking forever to finish up the twilight series, but i was only really interested in the first book, the rest have been a disappointment, but not as much as the movie! i also read, then watched, slumdog millionaire- should have watched the movie first.  my movie likes are judd apatow-related. anything with seth rogan, paul rudd, and the like. aside from a good comedy, i like wes anderson, foreign and independant films as well. amelie, stranger than fiction, kiss kiss bang bang, shopgirl, sin city, the fifth element, some of the pixar movies, sin city, juno, lots more.  music is very important in my life and there is a range of stuff i enjoy. i like local bands a lot (although most of them have now broken up), some popular hip hop, bossa nova and catchy stuff. favorites are flight of the conchords, incubus, jason mraz, fall out boy, panic! at the disco, gym class heroes, the avett brothers, the beatles, deathcab for cutie, mars volta, modest mouse, black eyed peas, the shins, neutral milk hotel, the weakerthans, amy winehouse, and lots of others.  not currently watching tv except for the office, glee, project runway, top chef, and the vampire diaries. in my dvd collection- smallville, jon and kate plus 8 (i don't want to talk about it), flight of the conchords, dark angel (i'm in love with jessica alba), viva la bam, freaks and geeks, arrested development, and nip/tuck.  as far as food goes, i think i would marry a bowl of good asian noodles, if it weren't just a conspiracy to eat it's entire family. and i'm totally into pastries, obviously. i could never be vegan or even vegetarian.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'it is difficult for me to choose favorites of things... as many things inspire/challenge/comfort/energize/humble/interest me. that is not to say i don\'t have tastes or ideas or opinions... but i suppose i\'m quite "open" per se...  i love to learn... and i think all of the above mentioned expressions and creations are potentially excitable. that being said... books. i am generally reading about 10 books at a time, and never quite get to the end of any of them... generally in the non-fiction realm (topics ranging from hiv/aids, mental health/illness, mindfulness, vegan nutrition, activism/organizing, etc). i think i could really use a good fiction book these days tho, any recommendations?  movies. at first i was going to say that i am embarrassed by the lack of movies i have seen... but actually i\'m ok with it. it usually turns into my friends threatening to keep me in for a whole weekend for a movie marathon of the classics, but this has yet to happen. once in awhile i get around to the local independent theaters for something that catches my eye and to some of the various sf film fests.  music. i like hearing/seeing/feeling it more than listing a bunch of artists and bands. lets go to a show or curl up under a blanket and listen and touch.  food. love going to the farmer\'s markets.... i\'m vegan..... but no preaching or judgment if you are not.  teach, share, and show me things, and i will do the same for you.', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "food: i love both northern and southern indian, i have a soft spot for thai, generally love spicy food, and am slowly getting acquainted with brussel sprouts. i am vegetarian and enjoy learning new and delicious recipes, or just hitting up souley's for tasty collard greens and vegan mac-n-cheese. i dig trying new food stuffs.  books: anything tolkien, anything steampunk, the moon and sixpence, pop-science, academic science (brain rehabilitation, selective attention, machine learning) current read: last chance to see by douglas adams and mark carwardine  movies: anyone who knows anything about me can vouch for the fact that i don't watch enough movies for even a basic pop-culture reference. movies i have seen and enjoyed: anything miyazaki, the prestige, monsters inc., 2001: a space odyssey, back to the future, count of monte cristo (yes the 2002 one, it's a guilty pleasure)  music: nobuo uematsu, spanish guitar, air, uakti, new order, the bird and the bee, stereolab, george winston, imahori tsuneo, joe hisaishi, bela fleck, tom petty, travelling wilburys, radiohead, 311, alanis, david gray, manu chao, jamiroquai, kylie minogue, scissor sisters, basement jaxx, ryksopp, trance, and cheesy electronic  video games: although i haven't played many lately, i'm including this category because i think it can be equally helpful in contributing to my character. morrowind, okami, ffvii and ix, shadow of the colossus, and the myst series.", 'diet'] = 'vegetarian'

In [20]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm musically illiterate ... never bought music, memorized artists or associated them with songs. since i know what i like, pandora has been perfect for someone like me. i do not watch or follow sports. i eat all kinds of food but am not a foodie ... keep kosher at home and eat vegetarian out. i'd like to read more novels and look forward to joining a great books club ... in my retirement. i'm trying to listen to audio books while working out.", 'diet'] = 'mostly anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "my absolute all time favorite book, barron's pass key to the gmat.........  food: everything from alligator to caimen, to ostrich, to sushi to cuban, to kimchee, bok choy, jamaican beef pies, to most types of steak, and fish, any latino style foods, salvadorian, etc., vietnamese, congolese and kenyan, ethiopian, american meat and potatoes, irish stew, polish & russian delicatessen, most everything kosher, if its middle eastern, its definitely on the menu, chinese any day of the week, and italian for sure", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: roman history (livy and tacitus mostly), a. s. byatt, jhumpa lahiri, harlan ellison, dashiell hammett, most myth and folklore, and the brontes. i read almost anything that crosses my path, though.  movies/shows: coen brothers movies, casablanca, half-baked, serenity, titus, bridesmaids. i am inordinately fond of the terminator movies, even the bad ones. i loved party down, but i generally can't stand embarrassment-type comedy. i never watched either version the office, and i have no plans to. i was a huge fan of the dearly departed friday night lights, but i have no idea why the writers couldn't come up with some believable plots for poor julie taylor.  music: johnny cash, mos def, rilo kiley, soul coughing, verdi, ella fitzgerald, the pogues, sonic youth, etc.  food: indian, chinese, mexican, thai, puerto rican, vietnamese... while i can be picky about certain ingredients, i'm willing to try almost anything that doesn't have tomatoes, mushrooms, or onions. i can't stand pork or ham, but i love bacon. who doesn't? besides people who keep kosher, i mean.  i mean, how do you pick just one thing (or fifteen) that defines your taste?", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i don't really believe in favorites, so most of the below are things i've read, watched, or listened to recently that i liked.  books: i like most of what i read. i am looking for recommendations now though- fiction mostly since i've been reading a fair amount of doctoring non-fiction recently.  movies: chinatown, i am sam, pirates, pans labrynth, mar adentro  music: paul simon, van morrison, fleetwood mac, adele, florence and the machine, mumford and sons, the boss, u2, black eyed peas, blackalicious, bob dylan, eric clapton, bebe, shakira, manu chao, mirah, bon iver  food: fresh pasta, figs, homemade pizza, ice cream, anything made out of potatos, sushi, (costco) kosher hotdogs, goat chesse, chinese, thai, indian, ethiopian, burmese...pretty much everything. i don't eat red meat except for those costco hot dogs i mentioned. i realize that doesn't make sense.", 'diet'] = 'mostly anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm a bit of the foodie. i like gluten-free cooking and i'm very good at it. at the same time, i also like vietnamese, thai, japanese, chinese and kosher/jewish cooking.  just to name a few types of cooking variations. i like to enjoy good food with good friends and great conversation. i'm a bit of the health nut. the diet that agrees with my body the best is gluten-free. however, i love exploring new restaurants with in the bay area that are new or have received good reviews from critics. if you're interested in joining me for a meal send me a message.  in terms of movies, i'm a hopeless romantic and i like chick flicks and fairly decent romantic comedies or dramas. i'm not above taking a date to the movies. i have been known to cry at movies.  in terms of tv shows, i like gray's anatomy, private practice. in terms of my music tastes i like all kinds of music. jazz, r&b, soul, a bit of top 40, and classical.", 'diet'] = 'kosher'

In [21]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "- the wind-up bird chronicle (really, anything murakami) - ender's game - my utmost for his highest (revised language edition, since i can't understand old-school oswald chambers)  - l.a. confidential - almost famous - freaks & geeks - love connection  apparently, i really like indie music. this has only been confirmed by staying at houses where people only play jazz or are 62 years old and we have no common musical tastes outside of the beatles and bob dylan. but i am not really on the cutting edge of anything so i probably don't have any new music to share. however:  - i love the middle east (australian group) and recently was introduced to tortoise and tunng. fan of both. also, i normally don't like anything too folky/twangy, but i am always comforted when i listen to mumford & sons.  - eggs, sunny-side up - kosher salt is yummy cause it's kind of sweet - big fat blueberries - crispiness in texture, savory sweet in taste, anything fresh", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "perhaps this ongoing list of random information will help exhibit my personality and quirks...  i often think of witty (what i consider witty) band names i hope to someday use when i quit my life as an attorney and become a rock star. so far here's the list:  kosher pork dirty soap erlenmeyer flask wiggum dry sponges the pedophiles (we'll be dressed as priests) arms surveillance prinskipple sinner bob sacamano mcgarnicle mingey lola & the purple haze sweet sassy molassey cookey coocoos  ..... if you can help add to the list let me know.", 'diet'] = 'kosher'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "perhaps this ongoing list of random information will help exhibit my personality and quirks...  i often think of witty (what i consider witty) band names i hope to someday use when i quit my life and become a rock star. so far here's the list:  kosher pork dirty soap erlenmeyer flask wiggum the pedophiles (we'll be dressed as priests) prinskipple sinner bob sacamano mingey  ..... if you can help add to the list let me know.", 'diet'] = 'kosher'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'born and raised in hawaiiisland boy at heart... 70f and above is my game- i hate the cold, but i always seem to live in cold climates...  if i had a bucket of things to do before tomorrow i would finish it...i don\'t sit around for things to happen... if there are potatoes in the bucket....well than i will see a bucket of peeled potatoes that need to be fried...  like michael jackson says..."it\'s as easy as a, b, c easy as 1, 2, 3"  if you prefer to stay home than go out than say so, if you want to eat jack in box instead of taco bell tell meif you would like to go next door and tip the neighbors cow...than what are you waiting for?  like most guys say...i love long walks on the beach, clean kosher music, and living the life....take each day one shell at a time....', 'diet'] = 'kosher'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i work at a san francisco startup trying to change the world. i\'m one of the worst jews i know; i get halfway through my double-bacon cheeseburger, look at it for a moment, and think to myself, "that\'s probably not kosher... awesome."  i listen to a lot of music in many genres; bad techno, hip-hop, alternative, a little bit of everything. (i have this theory that there is no such thing as "good techno", only different levels of "bad techno". daft punk, for instance, is among the best of the bad techno. )  i\'m interested in math and music, in computers and culture. i\'m really into micro-fiction and web shows.  i am looking for a new show now that battlestar galactica is over.  i am adventurous, adaptable, and alliterative.', 'diet'] = 'kosher'

In [22]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == "i live in the beautiful san francisco bay area, california. i'm currently into my second year of an originally 6 month tour of mexico, which i've extended indefinitely. i keep saying i'm going back soon, cuz i gotta keep a foot planted in the bay. eventually, i want to split my time between both places.  i gotta say, i've fallen head over heels with mexico. just last year, i was kidnapped at gunpoint in mexico city by the midget faction of an obscure drug cartel called los gigantes whose plan was to extort uncle sam for big bucks in exchange for my release. if the government refused to pay, the midgets would chop off my head. i was forced to sit in a windowless room for thirteen days with nothing to do but watch telenovelas through a feverish delirium brought on by a diet of swine flu laden street tacos (i'm sure this can't be kosher). by day fourteen the cartel finally realized that the $7 price of feeding me for two weeks was costlier than holding a brown american for ransom. i was worth about as much as the amero would be at a conspiracy theory convention. so they let me go.  ok, i made that up. but why is it that everyone in the states keeps telling me that that's gonna happen?  anyway, i completed a six month spanish language program at a funky art school in san miguel de allende, and now i've got projects that keep me busy but i also do a lot of chillin'.  i wear many hats and have multiple passions. i'm deeply involved with social, environmental, & food justice. i have a deep passion for gender equity, and i am an unabashed fan, supporter, cheerleader, lover, guerrilla fighter and all around badass for women...which means giving up my bullshit, stepping up, and playing big. the macho crap's played out, fellas.  i love working with my hands. i own a business designing and growing organic veggie gardens and have taught urban veggie gardening in bay area schools.  i used to be the quintessential bay area workshop junkie, and now i love coming up with my own and teaching them - such as japanese cooking, permaculture gardening & massage courses. i've done media and communications strategizing and consulting for grassroots organizations and non-profits developing pressure campaigns to take on big, bad corporations and politicians doing shitty things in the world. love it!! modern day david & goliath work. feels amazing to do an honest day's work by sticking your thumb in the eye of an evil corporation and getting paid for it!  i love to cook & i especially love to cook for friends. i show love by sharing food. food brings us together. it's the foundation of life.  update: i've been creating pop-up restaurants in san miguel. i can't believe how much fun i'm having. i get to mix my pleasure of entertaining guests, cooking and making a little money. what i've done thus far: vietnamese, indian, japanese, mexican, middle eastern/mediterranean. next on my list: ethiopian (but i'm a little intimidated to make it).  i am a lover of the senses. i am a massage therapist and bodyworker. i believe that the pathway to divinity is through the body. touch is my passion. it builds more empathy and connection with our fellow human beings.  i spent the first four months in mexico teaching organic veggie gardening & permaculture to elementary and middle school aged kids at a montessori school here in san miguel. i developed and taught a delicious, natural foods cooking course here. mmmm....que rico! i co-produced my second tedx conference here in san miguel. check it: http://tedxsanmigueldeallende.com/en/  currently, i'm building out a rooftop veggie & flower garden and bedroom for my friend and teaching a workshop on eco-architectural design. update: built, planted and growing.  here's something crazy about mexico. most middle-class educated folks here make about $4,000 pesos a month, which is a little less than $400 u.s. dollars. an a**hole gringo on a tourist visa can roll into town and make $500 pesos per hour doing massage, which is around $45 dollars. which is exactly what i did.  i live in a resort town, and demand for good massage is high. the problem is, is that the local mexican massage therapists generally aren't trained well and are paid low wages. so, a gringo like me comes along and takes their job. the spas love this set-up, because they mostly cater to english speaking clientele, while the therapists are left with no power, no real skills, and are easily replaceable.  so, i decided to start a massage program where local mexican women can learn advanced level bodywork, high-end luxury spa techniques, make great cash, and be so good at what they do that they can roll into any resort city in the world and be the a**hole taking someone else's job.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "sudoku. i use a pen.  people say i'm a pretty good listenter.  when i get to know someone pretty well and feel comfortable, i can be pretty funny.  i am also on a quest to cook the perfect steak. ribeye, iron skillet, kosher salt, fresh pepper, real butter, garlic crushed by hand, and a really good thermometer seem to be the most important ingredients. i also have a good vegetarian dish and an asian chicken with lemon and cilantro that people like a lot.  oh yeah, i'm pretty good at nerdy stuff like math, physics and computers. some people like the way i play the blues. i do a pretty good tom waits impression, and i can do ray charles but only in the shower. i play classical recorder and clarinet too; it'd be fun to play duets.", 'diet'] = 'mostly anything'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "currently working in public health. what's your wild rabbit? outside of work, i enjoy anything active whether it be an organized sport like slosh... err, softball, or a ride through the city. just built up my first bike and can't wait to soak up the sun... let's ride! workouts in the gym are my weekly staple, but during the weekends i enjoy a change of pace such as basketball, maybe some crossfit, or just anything outdoors. rock climbing? maybe even some roller skating. i'm open to anything as i like challenging myself from time to time. camping is a must in the spring/summer, maybe even winter despite the frigid temps. yosemite? tahoe? maybe somewhere in the southwest this year. halal chicken and yellow rice with white sauce.", 'diet'] = 'halal'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: the great gatsby. neuromancer. slaughterhouse 5. enders game. pygmalion. do androids dream of electric sheep? a dirty job. shade the changing man. the sandman. transmetropolitan. astonishing x-men 1-25.  movies: stranger than fiction. vertigo. crank 2. spirited away. moulin rouge. eternal sunshine. blade runner. trainspotting. mean girls. strange days. the fifth element. kill bill. gattaca. party monster.  music : neutral milk hotel. pavement. morrissey. saul williams. oasis. placebo. rem. weezer. sufjan stevens. nightmare of you. suede. pulp. dashboard confessional. the beatles. bjork. bob dylan. the cure. kt tunstall. matisyahu. nickel creek. regina spektor. suede. tori amos. vampire weekend. yacht.  foods: japanese. thai. halal. vietnamese. italian. chocolate.", 'diet'] = 'mostly halal'

In [23]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "well... in no particular order...  i'm a: vegan feminist anarchist polyamorous submissive  i love: roadtrips found art getting pierced morbidly obese animals and, of course, my partner banana_compadre  my idols: leigh bowery john waters marchesa luisa casati edward gorey cyndi lauper hunter s. thompson  i am playful, thoughtful, and demented", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i am a social worker, care about how people treat each other and the planet, passionate about alternative forms of medicine, classical music, wine, and geology. looking for a conscious relationship, respectful of process as well as content. affectionate, a good listener, vegan into cooking healthy, organic farmer's market veggies, and going to the gym.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "adventureous, experimental, travel, dj, electronic musician, grad school sometimes, respectful, organic, live, foodism , sound system culture, sustatinable living, biodiesel, vege/vegan, die hard new wave/goth/alternative/post-punk/synthy80's/weirdo  i am adventerous, searching, and weird", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i am an armenian/welsh mix transplant from another continent living in the mission district of san francisco. i have gotten to know a lot of my neighbours and really like the feeling of community in the the mission.  i come from a very supportive and loving family and enjoy their company though i live far from them and do not get to see them that often. community and friendships are really important to me. i am not materialistic and don't consider shopping to be a pass-time or recreational activity with the exception of farmer's markets.  i love the outdoors, hiking, backpacking, rock climbing, cycling and birdwatching. i have a good sense of humour and like to make people laugh. i really enjoy cooking - i am vegan but don't let that deter you, some of my best friends are omnivores! sadly i am allergic to cats so being around or living in a house with cats is not an option for me. i also enjoy gardening, enjoy growing food and permaculture. i am very passionate and tactile. big fan of kissing, cuddling and spooning.  not looking for a boyfriend as i have one. i am open to meeting new friends & outdoor activity buddies.  disclosure i do not own a car and i am part of a dying breed of humans who do not own a cellphone or an ipod. to clarify, i don't consider the ownership of these items an issue per se, it has more to do with how they are used, particularly cellphones - i have very low tolerance for bad cellphone manners. being present without constant interruptions from an electronic device crying out for attention is important to me. okay, enough about cellphones...", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i just moved back to oakland after being held prisoner in the south bay for the last year and a half, and, boy, am i happy to be back! and while i love san francisco, i am definitely an east bay girl at heart.  i have two small businesses that i am very passionate about. i do small scale metal work, printmaking, and i paint. i love to make stuff. i hike regularly, do yoga a lot, love camping, reading, cooking and zoning out in the sun in my backyard listening to my music kinda loud. i also just planted my first vegetable garden and am enjoying, really, really enjoying, watching it grow :) i have, what i consider to be, a fantastic backyard. i'm counting the days til summer and fresh veggies, fruit, sitting out in the sun, and lazy barbeques with my friends.  while i am an omnivore at heart, i try to eat a mostly a vegan in my day to day life, but have a hard time sticking to it. when i'm good, i'm good but when i'm bad, well... my dog doesn't approve at all but otherwise we're very close. i love graphic novels (preacher and sandman being my favorites), movies, traveling, tools, and spending time with my friends.  i'm looking to meet someone who is actively pursuing the most amazing life that he can imagine. someone who is well-balanced and funny, smart, motivated, loving, and sexy :)  i am fiesty, imaginative, and super loyal", 'diet'] = 'mostly vegan'

In [24]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "if i could spend all my money on art i would.  i worked at a couple nurseries growing flowers during high school...i love gardening...i built some raised beds last year and grew lots of vegetables but i'm no opinionated vegan organic tool (although i think its good to eat organic)...i actually think genetic modification can be good.....now instead of waiting tens of thousands of years (if not millions) for new species of flowers to be created, we can make our own new ones....well as long as they are not invasive or grow up to be like little shop of horrors...it ok.  i originally went to school for chemistry at berkeley...but ended up with a history degree...my concentration was the middle ages...can you say the invention of lying..... i spend most of my time making/producing/recording music out of sf", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'hi! i would like to meet someone who i can be happy with, believe in, and trust. somehow i managed to become a pediatric rn. it\'s kind of a strange fit for me. i originally went to school for "wilderness recreation", so i have a degree in how to go camping, basically. i work with some fantastic people but have never felt that i have much in common with them. the job itself can be rough. i think my dream job would be working a video store. i least that\'s what i\'m usually thinking when i am running around at my current job. i\'m fairly responsible, caring, considerate, thoughtful and generous. i feel like i\'ve had some bad luck in my previous relationships, but i\'ve accepted that that\'s life and i\'m still optimistic. i know there is someone out there for me. i\'ve lived in berkeley for about 5 years. i grew up in vermont, and went to school in upstate new york. i turned 35 today, july 2. i\'ll have to change that tomorrow. now i have to remember that....i\'ve never been married, or have had kids. it\'s weird that i feel like i have to mention that. i look and act a lot younger than my age, and my lifestyle solidifies that. i\'m a jeans and t-shirt kind of guy. i\'m not looking to start a family anytime soon, but i do want that eventually. i\'m fairly healthy. i love to cook from scratch. i like to be outside and active. i take a swim class 3 days and week, and enjoy biking, hiking, camping and backpacking. i play ultimate frisbee and frisbee golf. i bike when it\'s close enough. i don\'t follow sports. i like to read. i watch tv, but by no means am i a couch potato. i enjoy nature. i appreciate a sunny day or a full moon. i won\'t howl at or anything, i just think it\'s worth taking a second to look at it. i notice when the birds are chirping on my block. i love cats. i have a 15 year old black cat. i\'m into girls who like to look good but don\'t wear a lot of makeup or high heels on a daily basis. i could listen to npr all day. my favorite shows are this american life and wait, wait. i really like praire home companion but it also gets on my nerves. my favorite pandora station is rogue wave and the sea and the cake. a few more things: i keep my apartment clean. i have a car and love to explore. i\'m disease free. i\'m 420 friendly but don\'t smoke. i\'m an atheist. i used to be a vegan but am mostly vegetarian these days.', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "health is very important to me. i eat mostly vegan/vegetarian. i love cooking and learning new recipes. i never eat fast food (except chipotle). i love hiking, rock climbing, racquetball, table tennis, pool, any water sports, being outdoors, hunting for wild mushrooms and always enjoy adventures.  i love to travel, i'm often on a plane at least once a month for business or pleasure or both. ]. i've been to india, ecuador, italy, germany, switzerland, the netherlands, canada, mexico... can't wait to visit every country on earth!  i enjoy my amazon kindle - it's very practical since i don't have enough bookshelves to store all my books. i plan on buying a house in the tambor region of costa rica.  i believe life is what you make it. why waste a precious moment with regret? why not live your dream, do what you love to do, and have fun? why not love with all your heart? last time i checked, you've only got one life - make it count.  my values include love, integrity, contribution, courage, fun, adventure, and learning.  i'm committed to excellence. i'm not afraid to make mistakes and i'm not at all a perfectionist, i just want the best out of life.  i have a blog. come check it out http://www.tomcorsonknowles.com/blog/", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i\'m looking for a lover with whom i can share my heart and my life. in addition to the adjectives listed above and with a good match, i\'m passionate, loyal, intelligent, affectionate, and sexy. i\'m also open to making good friends with people who have time in their lives to make a true connection. i\'m vegetarian / vegan with some attraction to a raw vegan diet.  i am open, honest, and (com)passionate.  i\'d like to choose "queer" instead of "gay" on okcupid\'s sexual orientation menu, but it\'s not there yet. ;-)', 'diet'] = 'vegetarian'

In [25]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "im fun and fun to be around. i love to experience a new relationship. i love labs and birds! i have to have a guy tht does not smoke or drink. i live in the bay area. i am a vegan who would love to be surprised by a guy who can cook a great vegan meal. i hope that i can find the right guy to spend the rest of my life with. come check me out.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "perfecting gluten-free vegan baking. trying to figure out how to make that a career and what i want to be when i grow up... life is all about exploration and finding new interests. i thrive on constantly learning new things. i also really enjoy teaching other people. ready to learn?", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "perfecting gluten-free vegan baking. trying to figure out how to make that a career and what i want to be when i grow up... life is all about exploration and finding new interests. i thrive on constantly learning new things. i also really enjoy teaching other people. ready to learn?", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "my current time expenditures:   finishing 7x7's lovely list of 100 places to eat before you x_x  learning to be vegan so i can obtain psychic powers  scouring san francisco for the best taco trucks  newest quest: playing the guitar  reading & writing & searching  searching for slow loris  doing a head stand  freestyling limericks  dancing in the rain  taking pictures  coding ninja  spelunking  laughing  learning  creating  p90x * bbq *<3", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "working to pay rent. reading books. going on bike rides. going to dance parties. listening to records. practicing yoga. cooking yummy vegan meals. working on websites and computer programs. hanging out with friends. wandering around san francisco. going on adventures in the woods and by the ocean. not taking myself seriously. attempting to take life one step at a time.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "my intentions are to move through this world as lovingly and authentically as possible.  -----------------------------  my dad said that i always supported the underdog as a little kid.  years of riot grrrl and labor activism turned into 9 years working in the non-profit world in the bay. at my last job, i stumbled onto graphic design and fell in love, starting my own graphic design and photography studio. it has been wonderful to combine my love for design and social justice.  2011 has been my super active year and i realized just how much i need activity for a healthy mind and body. lately this means going to the gym, shooting hoops, playing on multiple softball teams, and playing around on my skateboard with friends.  other things im doing with my life include: enjoying the sporty dyke mode that i'm currently in, learning to be more spontaneous, making connections with people, doing photo shoots, archiving negatives and whisking up delicious vegan and now gluten free desserts.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == 'i am excited about developing my work, chilling with my family and the many other yummy people in my life, and exploring creative pursuits. i am a somatic psychotherapist, and i also practice a kind of touch therapy i am calling mindful bodywork. i am particularly excited when i meet people interested in touch and energy play with its many variations. i am from the east coast, and from what i\'ve heard people aren\'t surprised when they learn this. however, my 20+ years in the bay area has softened me around the edges. i have been identified as a "yoga-doing, burning man-going, spiritually curious, bisexual, polyamorous, vegan." friends say i am much more down to earth than people might think after reading that list.  in addition to my private practice bodywork and therapy clients, i work with at-risk youth at a non-profit media agency. in the past i did more marching; nowadays i do advocacy in quieter ways, and it has become part of my work life. years ago i got my mfa, wrote a book (got complimentary rejections from large publishing houses!), and taught creative writing. i used to read a lot, and even though that hasn\'t been my focus recently, i still love to talk about ideas and books i want to write/am writing. i love to be creative in daily life and in making art projects when i can.  i like hanging out with others who dance, stretch, cuddle, play, go hiking, listen to music, notice things, read, write, experience, engage, play with energy of various kinds...and tub. i love to go razor scootering around oakland and sf, walking (city or nature) and discovering interesting new places, and i would like a companion in those. i also love to discuss whether music sounds better on vinyl, the best way to get places, and i love to guess the time (i tend to be fairly accurate). i like to discuss the impermanence of our earthly stints and our relationship to "subtle energies" and at the same time i enjoy everyday talk too.', 'diet'] = 'vegan'

In [26]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == "i own the vegetarian health institute. we teach people how to thrive on a vegan or vegetarian diet. in the last year, about 800 students have taken, or are currently enrolled in our vegetarian mastery program.  i'm also an award winning songwriter. i perform my positive message songs at unity centers and centers for spiritual living. in the last year, i did music ministry at centers in orlando, new york, seattle, and santa cruz, as well as the immediate bay area.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "am enjoying all of the unexpected opportunities that have come my way, feeling grateful for that. work as environmental director and want to invent a new product. hopefully the tech ideas will find the right platform soon. otherwise, i enjoy being outdoors, planning fabulous vacations, cooking fabulous meals (budding vegan chef) and then eating fabulous meals with my good peeps!", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "performing improvisational dance shows, opening, working ethically helping people to become enlightened, sacrificing myself to open others, being myself, changing all who open, eating vegan, not doing drugs, not eating sugar, being efficient, loving myself, caring for life!, not producing children.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "by day i work at a vegan/organic restaurant. by night i am a talented and dynamic theatre artist. in between? i dance, i like to take a ride on my motorcycle(hopefully somewhere where i can get outdoors and take a hike or even go camping), i listen to npr in the morning. i see my friends and relationships as a reflection of myself and an opportunity to learn about myself. i want to help the world while having a good time i.e. eating organic avocados and then dancing my but off or go climbing the calories away at the gym.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "working on going back to school, currently grillin vegan food", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "i studied baking and pastry major at the art institute of california- san francisco. i want to eventually own my own cake shop specializing in vegan and gluten free treats! right now i just sling vinyl, cds and dvds.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "job searching and moving. so much moving. getting healthy. working out 4-6 days a week. going to therapy. joining a sleep study. i'd like to find activity partners for distraction such as: artistic collaborations. web comics. dancing (especially at bootie). ddr. drag shows. fighting for gay rights/basic human rights. forming an amateur dance troop. karaoke. massive pillow fights. playing dress up. vegetarian/vegan/raw food. sharing bob stories/tips. zombie runs. festivals. renaissance fairs.  i recently graduated from a sex ed training program in sf. yay me!", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "over-complicating things with creative fervor, (this is equally an immeasurable skill, a glorious gift, and a horrible curse.) cooking gluten-free/vegan treats, making epic messes, listening, affirming the amazing qualities of others, making things out of metal, giving compliments, coming up with crazy, seemingly insurmountable ideas and manifesting them into reality, confronting things i'm scared of, finding/creating laughter in tense situations.", 'diet'] = 'vegan'

# After reading all outputs from this for loop, I could confidently say that if the string containted vegan, the diet was 'anything'.
# a lot of times users used the word 'vegan' to express their distaste for vegan diets
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains('vegan'))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "writing and singing songs, making raw vegan dishes", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "tinkering. i am constantly taking things apart and trying to put them to other uses. currently, i've been spending my time focusing on these little black computer chips and making them make homemade speakers pop and whistle. i someday hope to make a moving synthesizer that people can interact with.  i've been told i am a good listener-- i can't remember who said it though or in what context:)  i have a biting wit. however, i am not an angry person.  i can cook. i bake tasty, hot bread and can mix a mean gibson and grill up a hell of a steak. yup, i'm not the guy for a vegan but i do eat tons of veggies.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "cooking (i cook mostly vegan at home - i'm vegetarian), being funny, listening to people, being honest, being supportive. taking walks around san francisco and discovering new neighborhoods. being neat and organized but not to a manic point - of course, this point is different for everyone :)  ummmm what else? returning phone calls, waving people in while i'm driving, always carrying an umbrella if rain is expected, calling my mom regularly, always tipping 20% in restaurants, not hitting the snooze button in the morning, being kind to animals, drinking 6 glasses of water every day, eating well, exercising regularly.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "(this is his partner stepping in! he's really good at frankensteining audio equipment into cool new configurations, finishing my lunch, being a gracious omnivore in the company of vegans, knowing more about queer female culture than most lesbians, making cumbia playlists, covering the house with rusty metal things, learning farsi greetings, kissing, and being a helpful friend.)", 'diet'] = 'anything'

In [27]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay2'] != np.nan) & (ok_cupid_df['essay2'].str.contains('vegan'))]['essay2'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay2'] == essay, 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay3'] == "smart, competent, strong (sometimes a little too much so), and a little weirdness are the things people generally see in me. i used to be described as aloof and stand-offish - now i'm often told that i am social and friendly (personal growth!). apparently i look like a vegetarian lesbian (although this seems to be increasing in strength, people are starting to think i'm vegan). i eat meat and date guys though. but people have been assuming this since i was 12, so apparently thats what i look like.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i love books! i have stacks of books, and i wish i had the time to read all of them. recent reads: fight club, the diamond age, (reread) ender's game, bringing down the house, the creative mind  i don't watch many movies but i do like gattaca, contact, the big lebowski, pulp fiction, fight club, boondock saints, hot fuzz and office space.  i am pretty adventurous with food. i don't mind vegetarian/vegan food, but last time i tried being a vegetarian i stopped healing from bruises. didn't mix well with the karate.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'a) the terror (dan simmons), hells angels (hunter s. thompson), the alienist (caleb carr), born standing up (steve martin), the world according to garp (john irving), ragtime (e.l. doctorow), songs in the key of z: the curious universe of outsider music (irwin chusid), to kill a mockingbird (harper lee), women (charles bukowski), our band could be your life (michael azerrad), high fidelity (nick hornby). harry potter. calvin and hobbes. the works of neil gaiman, douglas adams, daniel m. pinkwater, phillip k. dick, henry rollins, roald dahl, james ellroy, kurt vonnegut, robert crais, robert b. parker, tony hillerman, dave barry, carl hiaasen, preston/child and elmore leonard.  b) the works of the coen brothers, wes anderson, quentin tarentino, nicolas roeg, george romero, david cronenberg, stanley kubrick, jim jarmusch and robert altman. old horror movies with vincent price, or christopher lee and peter cushing. what else, high fidelity, eternal sunshine of the spotless mind, la confidential, better off dead, so i married an axe murderer, the usual suspects, shaun of the dead, airplane!, tremors, donnie darko, the man with two brains.  c) i agree with duke ellington, who said that there are only two kinds of music: good and bad. this is good music: tom waits, neil young, the velvet underground, led zeppelin, flipper, david bowie, swans, melvins, low, ac/dc, ramones, naomi hall, sonny sharrock, can, mos def, jurassic 5, einsturzende neubauten, black flag, meat puppets, flaming lips, frank sinatra, bobby darin, outkast, lightning bolt, medicine, sonic youth, sigur ros, david bromberg, beatles, buddy holly... this could take a while.  lately, i\'ve had a serious thing for psychologically damaged singer-songwriters (daniel johnston, jandek, skip spence, syd barrett, nick drake, roky erickson, tim buckley, wild man fischer). and to this day, i love "schoolhouse rock." all silly gen-x nostalgia aside, it\'s just great pop.  d) i can has cheezburger?  so not a vegetarian, let alone vegan. i love indian, new orleans cajun/creole, italian, burgers an\' fries. i won\'t eat mushrooms (unless i\'m gonna see a bunch of crazy crap about an hour later) or "variety meats." one of my favorite things about california is the abundance of cheap and excellent mexican food. i like cooking, but i\'m not great at it, and know pretty much nothing about it. we\'d be a much better match if you\'re the great cook and need an obedient kitchen helper (i can mince garlic like nobody\'s bidness, if that\'s any help). though i can manage a serviceable shepherd\'s pie, or a decent jambalaya.', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay5'] == "1) wayne coyne's tweets 2) music at work 3) french music...!magnifique! 4) gluten free cupcakes from gfb 5) independent movie theaters 6) stereomood.com 7) my daily vegan protein shake (even though i'm not a vegan, but not eating dairy is a big part of my daily diet)", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay5'] == "champagne red meat (i was raised a vegan) sunshine dna true friends the ocean", 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay5'] != np.nan) & (ok_cupid_df['essay5'].str.contains('vegan'))]['essay5'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay5'] == essay, 'diet'] = 'vegan'

In [28]:
ok_cupid_df.loc[ok_cupid_df['essay6'] == "right now - it's what to eat. i'm being 98% vegan, as a way to force myself to learn how to cook a less animal based diet. this is harder than it seems at first, especially when outside of san francisco. we are really spoiled here.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay6'] == "turning one dollar into two dollars.  the genius of.........doing too much  spatial mental math + patterns + associated thoughts.  strata  semiotics  where the fuck are my glasses?  is everyone on okcupid vegan/vegetarian?  how at the end of the day, no one is really unique, we are all the same, the point is to find someone else that gets the same punchline as you.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay6'] == "1) third wave feminism and the roll of masculinity within that perspective. 2) renting a studio to keep all the rusted things i find riding my bike around the ship yards 3) how i wish my unicorn neck tattoo was real 4) getting a dog (i find myself on pet-finder more then okcupid and facebook combined) 5) how oreos are vegan 6) what i will do in the fist few days of the apocalypse... in case you didn't know, what you do in the first few days can greatly improves your chances of survival.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay7'] == "there is no such thing as a typical friday night. but here are some recent ones in no particular order: - learning how to cook some vegan dishes and actually wanting to eat them (i'm a carnivore, this was a surprise to me). - experimenting with a form of dancing that mixes one skating dancer (me) with a non-skating swing dancer. - hanging out at a hackerspace learning about electronics. - composing a song with a friend based off a poem from lotr. - having a quiet board games night with family.", 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay6'] != np.nan) & (ok_cupid_df['essay6'].str.contains('vegan'))]['essay6'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay6'] == essay, 'diet'] = 'vegan'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay7'] != np.nan) & (ok_cupid_df['essay7'].str.contains('vegan'))]['essay7'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay7'] == essay, 'diet'] = 'vegan'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains('vegan'))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'diet'] = 'anything'

In [29]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "you believe in god and love vegetarian/vegan food, and especially if you're japanese. ^_^", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay9'] == "you want to grab some sightglass coffee, go on an urban hike, and eat vegan donuts overlooking the port.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay9'] == "you are polyamorous and lookin' for love.  if you're vegan and looking for someone to share meals with.  if you like to get high, party, and dance.  if you like to chat on internet relay chat (irc) or aol instant messenger (aim).  if you like to trade massages.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay9'] == "you're into vegan baking...just kidding! if you feel like there's some sort of connection and potential for fun, joy, laughter, meaningful conversation or adventure. vegan bakers are also welcome to contact me :)", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay9'] == "you read a newspaper ... you have a positive attitude about life ... you want a great vegan cupcake recipe.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay9'] == "if...you're nice, if we like the same stuff, if you will cook me vegan food, if you have a beard, if you are an alien who will whisk me away from this dreary planet in your spaceship, if you don't mind that i might not answer but it won't be a personal thing at all, if you're an interesting man/woman/mineral who would like to reach out to another carbon-based lifeform. also--if you think we could be friends. no such thing as too many homegirls/boys.", 'diet'] = 'vegan'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay9'] != np.nan) & (ok_cupid_df['essay9'].str.contains('vegan'))]['essay9'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay9'] == essay, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i love living in oakland and the bay area in general. i travel a ton, and play music a lot. i am vegan and love making amazing food with friends!  i like people whom are straight forward and honest.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'ignorant. complacent. helpless. these are all words that describe things that i do not want to be. i push myself to be better and when i come up short, i try to improve so i don\'t repeat the same mistake.  i\'m not looking for the mystical "soulmate" or the "one", because i know that doesn\'t exist. you don\'t have to be a supermodel to date me. i\'d just like to find someone who has common interests, a woman who is my friend as well as my lover, a person with whom i can laugh, and cry with as well as make love. i don\'t think that it\'s too much to ask. i am a mindful and nonjudgmental person just looking for a like-minded girl to share his time with.  one thing i would like from a partner is a passion for health and fitness. i am a vegan and i constantly engage in physical activity and exercise to maintain strength and energy levels.', 'diet'] = 'vegan'

In [30]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "socially awkward, often. always i like dogs. and cats. history. dinosaurs. plants. gardens. libraries. building things. architectonics. friday night symphonies. saturday night punk shows. sunday brunch. i'm vegan. andro. butch at heart. but i don't like appearances answering all questions, so sometimes i like eyeliner together with heels and drywall. i love a picnic and picking berries. on the beach. i make a mean pie. scone. lasagna. safe room. treehouse.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i always hate describing myself sometimes i tend to make myself sound bland which i don't try to do on purpose, i'm one of those guys that would like to be be judge on the quality of work that i do or in other words let my work to the talking for me. first things first i'm a vegan but i don't care if you eat meat, it really doesn't bother me if you eat meat until you push your meat beliefs on me! i'm a audio engineer i do studio recordings but as of late have been mixing more often than recording and i do sound at 924 gilman...so as you can see i'm always surrounded by music i have a wide variety of styles that i enjoy. i just got back into doing indoor boulder climbing, i also enjoy bike rides around town...if you can geek out on audio then you have some bonus points! =]  i am thinker, creative, and oddball", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "any given day you can find me rolling about town on my much loved bicycle, singing or grooving to the latest tunes i can't live without on my ipod. i adore cycling, running, hiking-just being outside in general. i recently went skydiving for real and it was amazing-once i got over the fact that i just fell out of a plane. i've become quite the avid cook, and you can often find me scouring the internet for recipes to try or that might inspire me, or floating about the local farmer's markets. i'm a vegan but am not a proselytizing one, and if you ask my friends, some of the food i cook you'd never know it was vegan. i've recently been delving into the world of vegan desserts, and according to my housemates(i have 6 and they're fabulous), that's going well. i love the parks and beaches, own and have previously longboarded, although it's taken a back seat to cycling. i love rollerderby, and have a fantasy of becoming one of those sassy derby girls. i just like to be out experiencing new things. i am always a glass-half-full type of person, and am ready for anything the universe is ready to throw at me. i love learning in every capacity. i work at a local cafe part-time and have gone back to school to become a social worker. i like volunteering and giving something back to the community. i love a good tune, a good movie(i'm especially partial to chair-arm-grabbing horror films and the cheezy flicks), and dancing. if i need to i don't have any problem going out by myself to dance. i think animals are phenomal, would love to have a puppy but it's just not in the cards for the moment. i love people who are comfortable with their nerdy side, as i am mine.  i am compassionate, fun, and amazing", 'diet'] = 'vegan'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("i'm a vegan"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'okay, as for books, i\'ll confess right up front that i got caught up in a couple of series aimed more at kids, due to reading/listening to them with my son- these include the harry potter and inheritance series. more recently, i\'ve gone on a bit of a health and fitness kick, and read books including "the primal blueprint" by mark sisson and "why we get fat and what to do about it" by gary taubes, which led me to discover a way of eating that works for me for health and weight maintenance, and it- gasp!- includes eating plenty of meat, so if you are a vehement vegetarian or vegan, we\'re probably not super-compatible.', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i am vegan, so i cook for myself and friends often and prove that vegan food isn't all sawdust and cardboard.  i like most movies, but really dig sci fi, action/adventure, and documentaries.  musically, i prefer classic rock, blues, jazz; however i am open to most music except rap, hip hop, and top 40 bubble gum.", 'diet'] = 'vegan'

In [31]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm currently in love with the poet anne carson. i haven't started stalking her yet but it may just come to that.  big freedia + nina simone, monica <3's brandy, rufus luvs neil young, kid sister & missy e. (gf's), tupac and dead prez (bf's), debussy n' britten (f buddies), dead mau5 xoxo lead belly, jeepneys=tlc  love food! i'm vegan except when i'm eating you out. or eating cookies and cake cuz those don't really count right?", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i am vegan.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "bookwise, i'm a sucker for memoirs. i usually cruise the biography section at libraries and leave with a stack or two. as i write this, i'm reading bruce moody's will work for food or $, peach friedman's diary of an exercise addict, and malcolm gladwell's the outliers. which isn't a memoir, but you get the idea.  my favorite directors are david lynch and david cronenberg (which probably says a lot about me right there), and i've recently been getting into andrei tarkovsky. oliver stone used to be among my favorites, but he lost me about a decade ago, though his rather obscure talk radio changed the way i look at movies.  when i start dating someone, i'll often show them videodrome, existenz, eternal sunshine of the spotless mind (also good for towards the end of the relationship) and punch-drunk love. (even if they don't like the movies, it still tells me a lot.) which isn't to say i'm just into the heavy stuff--i'm also a sucker for live-action 3d movies, and host a weekly show where we make fun of bad movies, 'cuz it's fun.  in my car, i'm usually playing an mp3 cd which consists of low (my favorite group), r.e.m., richard and linda thompson, sheryl crow and lucinda williams. at home, however, i'm usually listening to ambient or noise music through headphones--i used to a radio show consisting of that sort of thing--and i have drone music playing above my bed. i also love anything that makes me want to get up and dance, and have been slowly getting into techno and such.  and though i'm vegan, i'm all about sushi.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == '(a) my favorite books: the solitaire mystery, fluke, life of pi, anything by kurt vonnegut, ceremony, catcher in the rye, a hope in the unseen, harry potter, fight club, the illiad, anything by chuck klosterman, the lord of the flies, the little prince, east of eden, a walk to remember, holes, the giver,  (b) music: prince, empire of the sun, friendly fires, counting crows, outkast/andre 3000, ne-yo,daft punk, pink floyd, pm dawn, cake, earth wind & fire, red hot chilli peppers, 2pac, lady gaga, eminem, kid rock, michael jackson, new edition, babyface, avril lavigne  (c) movies: clerks, fight club, juno, the iron giant, 25th hour, o\' brother where art thou?, goodwill hunting, the return of the jedi, american beauty, the lion king, hotel rwanda, x-men 2, teenage mutant ninja turtles, transformers: the movie (the one from the 80\'s).  (d) food: i\'m vegan, sort of (i eat fish, so i\'m apparently i\'m "vegaquarian"). but i love the food that i eat. i do miss cheese, though.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "look i tend to read a lot of pop culture,to name a few the great shark hunt, high fidelity, sex drugs and coco puffs, fight club ( unfortunately ) i recently ordered theres a reason why i drink david cross women charles bukowski eating the dinosaur chuck klosterman skipping towards gomorrah dan savage  listen i have gone through so many phases as a teen now i listen to all kinds of nifty stuff like ....the descendents, harlem, sleighbells, rites of spring, crown of thornz, the cure,the smiths, the wipers, the mean jeans, butthole surfers,jonathan richman/modern lovers, state property, pavement, ghostface,the nerves black sabbath, krs one,black flag, the replacements,e-40, sheer terror, screeching weasel ,gang of four ,beach house, mission of burma, gangstarr, freshkills, martha reed, james carr, violent femmes, radio birdman, slick rick ,7 seconds(old stuff only please) ,anthrax (uk), outkast , the deathset, japanther, the ramones, fugazi, the templars,zounds, dj shadow, david banner, judas priest, dj morsy, cheap trick, modern english,pulp,cro-mags,ride,superchunk, the lone rangers i also find myself enjoying stupid mainstream radio stuff when the time is right .  observe waynes world , royal tenenbaums,dog day afternoon ,airheads,deliverance,idiocracy, thank you for smoking, 25th hour, arrested development , the wire, king of the hill,home movies, 30 rock, daria, south park,curb your enthusiasm,  food i like all kinds of food preferably spicy, i'm vegan one day and cooking lamb the next. i could go on but honestly if we do not have anything listed above this section in common we are more than likely not going to get along based on the fact that we both like thai food right?", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "currently in heavy rotation are; kraftwerk, dfa 1979, soulwax, muse, bloody beetroots, crass, leadbelly, diplo, vitalic, rammstein, captain ahab, foot village, marklion (of dat politic), blectum from blechdom, etc.  i realized recently that two of my favorite movies both star meatloaf. i'm currently unsure if i should be proud or ashamed of this fact. additionally, i love the world of apu by ray, the three color trilogy by kieslowski, and anything where steve buscemi appears.  i don't have one particular favorite book, but i love reading and making time to read books. currently i am reading theories of international politics and zombies by drezner, and i've recently read taliban by rashid, winston's war by dobbs, and the ware tetralogy by rucker.  i'm a vegan. i have been for a number of years, but i cook anything, including meat for my friends. i fancy myself a rather decent cook. i tend to cook with a french or californian influence, so lots of sauteed or grilled vegetables, stews, or pasta dishes.", 'diet'] = 'vegan'

In [32]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("i am vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "favorite books are scifi and fantasy, i really like all the star wars series novels anything written by anne mccaffery or robert jordan the dune series and dragonlance all amazing. movies i have really simelar tastes as my reading selection except i really like comedies. music is jam bands: grateful dead, string cheese incident, sts9, bassnecter, camo cowboys, blue turtle seduction, i really enjoy dancing at really grimy dubstep parties . food i am a vegetarian pretty strictly i eat sushi a few times a year as its normally ends up being a good compromise for fancy business dinners. i would like to stop eating cooked food altogether and go raw for a bit but i am not quite ready for that yet.", 'diet'] = 'strictly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("i am a vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "currently savoring books about the way our brains work.  i enjoy movies where underdog sports teams prevail (especially if based on true story). if the fifth element is on, i will always watch it. and i like amelie.  my ipod is full of melancholy chick music from the grey's anatomy soundtrack. with a sprinkling of hip tunes i got from my sister. currently diggin' oren lavie. well, just the one song with the awesome music video.  food: i am not a picky eater. though, for a few reasons i am 2/3rds vegetarian. meaning, i'm vegetarian for 2 out of 3 meals a day. for the third meal i'm just a raging carnivore.", 'diet'] = 'mostly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("i'm vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i\'m pretty particular about movies/books/ and food  quirky interesting drama/humor //some cult classics// wings of desire/the wim wenders version// pulp fiction/ big lebowski// anything lush and visual//dreams by akira kurosawa deadman w/ johnny depp//  my musical tastes are all over the map // from buena vista social club, gotan project, leonard cohen, kruder & dormeister to david byrne, louis armstrong, cassandra wilson, ray charles, willie nelson  i tell people "no, i\'m a "qualitarian" when asked if i\'m a vegetarian. if it\'s well prepared with love and attention, i will probably try it and like it //especially if it\'s spicy!', 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("i'm a vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books, fiction: science fiction and high fantasy. china mieville, neil gaiman, iain banks (with and without the m.), george r. r. martin ("and the knights who say fuck"), guy gavriel kay are my current faves but i\'ve got some old school niven and heinlein on my shelves as well.  nonfiction: lots of general sciencey reading. hofstadter - geb has informed a lot of my thinking. also into military history; i hate war but i am fascinated by military technology.  tv: battlestar galactica, firefly, buffy, dexter, weeds, the wire - enjoyed them all, but with a critical eye. star trek > star wars. tos > tng. for lighter fare, community, 30 rock, and the entirely fucked-up gravity.  film: lots of coen brothers works. the princess bride, la femme nikita, aliens, and chasing amy, and i\'m prepared to defend that last one.  music: i like 80s-90s industrial, goth, post-punk, weird electronic music and am trying to train a computer to make weird electronic music for me. kidneythieves, the prids, the damage manual, devo, big black, skinny puppy, scorn, peter gabriel\'s soundtrack music, brian eno, joy division, gary numan, meg lee chin and more.  food: a little of everything, but let\'s be honest: i really love me some noodles. i eat meat but can happily eat vegetarian meals.', 'diet'] = 'anything'

In [33]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("eat vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'widely varied-- from bjork to 80s, bowie to bartok, a taste of opera to a smattering of punk. books include chabon, rice, shakespeare, and shaw, movies is also as open as possible-- cocteau, merchant ivory films, superhero themed, but sucker for romance-- i love fine food, though a regularly eat vegetarian and seafood cuisine-- but when in rome . . .', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: cookbooks, yoga books, sex books, repair manuals, pronoia by rob breezny, days of war nights of love by crimethinc, the ethical slut, no logo by naomi klein, after the deluge by chris carlsson, repair & technical books  movies: any that's entertaining or that i can learn from. really bad hollywood movies that you can make fun of the whole time, one of my favorite movie watching experiences was watching 2012 with a snarky friend and mocking it all the way through.  music: balkan brass music, classic rock, electronic, punk, classical(especially when played by vordilic), dubstep(yes, deal with it), and more!  food: mexican, italian, indian, sushi, anything that's not overly processed or prepackaged(though i'm a sucker for cheez-its). i mostly eat vegetarian, but if somebody makes me a really good meat dish i'm not opposed to eating it :) i really like beer too- it's one of the things i'm sort of snobbish about, and budweiser or mgd doesn't count as beer to me :)", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: cookbooks, yoga books, sex books, repair manuals, pronoia by rob breezny, days of war nights of love by crimethinc, the ethical slut, no logo by naomi klein, after the deluge by chris carlsson, repair & technical books  movies: any that's entertaining or that i can learn from. really bad hollywood movies that you can make fun of the whole time, one of my favorite movie watching experiences was watching 2012 with a snarky friend and mocking it all the way through.  music: balkan brass music, classic rock, electronic, punk, classical(especially when played by vordilic), dubstep(yes, deal with it), and more!  food: mexican, italian, indian, sushi, anything that's not overly processed or prepackaged(though i'm a sucker for cheez-its). i mostly eat vegetarian, but if somebody makes me a really good meat dish i'm not opposed to eating it :) i really like beer too- it's one of the things i'm sort of snobbish about, and budweiser or mgd doesn't count as beer to me :)", 'diet'] = 'mostly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("mostly vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'mostly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("mostly vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'mostly vegetarian'

In [34]:
ok_cupid_df.loc[ok_cupid_df['essay2'] == "if i had to say one thing, it would probably be cooking. i wouldn't say i'm any kind of a pro at it, but i can whip together a few good things and i always find myself waking up in the morning with random ideas to remix things (steel-cut oatmeal cookies? chorizo infused hamburgers?). i grew up mostly vegetarian but i have since converted to full-time carnivore so i can grill up some good carne asada tacos (its all in the marinade). but i can't escape my roots, so i use zucchini in situations where i prolly don't need to  i grew up doing construction and working on cars, so i've got socket-wrenches and screwdrivers flowing through my veins", 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("was vegan"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: ishmael. the prophet. crazy love. 100 years of solitude. i have read a lot of classics. i'm always looking for a new book to read. movies: old boy. amelie. let the right one in. harold and maude. music: everyone's life should have at least a theme song, if not a soundtrack. i just can't seem to get enough. i listen to a lot of music. i have music playing in my house most of the time. one of my favorite past times is seeing live music. i've been to more concerts than the average person. i like all music genres. i like to turn my ipod on shuffle. i like the contrast of hearing totally different styles of music played one after the other. lately i am on a deadweather kick. food: i was vegetarian for years. i still don't eat much meat. i'm not as picky about it anymore. sushi, thai, and indian are some favorites. but i really just like food that is good for the body. fresh fruits and vegetables with whole grains etc.", 'diet'] = 'mostly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("was vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'mostly anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("was a vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'mostly anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("anything vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

In [35]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("i am vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("i'm vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("i'm a vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("was vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("was a vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("almost vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'mostly vegetarian'

In [36]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("been a vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("been a vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("no vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm optimisitic and enthusiastic/passionate about life in general, which is good because research shows optimistic people tend to live longer. and, i cite research studies in general conversation. (i don't know of any research on the relationship between longevity and nerdiness--it really could go either way i figure). i have a daily yoga practice but am not a yogi, rarely eat meat but am not vegetarian, love animals especially dogs but don't have any pets...yet. just got back from a month working hawaii and though i am happy to be back home, i do miss sleeping under the stars and getting papaya, mango and avocados right off the trees.", 'diet'] = 'mostly vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("not vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("not a vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [37]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("not a vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("i am a vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("been vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'my favorite books? geeze, ask a lifelong junkie what his favorite drug binge was, why don\'t you? okay, the handwavy author list: anything written by jonathan lethem. anything written by haruki murikami. vernor vinge. thomas pynchon. jeff noon. steve aylett. octavia butler. maxine hong kingston. hunter s. thompson. iain banks.  if you go by quantity alone, i\'ve probably re-read "fear and loathing in las vegas" more times than any other book not counting "the cat in the hat". but having confessed that, i\'d like to quickly aver that if your social group ever included that guy who wouldn\'t ever shut the fuck up about hunter thompson and seemed confused about the difference between "quoting thompson constantly while drinking a lot" and "having an actual career as a writer", you know the one? i am not that guy, promise. (also, now that he\'s passed on, can we all admit that the man was kind of a shit with some deeply creepy aspects to his personal life? it doesn\'t make "hells angels" any less fun to read, i promise.)  movies: blade runner. the big sleep. tampopo. l.a. confidential. monsters inc. brazil. my neighbor totoro, 12 monkeys. and it\'s not a movie, but the wire is better than any movie i can think of in the last 5 years.  music: blackalicious. elvis. new bomb turks. steely dan. stevie wonder. aesop rock. new model army. tricky. bjork. bach. gnarls barkley. tribe. kinky friedman and the texas jewboys. kate bush. lyrics born.  food: sushi. curry. foie gras. steak. fresh tomatoes. sushi. fresh bread. sushi. felafel. duck. duck. goose! (hm, that\'s a pretty meat-heavy list there: i should note that probably a slim plurality of the people i\'ve dated over the course of my life have been vegetarians, and it\'s no big thing.)', 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("been vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'vegetarian'

In [38]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == "for work: i tried doing nothing and while i was mildly successful at that, it became boring. i'm an entrepreneur. i start (or help start) companies. i have every intention of doing this kind of work until the day i do.  for fun: any and everything that doesn't include turning vegetarian.", 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay1'] == "now, i work in oakland for a nonprofit that advocates for the rights of medical marijuana patients and physicians, which would have been my dream job in law school, except that this job did not exist then. when i'm not trying to stick it to the man, i like to eat all types of spicy food and chocolate, or combine both pleasures at once by eating mole sauce, and playing basketball and dancing. i have also eaten snake in vietnam and kangaroo in australia, but i'm also vegetarian friendly. nothing, though, beats cambodian food.", 'diet'] = 'anything'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains("vegetarian"))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "being a vegetarian, touring", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "being a vegetarian.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay2'] == "i'd rather answer it this way - i'm extremely into fitness and health. i love working out, playing sports (snowboarding, tennis, ultra marathon running, triathlons, bikram yoga, soccer and more).  i also love cooking; i am a vegetarian and amateur chef as well as a food blogger.", 'diet'] = 'vegetarian'

for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay2'] != np.nan) & (ok_cupid_df['essay2'].str.contains("vegetarian"))]['essay2'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay2'] == essay, 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay3'] == "long, blond hair, in a ponytail. some of the things you'd expect to correlate with a guy having a ponytail are true, but others are not. for instance, i use linux, but i'm not a vegetarian.", 'diet'] = 'anything'

In [39]:
ok_cupid_df.loc[ok_cupid_df['essay3'] == 'books: this just in "i\'m down - a memoir" by mishna wolff... hilarious!  the catcher in the rye; i must have read it 1000 times. tuesdays with maurie, running with scissors, to see and see again  movies (all-time favs): the sounds of music, west side story, the breakfast club, when harry met sally, grease (i know... i\'m embarrassed for myself)  music: almost everything... but you should probably know that even in my 30s boyfriends make me music cds or playlists. that may (or may not) be saying something about how well you may like my music .  food: i love mexican, japanese, persian, greek... just about anything. i am not vegan/vegetarian but love many foods that are raw or vegan; and on the flip side a steak or burger can be the highlight of my day!', 'diet'] = 'anything'

ok_cupid_df.loc[ok_cupid_df['essay3'] == "a:to say nothing of the dog, breakfast of champions, perdido street station, burger wuss, frek and the elixir, lizard music, wicked, the fermata, holy fire, the art of deception, blood music, of mice and men, the handmaiden's tale, scorch, son of the circus, expiration date, zodiac, etc. i read widely if not always well. i can read trash for days and still get something worthwhile, even if it's just my own entertainment  b:repo man, donnie darko, bad santa, logan's run, jackie brown, la dia de la bestia, the seven samurai, the chronicles of narnia, the dark backward, akira, amelie, waking life, three o'clock high. even more so than books, i can really immerse myself in bad movies.  c:the pixies, devo, eugene chadbourne, mountain goats, foetus, public enemy, full moon partisans, neil diamond, johnny cash, the cure, tom waits, rube waddell, nick cave, run dmc, magnetic fields, dead kennedys, pidgeon, wesley willis, circle jerks, sleepytime gorilla museum, x, negativland, evolution control committee, desmond dekker, the eels. i could go on, but generally i like nearly everything, and have irrational dislikes, of just about everything too.  d:pickles, pupusas, circus peanuts, rice porridge with preserved egg, chirashi, salads with almost anything in them, falafel, curry in most forms, squid, kim chi, eggs, potato pancakes, unagi, burritos, radishes, cheese, cucumbers, gefilte fish, and when i fall off the vegetarian wagon, rare roast beef and sausages.", 'diet'] = 'mostly anything'

ok_cupid_df.loc[ok_cupid_df['essay5'] == "people - i am deeply enriched by my family, friends, lovers, collaborators, colleagues, street performers, cashiers, chance encounters, casual carpool drivers, the homeless guy who hangs out under the highway on my morning walk to work, and young children and teenagers. animals - i have two cats, would love a dog someday, and i like most bugs. i'm totally not a vegetarian though. not even at all. i don't feel bad that animals die; i think it's wrong how we treat them while they're alive. travel - my next big trip out of the country: brazil for carnival. fingers crossed. sex - and thank god i now know that this one belongs on the list. food - a barely cooked piece of salmon, a raw red bell pepper, a spoonful of peanut butter - i fucking love food. being active - politically, physically, mentally, emotionally - i want to shake the world.", 'diet'] = 'anything'

In [40]:
ok_cupid_df.loc[ok_cupid_df['essay5'] == "apparently pork, for one thing. i love me some carnitas, bacon (as seen above), ham, pancetta, proscuitto, char siu, siu mai, hom yu gee yuk beng, pulled pork, roast pork, kalua pork, pork belly, tonkatsu, tonkatsu ramen with char siu, chicharones, liempo, you name it. dammit, i could never be a vegetarian. i love animals 'n all, but god shouldn't have made them so delicious! i do have some veggie friends and out of respect for them i try to refrain from going caveman on a suckling pig in front of them.  okay...my friend just sent me this link: http://www.bbqaddicts.com/blog/recipes/bacon-explosion/ i like my pork products n' all but this is making my stomach hurt. bacon explosion. it must be named after what will happen to your colon if you eat it, and i've seen a colon or two in my line of work, i tell you what. thankfully i haven't seen any that have exploded....yet.  i'll keep thinking of the other five....", 'diet'] = 'anything'

In [41]:
ok_cupid_df.loc[ok_cupid_df['essay5'] == "my peeps my toys my gadgets my food (i love meat, not every person with indian origin is vegetarian you know)", 'diet'] = 'anything'

In [42]:
ok_cupid_df.loc[ok_cupid_df['essay5'] == "1) his ridiculous iphone 2) the blogosphere 3) tofu, etc. (vegetarian here) 4) ironically trashy dance music 5) appropriately cozy living spaces 6) super serious conversations about super important ideas", 'diet'] = 'vegetarian'

In [43]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay5'] != np.nan) & (ok_cupid_df['essay5'].str.contains("vegetarian"))]['essay5'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay5'] == essay, 'diet'] = 'mostly vegetarian'

In [44]:
ok_cupid_df.loc[ok_cupid_df['essay6'] == "what i'm going to name my next cat (branciforte if a boy, famke if a girl? also in the running: consuela). psychological theory and how people heal. upcoming possibilities. how to maintain hope in america's future when so many americans are so stupid. great tv. whether i am actualizing the potential of my youth. school. health. the godlike power of habit. why vegetarianism makes people so defensive. sex. where i will be in a year - in every sense. people i love, and why.", 'diet'] = 'anything'

In [45]:
ok_cupid_df.loc[ok_cupid_df['essay6'] == ". the way music affects my mood. . guessing people nature from first look(and mostly wrong! ) . if someone were watching us from far far away, would they find us funny, silly, entertaining, motivating. . shuttling between my love for meat and love for animals. i have been vegetarian for almost 10 months now. . go for startups or completely switch my field!", 'diet'] = 'vegetarian'

In [46]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay6'] != np.nan) & (ok_cupid_df['essay6'].str.contains("vegetarian"))]['essay6'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay6'] == essay, 'diet'] = 'anything'

In [47]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i have to say that i hate categories, i was taught to enjoy all things in life and filling out all this information seems to limit ones perception of what one likes and dislikes. hopefully i will make up for that here.  for the most part i like outdoor activities like camping, hiking, skiing, biking, walks. however i dislike doing most of those things alone... which in no way implies i can't stand being alone, just that i like company better. things that i like doing at home range from reading, cooking, music, messing around with computers. i have a tv, but the reason that i got one was to watch movies. i can be shy, but most people wouldn't describe me as that. i have been sitting on the fence of being a vegetarian (i never eat anything that has 4 legs anymore), but that doesn't mean that you have to be. i don't really believe in astrology, but i am an earth boar and a taurus... and some of the descriptions of those, that i have read, seem to apply to me. my taste in music is pretty diverse, at any given time i've got almost anything in my cd player. for the most part my reading is in the realm of science fiction and fantasy. but, as i get older i would say that i am branching out.  i grew up in a family of humor and pranks (eg: my grandfather lit firecrakers under my grandmothers chair and then laughed so hard at the effect that he could hardly stand). i say hello to the jack on the antenna of my car (yes, i know... a psudo vegi with a culture icon of anti vegi comercialism... what can i say...), have a thing for parenthesis, and have been known to say goodbye to the answering machine voice....  with respect to you, to me what is important is that we get along as friends. that two people that have a physical attraction for each other is significant. but... if that is all there is to it then that does not a relationship make.  if you think you can be friends with someone like what i have described, then send me a note or tell me you would like to get together.", 'diet'] = 'mostly vegetarian'

In [48]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("eat anything"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

In [49]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains("eat anything"))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'anything'

In [50]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay3'] != np.nan) & (ok_cupid_df['essay3'].str.contains("eat anything"))]['essay3'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'anything'

In [51]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == 'the stooges, the ramones, magnetic fields, johnny cash, sonic youth, hsker d, philip glass, abba, cocteau twins, liszt, the breeders, broken bells, the evens, arcade fire, harry belafonte, david bowie, john coltrane, patti smith, puccini, link wray, bruce springsteen, m.i.a., mozart, madonna, mtorhead, the meters, the minutemen, nick cave, the pogues, patsy cline, new order. chris burden, louise bourgeois, frieda, fluxus, trimpin, gieger, mil-spec. kurt vonnegut, van jones, bell hooks, jg ballard, jd sallinger, rick poyner, mike davis, joseph campbell, clay shirky, linda williams. film theory dork. live music buffoon.  mild steel, aluminum, occasionally stainless, aspiring to copper. corsets.  f2600, xr100, klr650, gs650 dakar (polaris thumper models). ktm curious. very much hoping to traverse eastern europe, cuba, and south america with 2 wheels & a motor, someday.  food: no dairy (moo-juice or goat). allergy: bad things happen. not into gamey meat, but would like to hunt someday (tho i\'d probably burst out crying in heaving sobbs of guilt if i killed a critter). i went vegan at 11, and after almost 20 years w/o meat and w/ lots of chinrubbing i have no quams about ethically owning my place in the food chain. i never pretended to like tofu and am rather proud of that. i continue to refuse to eat anything that qualifies itself as "sprouted" or is made from ingredients better suited to make rope or paper. legalize it, but my goodness- be civilized with it. i digress...', 'diet'] = 'vegan'

In [52]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("eat anything"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [53]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains("eat anything"))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'diet'] = 'mostly anything'

In [54]:
ok_cupid_df.loc[ok_cupid_df['essay8'] == "i have cattle in 2 african countries. my herd is growing. mostly because i am predominately vegetarian and don't eat my cattle, which is a winning strategy for ending up with more cattle.", 'diet'] = 'vegetarian'

In [55]:
ok_cupid_df.loc[ok_cupid_df['essay8'] == "--i'm single. --i'm not into drunken one-night stands. sober daytime trysts are way sexier. --i'm healthy. --i need sex for my emotional and physical health. (so do you) --my basic successful date would start with a cup of herbal tea, followed by a delicious organic vegetarian lunch, deep, soulful, respectful conversation, hugs, and plans for future adventures. on some occasions, kinky passion. --things just seem to work better for me with women who are submissive people-pleasers behind closed doors, but i cannot tolerate a woman who is not self-defined otherwise. --i'm really focused on my life and career, so i can't do the talk every day on the phone thing. i seek special, quality time. --i go to a lot of art and music events, and it would be fun to have companions. if they are lovers, even better. -i'm seeking a woman who's perceptive enough to recognize my strength through my gentle, humble, quiet persona. a dom need not be arrogant. -i'm creative and i know what i want.", 'diet'] = 'vegetarian'

In [56]:
ok_cupid_df.loc[ok_cupid_df['essay8'] == "i'm a fashionista hippie (and a bit of a cougar) and a vegetarian for 25 years!", 'diet'] = 'vegetarian'

In [57]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains("vegetarian"))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'diet'] = 'anything'

In [58]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == 'we\'re a good match or if you fit this description(though not subject to the entirety of it): a sense and appreciation for things away from the mainstream, latino/hispanic, about two to three-years-older than me, masculine (or just not feminine), educated, artistic/creative type(you get extra points if you\'re a good musician!), possibly vegetarian, straight-edge/sober, good self-esteem, good head on their shoulders, level-headed, a good balance between humorous and serious, and bisexuality is a plus. generally speaking, i want a guy who appreciates the same sort of things i do...  please don\'t contact me if you\'re past your mid 30\'s! also, i don\'t take kindly to up-front flirting on first contact, so that means no messaging me addressing me as "cutie," "sexy," "handsome," or any of the following. i will blow you off if you do so.  and it\'d be nice if you respond if i try messaging you. its actually quite irritating when someone just looks at your profile and not reply to a message, like really. this is a social website, and its not like you\'ll ignore someone who\'s talking directly in front of you.', 'diet'] = 'vegetarian'

In [59]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "you like vegetarian food, you need a top-roping partner, you distinguish between your and you're, you like to harmonize (poorly) to radiohead songs, you're a fluent spanish speaker who's willing to practice with me.", 'diet'] = 'vegetarian'

In [60]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == 'you find humor in (almost) everything, love to explore, and appreciate good food. you like to play, in many senses. if you enjoy long, rambling conversations with lots of non-sequiturs and spontaneous laughing. and sentence fragments.  you\'re pro-choice and for the right of everyone to marry.  and you had better know the difference between they\'re, thair, and o\'r thar. oh, and it\'s. and the correct way to use "myself." and the right one from peeked/piqued/peaked interest. (http://abstrusegoose.com/151)  it would be good if you are an omnivore, but my cooking can still make you swoon in vegetarian.  you want to convince me of the superiority of a good backscratch over a good massage. or the opposite.  if you are more than 90% my enemy by the okc rating, we must meet.  if you find that you frequently utter the phrase "if i told you, i\'d have to kill you" (or its ilk), please do not contact me.', 'diet'] = 'vegetarian'

In [61]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == 'you are a caring, grounded, and open-minded person with similar values who is interested in developing a meaningful connection. especially interested in meeting people who have an interest in mindfulness/meditation, are willing to eat vegetarian (at least some of the time), and who want to contribute to the world in some way.', 'diet'] = 'vegetarian'

In [62]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "...you're a person of color. ...you get that we need some serious change and power shift in this world! ie, you consider yourself progressive to left of progressive...just don't think it would go anywhere otherwise... (extra points for actively engaging in movements for change) ...you like outdoors and being healthy (or aspiring to); you love music, dancing; you're sweet affectionate and a good kisser (let's be real, what's the point otherwise?), and please, only if you got a good sense of humor (aka laugh at 90% of my jokes; so if you haven't laughed a few times already, don't even bother! ;).  oh, and u gotta be able to hang with a laid-back vegetarian.", 'diet'] = 'vegetarian'

In [63]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay9'] != np.nan) & (ok_cupid_df['essay9'].str.contains("vegetarian"))]['essay9'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay9'] == essay, 'diet'] = 'anything'

In [64]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "currently in a relationship which we're 'working on' or from which we're 'on a break'-- either way, i'm currently without a large pool of friends due to geographical/financial constraints and the fact that up until quite recently, a lot of my time was taken up by said relationship; however, i have temporarily relocated.  i'm not looking for dates, the structure of my life doesn't facilitate that. honestly, the 'structure of my life' is hard to define, except that it isn't very stable at the moment (for more legitimate reasons than being a party in a vaguely-defined whatever-it-is relationship). i have some things that are major distractions from personal growth; that being said, i'm really trying to focus on myself. i don't mean to be rude by not responding to many messages. i suppose i don't feel like leading people on (by implying that i would be up for hanging out without being a flake) but would like to meet new friends and hear others' points of view on life. so, basically i am okay to have online communication, but beyond that is not very doable at the moment.  i consider this to be a great time to redefine my direction in life...  i know i am on the more 'hippie' end of the spectrum as far as my beliefs and opinions are concerned. i am not well enough versed in yoga or buddhism to claim to be a devoted follower but they're definitely where it's at, as far as i'm concerned. i am learning more about them, and believe wholeheartedly in the benefits of meditation.  i'm 80-90% vegetarian/vegan at this point. inspired to transition after learning about which foods do what sorts of things to a person's overall health, but it sits well with me (to say the least) that it doesn't support an unethical animal-farming industry.  i really like art/anything creative. i also have a lot of interest in things highly logical and enjoy reading scientific publications. i haven't quite defined a career goal, but have considered things like writer, yoga teacher (even though i'm presently about as much of a novice as i could be), therapist.", 'diet'] = 'mostly vegetarian'

In [65]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'hi, my name is lizzy. i am single, queer, and sooo open to bringing in both dates and new friendships!  i am currently completing massage school, and will soon be a certified massage therapist :) i enjoy all sorts of intuitive and spiritual topics and would love to meet other queer ladies who have interests in these areas as well.  other things i enjoy include travel, hiking, adventures, meditation, good food--especially raw and vegetarian cuisine (i do not discriminate based on what you prefer to eat--no worries ;)  i am interested in developing more authentic, real connections with people, and i also like to have a lot of fun! i would love to hear from you :)', 'diet'] = 'vegetarian'

In [66]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "my name is cristal i'm genderqueer and i'm on okcupid to make friends. i love cooking vegan/vegetarian food. i work a full-time job that tends to have a lot of downtime. so i'm usually trying to figure out the sudoku puzzle in the examiner or people watching. on nice sunny days i'm usually at delores park smoking a bowl or drinking a beer. i enjoy riding my bike and taking muni.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm a chill, friendly, and loving person. i sometimes live a life out of a magical realism novel.  some basic demographics: i'm a nerd, a (vegetarian) foodie, a world traveler, a libertine, a news junkie, a meditator, and someone with an irreverent, possibly insolent sense of humor. actually i'm possibly just plain weird.  my general philosophy in life: try anything once and try some things several times. i love novelty, experiencing new things and people and new ways of looking at the world, and understanding people and their different walks of life.  i like maple butter, massages, beaches, parties, nuzzling, extra spicy food, and strange mystical states involving elder gods and pipe smoking salesmen.", 'diet'] = 'vegetarian'

In [67]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "hola muchachos de okcupid. i find it a little challenging to summarize who i am in a few paragraphs (we are all unique and complex after all). so...... here's a quirky fact about my personality - i still love and listen to music from my high school/college years while running on the treadmill to drown out the all too often cheezy tunes blasting from the gym radio. i'm more of the arty guy who has a love for bikes (i'm not referring to motorcycles), animals (especially the ones we keep as pets), pescetarian/vegetarian cuisine and a lot of the guys with profiles on this site thus far (wink wink).  i am grounded, friendly, and eccentric", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i\'m a collection of conundrums.  i\'m a hippie/yuppie who works hard, parties hard, chills hard and wants to save the world and himself while having the most fun possible. this manifests itself though dancing, riding my bike around town, building websites/companies, eating delicious vegetarian food, throwing parties and traveling.  some math: experience > possession sf > everything else < london < nyc heart = mind 80% supportive, 20% critical  for the demographers: california dude: cupertino, stanford and sf occasional burner 30yo, 150lb, 5\' 10" man.', 'diet'] = 'vegetarian'

In [68]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'recently employed away from a life on the road traveling the california festival circuit. berkeley resident and love it. vegetarian, camera geek, mac enthusiast, bike rider, wet shaver, thai food eater, rock climber, burner, modern gentleman / outdoorsman.', 'diet'] = 'vegetarian'

In [69]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "life is a journey and the experiences of life make us who we are today. i would never change the paths i have chosen, both good and bad. i want to continue to grow and express myself through my music. i appreciate art, love vegetarian food. music not just as inspiration but, i believe that music can change the world...  i love cooking delicious vegetarian/vegan food. especially, but not limited to, thai/indian fusion or asian inspired dishes or anything delicious. mushrooms!!!!  life is too short to waste. every breath should be lived to the fullest. don't sweat the petty stuff, pet the sweaty stuff :d  i am passionate, romantic, and i love music", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'my names are chris hughes, tristissima, flurp, stacey everheart, grok hussein amiri, et alia laughing and weeping, wizard lizard, princess teacup, and all the names i have not yet acquired. it\'s a magickal thing. if you\'re curious, just ask.  i am a pagan pansexual polyamorous genderqueer and i\'m actually much more normal than that sounds :-) i perform slam poetry. and, for the sake of okcupid\'s interest matching, i have to say it again :-) slam poetry. there :-) barefoot coffee roasters is my home away from home. i have a b.a. in latin and religious studies from santa clara university.  i have asked my high priestess in the temple of inanna and dumuzi to kibitz as she believes necessary on my profile. here is the first: be prepared for some awesome poetry.  here is more: tristissima is committed to ecopolitical justice and to the daily personal ethics s/he perceives as inherent therein (for example, s/he is an ethical vegetarian, while not presuming to judge others who are not). s/he is a cultural radical, but not a slacker--s/he\'s dedicated to making a real difference in the world, and following the advice of mahatma gandhi: "be the change you want to see in the world." i shall quote here an excerpt from tristissima\'s pre-ritual robing litany:  "naked and ready, willing and clean, purified and able, i stand before the world to dress myself in its beauty."  i am poetic, bohemian, and magickal', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i always think of myself as a typical bay area person as one of the few that are actually from here. i remember visiting the mojave as a child, whale watching off the coast, and playing in the orchards when palo alto had any.  i love travel, the outdoors, learning new things and just having a good time in life. i'm a really balanced person who's looking for the same - meaning i'm as likely to be spending a sunday hiking windy hill or rancho san antonio, waking up in the pinnacles or lassen nat'l park, wine tasting in napa, watching my nephew play baseball, or watching football on the telly.  i'm attracted to overly intelligent people who have great senses of humor. please like to learn and be willing to challenge me and help me grow as a person.  what makes me...me? i'm obsessed with all things canadian. i know more about the niners than is healthy (and get super passionate about it - best to just let me rant. i'll move on eventually, promise). i like going to hockey games...for the fights...yet i hate war and mlk jr is a personal hero. i'm a huge foodie and am also a vegetarian (you can eat whatever you'd like, though i might enourage you to buy it from local farmers/ranchers). airline travel might stink, but i'd still rather be in a security check-point line that just about any other place in the world.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "hey there! texas-born bureaucrat/foodie/geek here. i guess i should go through those in order, with the power of allcaps to aid me.  bureaucrat: i don't bore people with talk of my job, but i do bore people with talk of all the world's problems. the other side of this: i love listening to other people talk about their perspectives; i've got this terrible problem called empathy that makes me not want to go near politics ever.  foodie: i love cooking and i love trying new things in the kitchen or local restaurants. i'm a fairly recent vegetarian, too. i like animals too much to eat them.  geek: i've known i was a geek ever since i was twelve years old and my father kept buying me fantasy novels (on account of i asked for them). i don't read as much genre fiction as i used to, but i still enjoy it. and yes, i've been known to watch the occasional anime or sci-fi television series. i'm also a geek about public policy and sociology, but that's because i'm too nerdy for my own good.  i listen to npr a lot, but i also listen to music, sometimes! i am open to new audiotastic experiences, but lately i've been into vaguely pop-sounding indie and weird chicks with pianos.  i'm from texas, but don't own a cowboy hat. this is a perpetual source of shame for me.  i am silly, curious, and dorky", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm looking for a best friend and partner to share the simple things in life--someone who is thoughtful, romantic, family-oriented, and interested in a monogamous relationship. my ideal date isn't a night out on the town or an adventurous hike in the wilderness. instead, i'd rather spend a quiet evening with someone special and sharing some laughter, discussing our day, and talking about our hopes and dreams. of course, an occasional trip looking for a good karaoke place or new vegetarian restaurant would be fun, too.  in my free time, i like working out, reading, watching sci-fi and crime shows, playing board games with friends, taking a weekend trip to see a concert, going to karaoke (room styleyou think i want to frighten strangers with my singing voice?), and learning new things. i'm definitely a lifelong learner, because my idea of fun includes learning about almost any subject, whether it's enrolling in a math class for fun or taking the summer off to go study a foreign language in another country. i suppose it's appropriate that i ended up working at a universityi never wanted to leave.  i think everyone is unique and interesting in their own way, so rather than necessarily sharing similar interests, i'm looking for someone who shares similar goals and valuessomeone who values commitment, honesty, working together through obstacles, and who wants a family of his own one day. (a healthy lifestyle is a must, so i'm not a good match for smokers or people who use drugs, including occasional pot and party drugs).", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == '***update*** as i\'m currently in a relationship, i\'m looking for "activity partners."  i love the outdoors, and any kind of athletic endeavor. lately i\'ve been really excited about soccer and surfing. i swam in college, so anything connected to the water makes me exceedingly happy, elated, even. i like to stay active and go out, but really do value relaxation as well. perhaps that\'s why i\'ve chosen to live in a shared house with three guys surrounded by open space and hiking trails. growing up with two older brothers has given me a great deal of understanding about men. i\'m not afraid to live with them, and i sometimes feel that i prefer their company to that of women. not always, just at times.  i like to dance and paint my nails pink. i like to giggle with my friends. i like to laugh at myself. i\'m passionate about what i do. i believe in obama. i became a vegetarian just over a year ago. i think carefully about what i put into my body and love to prepare meals for others. the farmer\'s market is an integral part of my sundays.  i am vivacious, athletic, and empathetic', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "it 's so hard to describe oneself ! i'm very upbeat, open minded, quite responsible. originally from france, but been in the us for quite a while. i care about saving resources, the environment, animals, and life loving people. i love to travel, take pictures, and i was called a foodie several times, so i must be a lover of food & wine too. did i mention that i love animals; i try not to eat them, but i'm very respectful of those who do. i'm certainly not an obnoxious vegetarian, no worries ! i like all forms of visual arts including fashion. i'm extremely straightforward and have a sense of humor that reflects this trait of character ...i've noticed that people quite often do laugh at what i say . i'm very responsible, and i think that i'm a very strong person, but i'm not necessarily very assertive. overall, i've already achieved my main goals in life, so i'm now looking for new challenges. i do admit that i can be overly silly, and even though i've been in the us for almost 10 years some people can have a hard time understanding me due to cultural differences. i'm pretty much the ultimate anti-materialistic, and i do not enjoy talking about money or the things that people buy. i'm very self confident, but i'm not very judgmental or pretentious. i have a very low tolerance for selfish, thoughtless, and inconsiderate people. lately, i've learned to be a better listener, and i'm enjoying it. i am a bit crazy, at times difficult, but never boring ! ( one of my co-workers came up with this sentence ! )", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm pretty laid back and fun. believe in 'live and let live' philosophy.  take good care of myself - but i'm not shallow - looks don't matter as much as a good heart does.  i'm honest, drama free and looking for someone steady, someone that works hard and plays harder.  i'm into singing/reading poetry/prose in an esoteric language, traveling, yoga, white-water rafting, renting cabins, playing badminton/tennis/pool (possibly can kick your butt), concocting plans to get out the rat race.  what i'm looking for: since i'm secure in my pursuits and enjoy them thoroughly - i wont tell you how you should lead your life and what choices you make. i hope i will have something to learn from you. it'd be nice if you: 1) have something going for yourself (intellectual/creative or career wise) 2) are independent/self sufficient, take care of yourself mentally, physically and spiritually.  dislikes: 1) judging others without getting to know them. 2) people who think they know it all.  chemistry is important, we'll have to find out.  my interests: road trips,travel,whitewater rafting,bowling,shooting pool, vegetarian cooking, learning sanskrit, singing sanskrit poetry,comedy clubs,plays/musicals, ,tennis ,badminton ,yoga ,perl/java programming ,simplifying things  i am curious, compassionate, and inquisitive.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i suppose this is the place where i get a few seconds to convince you i'm worth a continued read...  i'm nice. really. i work with animals and stuff. wait! i mean it, it's not a ploy to get in anyone's pants. my parents work at a dog rescue, so i regularly help out with adoptions, and am apprenticing to be a dog trainer. i love animals of all kinds because, well, they're adorable. i could easily spend hours watching cat videos.  along with being nice i also know how to dish out some sass and take it in return with people i like. i actually do a popular comedy podcast that i'll tell you about if we meet. if you can't take the occassional offensive joke in jest, we probably will have a hard time getting along with one another. in smart comedy nothing is sacred.  a few other things i'm into are video games, vegans and vegetarians, music, going to shows, being at home and watching a movie, real conversations about meaningful things, coffee, and a good cup of interesting tea. i especially like video games since that's what my current career is focused in. no, no, no, i'm not some giant manbaby like in grandma's boy, but i do write about games for a living (and get paid pretty decently too). someday i'd like to transition to making games, but for now i'm content that quite literally millions of people like to hear what i have to say about them.  so yeah i am a nerd. and proud of it. i am also a musician. want to hear some of my singing and playing guitar? check out my music page. i'm hoping to put out a new album this year.  i like history and religion (both of which i have a bachelors in). i am not religious, but i just think that religion--because it is so much a part of so many people's lives--is really awesome to study and learn about. having some knowledge about this, or even just enough interest in it or me to tolerate my blabbing about it, is rad.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i would be an excellent match for someone with keen relationship skills, high self-awareness, a commitment to taking responsibility for herself in encounters between us, and a vision for delightful possibility.  i am ready to connect with a wonderful woman. i am compassionate and a great cook. hiking, nature, shabbat, meditation, yoga, time apart and together, best practices of happy couples, contemplation, passion are important to me. i have an eclectic spiritual practice including deep and creative judaism, meditation, and yoga. i am also funny, energetic, and thoughtful. my life is full and i am happy.  i am a gourmet vegetarian cook. i love plants and nature, except i am allergic to dogs and cats.  most importantly i seek someone who is available for mutuality and clear communication.  i am meditative, brilliant, and sexy.  since i've been single, i've made a commitment to practice speaking the truth more clearly, both about what makes me feel happy and connected and what bothers me. i've found that all of life, including friendship, work, dating, and customer service, can be fertile ground for showing up with more compassion and honesty.  in studying and teaching mindfulness and couples' work, i bring a sense of increasing compassion and clarity to my interactions with the world.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'in a relationship with corispence, a gorgeous and talented artist and aspiring chef. we would like to meet a woman to join us....  grew up in brooklyn and have not fully lost that east coast edge even though i have lived here in the bay area for well over a decade. my education was in florida where i earned a degree in anthropology and then did graduate studies in ethnomusicology where i began playing shakuhachi, a japanese bamboo flute. my most recent musical project is redoing soundtracks to old silent films by yasujiro ozu.  for 2 decades i have been a professional musician and music teacher. shakuhachi is my instrument which is an bamboo flute from japan.  in addition, i am self employed as a vegetarian (actually vegan but i do not like to use the word as it scares many people away) chef with my own personal chef and catering business as well as teaching cooking classes and hosting an ongoing dinner/concert series (sometimes referred to as a secret supper club or underground restaurant).  i am an avid film watcher, observer and listener and taster of various art forms.  love to travel and hiking through the redwoods gives me incredible joy. recently rediscovering my love of cycling. purchased a new road bike recently and am riding 150 or miles per week, now. decided to stop driving a car unless needed for business so using the bike for local errands and also getting out and around northern california.....seeing this beautiful area at 15-20 mph is quite a blast!!  japan is the country i have spent most time outside of the states and i have always been attracted to various aspects of japanese culture since i was a young child.  spiritual yet not religious. was raised reformed jewish and have studied buddhism as an adult.  huge soccer fan! used to play and now just observe. love to watch teams like barcelona and arsenal with their virtuosity teamwork and skills. however my politics leads me to be a huge fan of fc st pauli and also celtic. :)  casual, infrequent drinker and love belgian ales, fine wines and occasional brandies or apertifs both to sip as well as to cook with. enjoy smoking a good bowl as well.  very progressive in my political beliefs and actively engage them in my daily lifestyle. back in college it occurred to me that we can easily change much of the corporate takeover of the world by simply avoiding all, or as much of their products as possible. so for me, it is a lifestyle of choices that i decided back in college when i started to really became aware of social, ecological, economic and political problems.  total turnoffs to me: republicans, chest tatoos, most hip hop and so called progressives and liberals who still think obama will actually change something. ok, i just alienated a large part of the bay area community :)  meanwhile for more stalking material you can look at my websites http://philipgelb.com new food blog http://philipgelb.blogspot.com/ want to listen to a concert i did in nyc in winter, 2010? this is a live soundtrack for yasujiro ozu\'s film, "passing fancy" http://soundcloud.com/blackcat/passing-fancy-part-1?utm_source=soundcloud&utm_campaign=share&utm_medium=email&utm_content=http://soundcloud.com/blackcat/passing-fancy-part-1', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "never know what to write here, but here goes.  i have a habit of counting exact change when paying cashiers. i love to start new personal projects, but usually don't end up finishing them. i love driving around sometimes and just looking at homes. don't know much anything architecture, but love looking at them none the less. i toggle between overeating and working out a bunch. i enjoy going to the gym, yoga, krav maga, being with friends, board game nights, making beer and sunday brunch with the family. some week nights i enjoy switching channels between msnbc and fox news. sometimes i switch back and forth every 5 minutes.  i've played different roles at my job. in the end i think it's of little importance what i do for a living. who i do it with though is critical. i love my coworkers, free food in the kitchen, plane trips and dinners to work and schmooze with counterparties, planning meetings with my team around this or that, and solving problems in excel. also, office politics there is a drag only once a year during bonus and promotion season. sometimes i work in spreadsheets so long i lose all ability to socialize. i can't tell jokes, laugh or crywell usually i can still cry. those are the nights when i watch msnbc and fox news. in short, i've been at my current job for seven years now and still love it.  i get excited about cooking, but usually don't have energy for it and eat out a lot. i'm usually a vegetarian, a paleo dieter, a junk food eater or am living solely off of lemon juice, maple syrup, and cayenne pepper for 10 days straight. i'm very strict about only doing one at a time, however.  how about you? do you know what you want to be when you grow up? do you always stick to what you set out to do and never back down to anything in your way? have you cured yourself of all eccentric habits only weird people do? are you........ efficient? if so then i wish you the best of luck finding your dream guy.  do you love your body and celebrate it both through indulgence and with a healthy living? do you enjoy being lazy, but tirade yourself when you do it in excess? does my profile sing to you? if so maybe you and i should get to know each other for real.  hopefully, by the time you're reading this i've already felt the same way about you and sent you email. if you're making the first move don't fret. maybe i already saw your profile and was moved. maybe i'm just being a jerk thinking i have something more important to do than send you an email tonight. it can happen because it takes me a long time to write one of these emails. anyway, don't be afraid to send an email and we'll take it from there!  ttys!", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i have been accused of seriously not taking everything too seriously. i take the environment and the state of the world (not just the usa) very seriously. i will probably take you seriously, but not everything should be taken seriously, especially the amount of times i just used the word "seriously" in this first paragraph! seriously!  apparently i should make note that i do not actually live in san francisco right now, but rather i spend a good deal of time there for work and where i live, i often find it difficult to find quality women whom i can relate to and they lack the diversity and culture that i look for in a woman. then again, it really doesn\'t matter because i don\'t care if no women hit me up on here.... i\'ll still be perfectly happy!  i love working, learning, gardening, pool, movies, nature, music, photography, sex with the right person, movies, cooking, kissing, documentaries, sushi, discussing philosophy, martial arts, wine, walking, climbing, vegetarian food, talking on the phone, animals, love, friends, cuddling, and so many more things that i\'m not going to just randomly list because mystery shall prevail.  i play a bunch of instruments, draw, and love photography (manual slr\'s mostly). i also do martial-arts and i am on my 5-year plan for law school. i plan on being a socially conscious lawyer and not a sleazebag or a criminal defense lawyer (sometimes synonymous). right now i am doing general education, but soon i will be there. currently, i work in the music industry and run my own business to that effect. i am educated and learned enough to realize that college is all about buying a degree. i like olives, women who are beautiful (inside and out), sex, karaoke, live bands, blacklisting clients who deserve it, and not in that order.  i do not choose to associate with organized religion because i believe that it is separatist in nature, but i do believe in nature.... i have an admiration for others who have the strength to think and speak for themselves and i believe that following is not always the best path and being a leader has gotten me as far in life as i have come.  i am from an interracial family. my mother is an oldschool feminist and ex-flower child and my father is just.... awesome in all ways! i love my family and i feel very lucky that they care about me.  i believe that sex is not anything to be ashamed about and i am open and honest about it, although i also respect others who are not at this level of openness. i do not think that a relationship would be very successful when based on sex alone, but rather that sex can enhance a relationship with a special someone greatly. i don\'t feel that there is a need to rush it and getting to know someone is very important to me before the physical element.  i am more attracted to women between the ages of 23 and 34. women younger than 24 tend to not be mature enough for me, but that is not an absolute rule, just general experience that i\'ve had.  i have a great profession and wonderful friends. i have lived surrounded by nature and i live well. i feel that people can be more self-sufficient than they have been conditioned to think that they can. i think that i am a really positive person and intellectually stimulating and look for the same in others. i am not interested in flings or superficial relations. i deserve better than that, and other people who feel the same way are people who i would consider potential friends or more. i am into monogamy and have never nor will i ever consider cheating on anyone who i am in a relationship with. in a relationship i am more of the dominant type of male..... more of the alpha-male in me, there is :) i do not believe that men should run everything though, and i don\'t really get along exceptionally well with pushovers, so you should be able to stand up for yourself when it counts.  i enjoy the arts and have a strong creative streak. i do not do any drugs, but i do not have a problem with people who do natural ones. i do not watch tv although i\'m sure that there are some good things on tv occasionally. i am also not very materialistic and do not want to surround myself with others who are high maintenance in that aspect. i am a supporter of gay rights and believe that anyone who says that gay rights is different than civil rights should probably educate themselves on the definition of "civil rights".  i\'m overall just happy to be myself and happy to feel as alive and awake as i do now!  that\'s a little bit about me in a nutshell. feel free to ask me more!  i am funny, nice, and scintillating', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm a transplant from the east coast who came to the bay for a job and was pleasantly surprised by the area. i'm a bit of a geek who loves movies, books, hiking, weightlifting, and playing board games with friends. yes, i like long walks on the beach, trying out new restaurants, cuddling on the couch with someone special, but really, who doesn't? that doesn't really tell you anything about who i am. instead, i'll write down some of the moments in my life i remember best and let you decide what they mean about me.  back east, i was in a meetup group devoted to trying unusual foods. when we decided to have a vegetarian meetup, we held it in the back room of a local restaurant. i learned a lesson that day: never, ever open a durian in a room with no windows.  it was really unnerving camping out in a field populated by highly intoxicated bees. the entire night was punctuated by the very soft thuds of passed-out insects falling to the ground.  when i'm really down, i find myself singing along with washing of the water, by peter gabriel. it always seems to help.  my friend marq and i used to eat wasabi tobiko sushi *at* each other. we'd face each other, take a piece, eat it, and see who could keep a straight face the longest. marq's girlfriend would be the commentator.  sometimes, when i'm having trouble sleeping, i picture myself on a roller coaster full of loop-de-loops. it usually works. (i don't fall asleep on actual roller coasters, incidentally.)  you might not think cracked-pepper sorbet would work, but it really does. also, japanese tourists on vacation in france look very surprised when americans speak to them in their language.  when i drove down highway 1 at dusk for the first time, i put the top down, turned to watch the sun fall off the edge of the world, and thought: if all of the joys and achievements and hardships and risks of my entire life have led up to this single, perfect moment then it has all been entirely worth it.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "flaming, freak, fun, foolish, fried, family, all f'd up. bisexual, vegetarian, poly, raver, goth, queer, kinky top, smurf. computer programmer. beach dweller. skirt wearer.  check out my online presence: i'm inki on livejournal, inki on polymatchmaker, and inkidoh on ym. my writing website is www.pepperminty.com and my analysis blog is freaksexual.wordpress.com.  i have gotten really into poly activism and social theory over the last four years, doing a whole lot of writing. are you into queer theory, feminist theory, anti-racism, or the like? write me!  i'm into bdsm. i like to go out to fetish clubs, psytrance nights, and raves. i have a thing for movies. i hold amazing parties on occasion, some of which are sex parties and some of which are not. i'm a regular at burning man.  these days i am quite the busy little bee, as i am currently dating a number of local people. i am still looking for play buddies - if you want to get together casually or meet at the dungeon, let me know. i'm interested in all genders.  oh, and my name is pepper! that would be my real name. nice to meet you!  i am purple, blue, and red all over", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "hi there! i'm new here, and looking to make some friends. i'm a creative, funny irish musician who has just moved to san francisco. i travel a lot, and also live in indiana and ireland. i'm studying anthropology and ethnomusicology in the bay area. my hobbies and interests include playing music (irish, breton, and chinese traditional music and folk music, along with classical and some jazz on irish flute, concertina, uilleann pipes, bombarde, and gu zheng, hiking, vegetarian cooking, british comedy, irish studies, irish gaelic, linguistics, pc and mac gaming, dogs and cats (to play with and take care of, not to eat), environmental issues, folk art, cartooning, web design, hanging out at bookstores and coffeeshops, sociology, medieval studies, comparative religions, theatre,historical re-enacting, genealogy, independent films, philosophy, set dancing and meeting new people.  i am spontaneous, unconventional, and humorous", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "hey. names dan. and i'm an easygoing guy from the bay area, born and raised on the peninsula! i love california, it's beautiful. i have a ba in environmental studies from uc santa cruz. i love my friends, i'm a pescatarian, or a vegetarian that loves sushi too much. i like having a good time whether that means hanging with friends, hiking, camping, sleeping, goin out for drinks, staying up late, hey liz! my name is dan, and i have been a part of okcupid for a very short while! i've been delivering around menlo and palo alto, with an envs degree a little while i figure out what to do, going to the beach, playing with my beatiful cat stella, swimming, painting, playing guitar, fishing, doing a puzzle, or talking with fun interesting people!", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm a washu student - on the brink of graduation - who loves traveling, black and white photography, live music, ethnic food, beer, the red sox, and a whole lot more. an anthropology major, i'm hoping to someday set health policy in an international context. i'm an east coast native, but the midwest has really grown on me during the past few years and i'm a big fan of st louey. i live in a co-op off campus, which has given way to some great relationships, a whole lot of hookah, and has turned me into somewhat of a pseudo-vegetarian - although i still enjoy a great burger when i get the chance! i'm up for anyone who savors new adventures, honest conversation, and lots of laughter.  i am inquisitive, giggly, and motivated", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'liberal, free-spirited, independent minded, artistic, active yet mellow, pensive, activist - anti-war, anti-displacement, fair-trade, labor rights, all that jazz.  vegetarian, i cook a lot, bake a lot,  wanna be a writer, photographer, social worker, teacher, too many things.  i went back to senegal last year to intern with a senegalese journalist - trying to build stronger connections between senegalese and american journalists.  i am me, myself, and i ?', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "some things about me... i...  - work in the technology land - am a heavy consumer of information - don't have a tv (but watch some shows on hulu) - write - walk - am working on becoming vegetarian (about 70% right now) - am on twitter and facebook but not myspace - am very environmentally conscious - volunteer for sexual health/education/advocacy organizations - have walked across the brooklyn bridge in the snow - am very organized and punctual - like to talk and to listen - show up and do the work - identify as sex-positive, creative and open - like to be present and engaged in the world and with others  ... just to name a few things. i like open minded, interesting, active, smart, complex people with similar outlooks on life... bonus points for good grammar skills...  my girlfriend and i are emotionally exclusive; we like finding open minded and interesting play partners.  i am open-minded, communicative, and surprising.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "hi there...kristofferson here...been living in san francisco for 8 years now...love this city and everything about it...the multiculturalism, architecture, art, social vibe, and vegetarian food! looking for someone who can appreciate what this city has to offer with me...look younger than i look...i've been told...which i'm finally embracing as a good thing! looking for down to earth guys, a sucker for homebodies ...who would rather stay in and watch episodes of this american life over some good ole wine and... maybe a lil green ;)) than go out... must be nice, sweet, considerate and do enjoy me some nice facial scruff!  visiting waikiki may14-22 2011..looking for friends to show me a good watering hole! :)", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i've been a geek my whole life. i grew up learning basic, playing my nes, and reading fantasy novels like the narnia series and mrs. frisby and the rats of nimh  in high school, i converted to wicca, although i'm much less serious about it than i used to be. i'm still vegetarian, though. i felt a real connection with wolves in general and werewolves in particular. i was also drawn into my love affair with anime, thanks to series like bubblegum crisis and outlanders.  i loved anime so much that i took four years of japanese in college. i'm now relativly fluent, although i need to practice, to keep my skills up. i also majored in mathematics, with my main area of interest being probability and combinatorics.   i am intense, cavalier, and introverted", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == '(originally from bombay, in the bay area since last 5 1/2 years)  "\'when one does not love too much, one does not love enough." blaise pascal  i am deeply moved by compassionate beings!  i thrive on love and care,giving as well as receiving- a lot of it : )  nature makes my soul feel at home......  creativity feeds my deepest hunger for self-expression....  children bring out the bessssssssst in me : )  art makes my life seem most meaningful,purposeful.....  i love loving,dancing,playing,laughing,crying..... with reckless abandon...  i have too many passions, but dancing and singing top the list... i\'m also a trained yoga instructor and hold a master\'s degree in philosophy...  am vegetarian...into- meditation,watching plays/concerts etc, stimulating discussions,eye-gazing,people watching, five rhythms, tickling,volunteering,massages,wandering in the himalaya mountains etc...', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm 49 years old and a fairly good-looking and fit white guy, 155lbs, 5'10.  i have had a very eclectic life and am broad-minded about all sorts of things. the range of experiences i have had have made me a better person, or at least interesting. i have degrees in science, arts, and humanities. i perform different kinds of music--music of bali and western classical music (in choruses). i've traveled a lot--to about three dozen countries--and have lived about five years overseas, mostly in europe and indonesia (speak fluent indonesian and some balinese). i still travel to bali often and own land there; many of my friends are balinese. it's been a gift to be exposed to so many different places and cultures, and it's a core part of who i am now.  being environmentally sensitive is an important aspect of my life. i would want environmental awareness to be important to my partner. i recently completed my ph.d. in environmental theory and increasingly try to tread lighter on earth in my everyday practices, though i'm not fanatical about it (well, ok, maybe sometimes). politically, i'm on the left, but i challenge the dogma of the left, too. i love animals and volunteer at an animal shelter. i'm pretty much vegetarian except for seafood.  i'm shy, but open up a lot when i feel accepted and supported. i'm not very detail oriented, so someone who likes, say, planning the minute details of a trip would be a particularly good counterpart for me; i'm good at the big-picture stuff. i have a wry sense of humor that some say is very funny.  i'm a sensuous person. i like to cuddle. aside from playing music, i am into hiking, traveling, occasional camping, hot springs, and various other fun things. mostly, i am into being with you, if you are the one.  i am sensitive, eclectic, and circumspect", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "and now the hard part - i am a transplanted east coaster, totally in love with sf, professional, intelligent, attractive, love animals, travel a lot, have a great sense of humour, close to family and great social circle. i am adventurous and spontaneous. i have a really good life and am looking for that romantic string to tie it all together. i am very loyal, will always be there for the ones i love, even if its only for shopping :). i work out and exercise both mind and body regularly. i also willingly give great massages (really!). happy to cook with you, debate your views on the world, explore dark corners. dogs are a big part of my life, so dog hair on the bed is part of the deal! music is also crucial - always out for live shows, dancing, and bonus point if you'll go with me to bm. interested in buddhism - not religious, but spiritual - helps to tie in my beliefs about animals etc you - smart, attractive woman, with tons of energy, an athletic and adventurous spirit, vegetarian or light on meat, intellectual curiosity, and a sharp sense of humor. i am very passionate, huggy, romantic and totally devoted and loyal - my partner is my queen.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i've been in the bay area for more than two years and i freaking love it. how do i love thee? let me count the ways: 1) weather - i was born and raised on a tropical island where it rains about 100 inches a year. lush rainforest is overrated. cloudless sunny skies 85% of the year? hell yes. 2) cool people - i love the startup/techie crowd around here.. smart and ambitious, yet fun and down to earth. 3) good food from every cuisine - food is my weakness!  not to mention everyone here seems to care about health and the environment at least 1.5 standard deviations more than the rest of the nation. since arriving i've achieved many personal firsts like running 12k (bay to breakers), biking > 20 miles, and going vegetarian for 6 months.  other random facts about me: my mom's name is june, my sister's name is may. guess what my name is? my dad's from hong kong, my mom's from singapore, my sis married a hungarian - i spend way too much time traveling to visit relatives at different ends of the world. i spend way too much time on reddit. mirri is a character from magic the gathering. yes i am a nerd.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'd like me if i met me. i'm a nice, loyal, honest, adventurous, little vegetarian girl.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i follow politics daily. i am very socially liberal...but not quite on the tree-hugger side of things.  i follow the "everything in moderation" mantra.  i am the greatest snuggler/spooner of all time...no, really.  i almost always prefer eating some sort of ethnic food from an independent restaurant.  i eat on average 1lb of chocolate a week, that is 52 pounds per year.  i watch sci-fi for the escapism, foreign/indy movies for the alternative perspective, and documentaries for the curiosity.  i have strong opinions about most things...but am generally very laid back and easy going, live and let live no?  i don\'t follow, and rarely watch any kind of sporting event.  i love dancing to electronic music till the sun comes up.  i eat lots of vegetarian food, but i\'m actually a pescatarian....because there is no way i could ever give up the bounty of the sea!!!  i just moved to san francisco because i was too eclectic for san diego.  i am alternative, geeky, and quirky', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "http://www.youtube.com/watch?v=-auug1qme5i  aside from knowing how to rock, i love animals and was raised in a vegan household. i was vegan for 6 years and haven't eaten meat or drank milk in about 15. being vegetarian is a big plus but not a must. i currently work for a non profit.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "my dream is to live in the bay area and help people. this is why i'm studying to become an rn. to work in kaiser oakland would be a dream come true. whether a woman dreams of working at mickey-ds or dreams of becoming president, dreams are important to have. in my spare time a write essays and short stories.  some families are tied biologically, others through a circle of friends. either one counts. i grew up in a big family where humor and eloquence got you respect. my sisters tell me i'm attentive, funny, intelligent and a good listener.  i was raised catholic though i am not religious. i've read extensively on eastern thought and at one time considered myself a taoist. i also wish to finish reading the koran one day. it's good to be curious. every exciting woman has some spiritual curiosity, whether or not she identifies with a religion or not.  i've worked on several political campaigns including congresswoman jackie speier (d) and carole migden (d). i'm a democrat, but i don't rule out dating republicans.  a woman should have a respect for education, be she a high school drop out or a phd. i love a good teacher. history is my favorite subject. ancient history i find the most compelling. it tells us who we were and where we're going. yes, museums can be a snore but the right exhibit rejuvenates.  as far as the news of the day i get mine mainly from chris matthews and npr.  growing up in the multi-cultural bay area, art is essential for community survival. i love art, music and movies for the same reasons. they can bring different people from all walks of life together on a common plane. i played the clarinet as a kid. should have tried the saxophone. a woman with varied tastes in music is a plus. a love of cultural diversity goes a long way as well.  a's games are awesome!  i like walking. an ideal day includes a 4-5 hour walk, but sometimes a have to settle for an hour. i work out about 5 times a week and find exercise to be the greatest drug. i wouldn't mind going to a vegetarian restaurant for a date. or we could cook something together. the best dates would involve exploring the bay area, preferably on foot. i'd like to get to know my town better along with it's neighboring cities.  so, what makes a good woman?  intelligence is worth listening, a heart is worth protecting, humor is worth keeping. intelligence, heart and humor together is golden.", 'diet'] = 'mostly vegetarian'

In [70]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("vegetarian"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

In [71]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "oh the lists: naked, catch 22, wicked (the book, i hate musicals) benjamin franklin biography by isaacson. ol' ben is my hero.  i read vonnegut when i'm really down. i know that might seem like a silly thing to do but it works. how can you not laugh at the asterisks?  farm city - such a good book. if you live in o-town or you love pork it's a must read. as a vegetarian some parts of the book were a bit creepy but admirable.  alt-80's , radiohead, be good tanyas, luna, the duhks, the white stripes, neko case, pixies, echo and the bunnymen, ..you get the idea. i still love to go dancing at new wave city. i know that makes me a complete and total dork but i don't care, damit!  deadwood, the fantastic mr. fox, nightmare before christmas, lost in translation, american beauty, the wire  i'm not ashamed to admit i like harry potter.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: a world undone, a bridge too far, eating animals...  movies: the untouchables, the godfather trilogy...  music: i love house music, i also like hip hop and jazz music. when i am stressed and need to relax, i like listening to yanni or enya.  food: recently, i have made the conscious decision of being a vegetarian and i am still trying to figure out what i really like.', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "catcher in the rye, crime and punishment, the death and life of great american cities, dress your family in corduroy and denim, earth in the balance, eats shoots & leaves, the elements of style, eloise, fast food nation, the giver, good omens, harry potter series, a heartbreaking work of staggering genius, hitchhikers guide to the galaxy series, holes, holidays on ice, how to overthrow the government, lake woebegone days, lord of the rings trilogy, matilda, me talk pretty one day, naked, nickel and dimed, normal is just a setting on the dryer, on the road, one flew over the cuckoo's nest, shrub, slaughterhouse-five, small gods, the stranger, to kill a mockingbird, waiting for godot, why people believe weird things, a wrinkle in time  almost famous, amelie, airplane!, best in show, big fish, breakfast at tiffany's, eternal sunshine of the spotless mind, finding nemo, forrest gump, harold and maude, i heart huckabee's, lord of the rings, lost in translation, monty python movies (all of them), moulin rouge, mrs. doubtfire, napoleon dynamite, o brother where art thou?, pleasantville, the rocky horror picture show, the royal tenenbaums, saved!, school of rock, the shawshank redemption, star wars iv-vi, star trek movies, toys, v for vendetta, weird wednesday at alamo drafthouse, young frankenstein  alanis morissette, all-time quarterback, azure ray, badly drawn boy, barenakedladies, the beatles, belle and sebastian, ben kweller, ben gibbard, beulah, cake, cat stevens, celtic fiddle festival, clandestine, damien rice, dave matthews band, death cab for cutie, the decemberists, ella fitzgerald, elliott smith, elvis presley, flogging molly, the go! team, goo goo dolls, green day, iron & wine, jimmy eat world, jose gonzalez, lali puna, madonna, matt pond pa, miles davis, the new amsterdams, pinback, the postal service, rainer maria, random indian music, rent, rilo kiley, the rocky horror picture show, semi-gloss, the shins, simon and garfunkel, styrofoam, sufjan stevens, tammany hall nyc, tegan and sara, third eye blind, tom lehrer, weezer, wes montgomery, yo la tengo, classless dance music, (not a comprehensive list)  i am a pescevegetarian: thai, sushi, italian, mexican, indian, pizza, cajun, berkeley food...everything?", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "favorite music: dave matthews band, jason mraz, jack johnson, maroon 5, postal service, barenaked ladies, the cure, no doubt, cake...and i've recently gotten into israeli pop music too  a few movies i really like: american history x, garden state, office space, when harry met sally, the usual suspects  i'm a huge fan of vegetarian food, even though i'm not exactly a vegetarian. i just don't eat red meat. not for any idealogical reason though. i just never liked it much, so i stopped eating it. turns out red meat isn't all that good for you anyway.", 'diet'] = 'mostly vegetarian'

In [72]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("vegetarian"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [73]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay7'] != np.nan) & (ok_cupid_df['essay7'].str.contains("vegetarian"))]['essay7'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay7'] == essay, 'diet'] = 'anything'

In [74]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "living vegan brings me joy. i'm also sober 13 years. i love walking my dogs each morning and going to my local coffee shop for coffee. i keep it simple out of preference and necessity.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == '"life is a shipwreck, we must not forget to sing in the life boats"- voltaire  i try to smile at people, even when they don\'t smile back. i\'m excited by interesting shadows. i feel myself getting older, and i think i like it. yet i\'m still trying to avoid growing up. maybe i could become a wise child. i think that would suit me. i\'m fighting to not take anything too serious, including myself. i\'ve been thinking a lot about the ebbs and flows of life. how we fight change, even when it has our best interest in mind. i used to go to burning man, but i think i have retired. i love the musee mecanique, vegan soulfood (i\'m not a vegan, i\'m a pescatarian), jejune institute (extra bonus points if you are familiar), being somewhere you can see no sign of mankind. i\'m not to fond of show-offy folks, narrow mindedness, blockbuster films, indulged negativism. i think people, when riding bicycles, are sexier when they wear helmets, use turn signals, and have lights.', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm an admittedly sometimes lazy guy. i like vegan cooking and baking, tasty ales, and horribly obscure and noisy music, particularly with gee-tars involved. i started running seriously two years ago. i've run a couple of half marathons and a couple of marathons. with a change in my work schedule, i haven't been able to run much but i bike over 100 miles a week and want to get into racing.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == '"i don\'t want to end up simply having visited this world." ~ mary oliver  emotionally mature, financially secure, spiritually unfolding, and younger growing (with each day feeling freer and more alive)--i am juicy, spicy, strong, tender, warm and sweet--makes for a fun, nourishing and deeply satisfying relationship...:-)  blessed with a second career doing work i love while being of service to others (first career as a technology entrepreneur was successful/meaningful as well), i am a bit shy and reserved in groups (unless i am the leader). yet, i am intimate, relaxed, and vulnerable 1-on-1. others describe me as passionate, energetic, intense, intelligent, playful, adventuresome, empathic, and a romantic at heart. i am naturally curious and most find me deep. depending on the moment, i can be strong, independent, and fierce as well as tender, gentle, and vulnerable. i love nature, adventure, truth, freedom, and intimacy. i enjoy solitude as well as communion. i am sensual, affectionate and enjoy touch. i am conscientious, and caring as well as can be spontaneous, playful, goofy, and care free (\'angels fly because they take themselves lightly\'). i balance masculine strength, autonomy and direction with the feminine qualities of attunement, compassion and sensitivity. rumi says it much more beautifully:  i was deadthen alive weepingthen laughing the power of love came into me and i became fierce, like a lion then tender, like the evening star.  i love nature and i love music, kirtan and dancing (especially soul motion, 5 rhythms, ecstatic dance, contact improv, a little salsa and blues). i enjoy yoga, tai chi, and meditation as well as hiking, biking, cooking, movies, art, and reading and writing poetry.  i enjoy travel, adventure and exploration--especially on the path of self discovery ("at the end of all our exploring will be to arrive were we started and know the place for the first time" ts eliot.) my core spiritual path is the diamond heart and i also draw buddhism, mystical judaism, shamanism, and tantra. i know it\'s a lot but what else is there that is of greater importance. perhaps love. and then again, the spiritual path is helping me live from the realization that our true nature is love.  p.s. did i forgot to mention (it must be the age) that i am proud to have attained humility?...:-)  p.p.s. kidding aside, i am actually in excellent health, and live a healthy life style. most who meet me think i am in my 40s. i guess all that exercise, yoga, meditation, biking and healthy food (organic vegan) pays off...', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "-qubcois but grew up in tennessee (french is my first language) -bike is my only mode of transportation -i love to cook vegan food (for 15 people) -i'm a feminist and glbtq ally -i love to calculate. -i'm a french teacher. -i believe in consensus -i value suffering and humility as they balance life. -i am patient and understanding  i am interested in you, rational, and happy", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'subject is healthy california lifestyle refurbished 82\' model leprechaun featuring silky smooth irish brogue, a premium audio system and comfortable steering wheel (cruise control is an optional extra). fuel efficient hybrid engine, takes both raw vegan and "normal" food. crossfit enabled with good off road capabilities, yoga enriched suspension, loves driving in all conditions, bike rack available upon request. comes fully equipped with a warped sense of humor and a strong desire to make you smile. a real pleasure to drive.  unfortunately this model does not come with the optional bay area vocabulary recognition system. phrases such as hella, beezy or hyphy may provoke sarcastic remarks or equally unintelligible hibernian slang spoken in some indecipherable paddy accent.  as a bonus this unit features an extensive in built audio book library covering a wide range of conversation provoking topics from film noir to pop culture, early twentieth century literature, music, philosophy, anything russian, technology, ontology and various cookery opinion pieces on the finer points of french cuisine.  the perfect fun ride to maneuver round chic san francisco areas skipping from one chill venue to the next.  i am incorrigible, inspiring, and positive', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == "i enjoy hanging out at a coffee shop sipping chai over a good conversation, eating a good vegan meal with a friend, riding my new bike and occasionally talking tech. i like having season tickets to things like the symphony and musicals so i don't have to decide what to go see.  i'm recently single so i'm looking to meet new people and make new friends. i like working on projects with people. i find that sometimes i get bored if i'm not engaged in some sort of project. i find that collaberation of some form is also a great way to get to know someone. i tend to learn towards the more technical side of creative things.  i am saucy, godless, and opensource", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay0'] == 'my name is joe, but sometimes i get "joey fitz". i like that nickname.  http://www.youtube.com/watch?v=scngsgfewuq  i\'m a native san franciscan and i probably think a bit too much for my own good, and i swing between playing the masculine gender role, followed with a bit of goofiness. i\'m always up for exploring sf, because even though i\'ve lived here my entire life... i haven\'t found everything yet. and if you\'ll go watch hitchcock films, or noir in general with me at the castro theater i might be damned enamored with you.  just yesterday friend-griff and i found vegan ice cream, and we absolutely had to have the flavor "obamagranate". it was as deliciously presidential as it sounds. then we danced a bit to lady gaga, because the woman serving the ice cream had good taste in music.  afterward, we found a novelty shop where griff and i fulfilled our 15-second long hopes and dreams of owning a super-ball, and miniature kirk-brand-communicator, respectively.  awesome day? i think yes.  i am confident, iconoclastic, and hi mom!', 'diet'] = 'mostly vegan'

In [75]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("vegan"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

In [76]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "(a) ursula leguin, neal stephenson, paul auster, dfw, kerouac, the tripods. i used to read a lot more fiction, lately it's more technical stuff and short-attention-span internet fare. please recommend some fiction if you think you have similar tastes!  (b) dr strangelove, dead man, black orpheus, repo man, almost everything hitchcock made, and pretty much the entire coen brothers' filmography. i recently had a conversation with my roommate where we tried to identify the greatest film of the '00s. he thought it was the dark knight. i can't decide between city of god and wall-e. what do you think?  my favorite thing to do is wander around aimlessly, taking photos of things. you can see some of them here.  (c) chess records, delta blues, 90's alt.weird, bebop, hiphop, idm, ethiopian psych, pixies, david byrne, the velvet underground, radiohead, the kinks, can, dylan, little wings...  (d) japanese, middle eastern, or thai when i go out. i try not to eat meat more than a couple of times a week. i'm allergic to milk & cheese. if you're vegan or semi-vegan, that rocks!", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: the dictionary? (actually, i never tire of reading the dictionary). i love reading what i find in free boxes as a way to broaden my horizon through fate. i pretty much only own reference books on gardening, cooking, travel, and natural healing.  movies: indie or foreign preferred, though i have to admit i see everything kate winslet, kate hudson, and drew barrymore are in even if i know the movie will be cheezy. my friends are amazed that i can rent a movie thinking "hmmm this looks like a good film" and after about 1/2 an hour(!) into it i think "hey, wait a minute, i\'ve already seen this movie!".  food: east indian and asian influenced chow are my faves and what i tend to cook. emphasis on organics and whole grains, but so over dietary militancy. mostly cook vegan or veggie at home, though i sometimes dabble in dead things when i dine out. chocolate!  music: no particular faves, wish i was more into specific artists. funk, blues, funky jazz, old time, and some classic rock will get my toes tapping. electronica, house, and hip hop will leave me asking in a fake old lady voice "are they playing this at the right speed?". paradoxically, a friend taught me how to find death metal and hardcore punk rock relaxing.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: siddhartha, the alchemist, the sword of shanara, the wisdom of yoga, andrew weil, john robbins, osho, byron katie.  movies: the matrix  food: thai, korean, sushi, vegan, pho, salads, organic treats', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm into everything, but i like short books, battle star galactic/ sci phi, jam music and pop, and simple vegan food.  me gusta todo, pero libros cortos mejor, battlestar galactica, sci phi, mi musica, y comida vegano.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'when i come across things i like, i usually muse about how i will make something like it, or better. this often happens in furniture stores. i don\'t like to buy new things.  i had meant to be a writer, but i don\'t think i have the psychological fortitude. still, i think i am slowly going crazy, and i may eventually find that i need to write in order to hold it together. the other day i bought the first volume of shelby foote\'s history of the civil war from some dude on the street, so i think that\'s what i\'m going to read next. (update: awesome. covers both the decision-making at the top and the bleeding-and-dying on the ground. i am pretty obsessed with war stories, and have read a lot about wwii and the u.s. war in viet nam. also that chris hedges book is great: war is a force that gives us meaning. if you\'re wondering what the fascination is, it\'s not macho "heroism" or anything: war forces a lot of human questions to the fore, and although it is certainly not always pretty and almost never a good idea, it really helps you understand who we are as a species.)  my favorite movie, if you like, is wong kar wai\'s "chungking express". it is as much a painting as it is a story. and it is so sexy in so many ways. (one of my favorite scenes: the stewardess is enjoying a post-coital half-snooze, and he gracefully lands a toy plane on the glistening small of her back.) i like beauty in lots of tiny bits; larger narratives are hard for me to assimilate.  i really like music. really. but i am not a connaisseur (i tried to put in all the good french vowels, but that word still looks wrong), and am not able to list many hip bands here. anyway they all sound like sonic youth to me, which is not a bad thing. i studied jazz a bit (playing the drums), and thrill to the recorded conversations that have been made in that cosmopolitan tongue. (i\'m sad to be missing the incredible live scene that was brewing in brooklyn when i left it four years ago.)  my favorite food is ... rare steak? but it\'s been awhile: i don\'t usually eat meat. (except the other day when i ran into some bratwurst; there\'s still some wisconsin in me that needs to be fed.) sushi is my other favorite food. and stinky-like-socks cheese. and, uh, apart from all that i\'m pretty much vegan!  although i don\'t have a very developed repertoire of meals, i am very comfortable screwing around in the kitchen and cooking is one of my favorite diversions: a great way to relax at the end of the day, and a great project to share with a sweet someone. give me a good chef\'s knife, a cast-iron skillet, and a glass of wine, and i\'ll find a way to make a nice dinner out of whatever is in the cupboards.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "words: pessoa wrote my bible  sounds: cream, led zeppelin, hall & oates, timber timbre, chad vangaalen, bon iver, arcade fire, beach house, ccr, the beatles, sleepy sun, sigur ros, tom waits  films: it happened one night, une femme est une femme, father of the bride  i'm also a non-judgmental vegan", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "becoming vegan with my 6 yo. buddhism for mothers womyn's music from the 80's", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "lord of the rings. wooooo! i don't read a lot because it's slow. i read fast, but still too damn slow. foucault, the little prince, cute stories, and a bunch of feminist/anarchist/theoretical/spiritual stuff.  movies: i don't like most of them. i am getting freakishly into sci-fi movies as of late though. so maybe i need someone to watch them with? right now, i watch farscape, i will be rewatcing battlestar galactica soon.  all of these things are boring. i like food. i bake wicked vegan delights and am a champ at cheese/cheeze sauces.  music: this area bothers me... here's what i've been listening to religiously, recently: neutral milk hotel, animal collective, radiohead, mumford and sons, lovesong, nextdoor neighbors, beirut, black moth super rainbow, the castanets, destroyer, sunset rubdown. ya know, i like a lot of stuff, this is just teh soundtrack for the now (ish).", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: counte of monte cristo, sword of truth series, book of five rings, various sci-fi and fantasy series i don't remember the names of.  manga/manhwa: one piece, bakuman, the legend of koizumi, soul eater, full metal alchemist, holyland, hana kimi, beck, naruto, bleach, 1/2 prince, cross game, slam dunk, initial d, kuroko no basket, ginen shounen, kenji, bartender, liar game, magi, dreamland, hitman reborn, death note and many more. i read a lot of shounen stuff mostly, but if i find the story interesting enough, i will read it.  comics: chew, the walking dead  web comics: penny arcade, pvp, vg cats, questionable content, ctrl+alt+del, order of the stick, erfworld, dueling analogs, dr. mcninja, looking for group, multiplex, yellow peril, three panel soul, mac hall, nerf now.  movies: army of darkness, scott pilgrim vs the world, the fp, the avengers.  music: jpop, kpop, gackt, maximum the hormone, nobuo uematsu, and just about anything that's not country or polka. i find that if i hear it in a video game and like it, i am willing to give other genres a music a chance.  food: vegan just about anything. japanese, chinese, indian, etc. if it's vegan, i'll eat it. i'm not a terribly picky eater, just needs to be vegan friendly. i don't expect others to eat vegan if they don't want to though.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "abel's island, anything by chuck palahnuik, kurt vonnegut, the jungle, i have too many favorite movies, i've grown up listening a lot of hip hop/jazz/spanish music, but i've also gotten into a lot of rock. i've been listening to a ton of daft punk/terrible things/damned things/joanna newsom/muse/the movielife/adtr i like a lot of mashed potatoes, and pasta. anything vegan.", 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "to kill a mockingbird and in cold blood i like books that make me laugh, think and maybe even cry on occasion.  hgtv is like porn to me. i never tire of the golden girls, american dad!, cnn or desperate housewives.  i don't go to the movies, i'm prone to falling asleep. i love movies that feature crass humor, real life experiences, a good mystery or an unhappy ending. action flicks and violent movies bore me.  i love music and always have pandora playing in the background. currently tuned to lcd soundsystem.  i love npr.  i eat a mostly vegan diet but do not discriminate against carnivores. truth be told, i love lamb (i'm greek) and think bacon is pretty awesome. i love cottage cheese, tomatoes, avocados, broccoli and cauliflower. don't stand down wind from me.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books... hmmmm, well i was going to an eckhart tolle group in the east bay and after finding the present moment, it seemed a little strange to be watching reruns about the present moment. so i haven't gone in a while.  how about that depok chopra guy - i like the way he explains things to the western mind.  books, looking back to some of the authors i've read... baba ram das, victor kulvinskas' book survival into the 21st century, carlos castaneda series, conversations with god, siddhartha, autobiography of a yogi, the works of darshan singh  movies... i'll get back to you - so many to choose from harry potter, dune series, all the star treks, being there, blast from the past, i love the tv series weeds, and got hooked on lost last year. i assimilated the tv series lost. more to come  music... i listen to a variety of music - love to dance to rythem and beats, and improvise with free flowing spacial tunes. some of the artists i have on my itunes are achillea, stephen halpern, diva primal, delirium, karunesh, anugama, susan teng, etc. etc.  my favorite food is non-material, more energetic food for the soul! with edible food, i'm doing all raw foods leading to being a vegan.", 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i generally read science fiction and fantasy books. i like all sorts of movies and music. as far as food, i am not a picky eater at all, but i am mostly vegan.', 'diet'] = 'mostly vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'authors: dave eggers, j. d. salinger, tom stoppard, jonathan safran foer, george orwell, aimee bender books: the unbearable lightness of being, the ultimate hitchhiker\'s guide to the galaxy, nicomachean ethics, rosencrantz and guildenstern are dead, a long day\'s journey into night, east of eden, jane eyre, a clockwork orange, gdel, escher, bach, the hunger games trilogy web comics: xkcd, a softer world, dresden codak, achewood, order of the stick music: radiohead, the dresden dolls, pony up!, bjrk, counting crows, sigur rs, yann tiersen, pearl jam, celtic folk music, bulgarian folk music, israeli folk music, blair and brother, devin fry movies: rushmore, dead poet\'s society, amlie, the big lebowski, inception, the royal tenenbaums, eternal sunshine of the spotless mind, being john malkovich, adaptation  food:  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ @@.......@@...@@@@@@...@@@@@@........@@@.......@@.......@@ .@@.....@@....@@...............@@.................@@..@@.....@@@....@@  ..@@...@@.....@@@@.........@@...@@@....@@....@@....@@..@..@@ ...@@.@@......@@...............@@......@@...@@@@@@...@@....@@@  .....@@@........@@@@@@...@@@@@@...@@......@@...@@.......@@ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ... but not judgmental.  (and, in case that looks weird/illegible on your browser, it reads, "vegan.")', 'diet'] = 'vegan'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'books: brave new world!!!!!  movies: vanilla sky, the kids are all right, the hours, eternal sunshine of the spotless mind  music: electronic music with an indie twist. bjrk, lykke li, goldfrapp, tori amos, bon iver, fleet foxes, broken social scene, the xx, gorillaz, iamamiwhoami, the dresden dolls, the dirty projectors, fiona apple, fischerspooner, sigur rs, fever ray, imogen heap, the list goes on...  food: hippie vegan food, veggie chili', 'diet'] = 'vegan'

In [77]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("vegan"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [78]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("steak"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

In [79]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains("steak"))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'anything'

In [80]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay2'] != np.nan) & (ok_cupid_df['essay2'].str.contains("steak"))]['essay2'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay2'] == essay, 'diet'] = 'anything'

In [81]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay3'] != np.nan) & (ok_cupid_df['essay3'].str.contains("steak"))]['essay3'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay3'] == essay, 'diet'] = 'anything'

In [82]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("steak"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [83]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay5'] != np.nan) & (ok_cupid_df['essay5'].str.contains("steak"))]['essay5'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay5'] == essay, 'diet'] = 'anything'

In [84]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay6'] != np.nan) & (ok_cupid_df['essay6'].str.contains("steak"))]['essay6'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay6'] == essay, 'diet'] = 'anything'

In [85]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay7'] != np.nan) & (ok_cupid_df['essay7'].str.contains("steak"))]['essay7'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay7'] == essay, 'diet'] = 'anything'

In [86]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains("steak"))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'diet'] = 'anything'

In [87]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay9'] != np.nan) & (ok_cupid_df['essay9'].str.contains("steak"))]['essay9'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay9'] == essay, 'diet'] = 'anything'

In [88]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm a fit, interesting, virile, intelligent, well educated and informed individual. my interests include art, literature, economics, politics, anthropology, sociology, linguistics, the psychology of indoctrination, social change, photography, printmaking, ceramics, hiking, nature, and having an active sex life. for the latter, i would like to meet someone provocative, adventurous, exploratory, uninhibited and imaginative. i don't eat meat, watch tv, or support the rightwing policies of newsom, obama, or whitman (? perish the thought!).  i am imaginative, creative, and reliable", 'diet'] = 'vegetarian'

In [89]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("eat meat"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'diet'] = 'anything'

In [90]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains("eat meat"))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'diet'] = 'anything'

In [91]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay2'] != np.nan) & (ok_cupid_df['essay2'].str.contains("eat meat"))]['essay2'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay2'] == essay, 'diet'] = 'anything'

In [92]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == "i like broccoli and mac/cheese. i can eat pizza forever. don't eat meat, tho.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'ahhhhh, im lame. :( i dont know any cool books, but ive read the davinci code and all of the harry potter books <.< sue me not really a movie buff, i just like them long with a story line and in colour! black and white is too cultured for me i have the music taste of a 14 year old emo girl, so im not name dropping bands im a vegeterian and not because of some pretentious white middle class reason! i just saw a video of what they do to animals on farms and it grossed me out, but i have no problem with people that eat meat.', 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "my favorite books are about dinosaurs. i don't like books about meteorites or the hindenburg. oh the dinosaurmanity.  i do not watch movies because film was not yet invented in the jurassic age.  i like patriotic music and music with lots of cymbals. large cymbals that make crashing noises. when i land my dirigible at an unknown airfield i prefer to be welcomed by a small band playing patriotic music with lots of large cymbals. it rarely happens, but a dinosaur can dream.  because i am not a t-rex i do not eat meat. i mostly eat leaves.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: anything by agatha christie, kurt vonnegut, carolyn hart, elizabeth peters, christopher moore and chuck palahniuk. the outsiders. les miserables. the portable dorothy parker. the charlie moon series by james doss. harry potter.  movies: glory, shawshank redemption, se7en (do you understand how much i love morgan freeman?), the outsiders, american history x, bubba ho-tep, bury my heart at wounded knee, the birdcage, the goonies, guys & dolls  music: descendents, all, manic hispanic, madball, dead kennedys, the temptations, american steel, bouncing souls, the supremes, backstreet boys, sick of it all, good riddance, allister, reverend horton heat, old 97's, abba, dolly parton, willie nelson, taylor swift, the loved ones, the clash, ramones, shai hulud, terror, street dogs, me first and the gimme gimmes, brain failure, 60's girl groups  food: for the most part, i refuse to eat meat. however, i have a crazy love of offal, so very rarely, i will eat tripe, tendon, and tongue. i adore spicy food. favorite foods include curry (indian, thai, whatever), falafel, pupusas, dim sum, burritos, and eggs. i drink lots of coffee, tea, and cranberry juice.", 'diet'] = 'vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i'm much more into music than movies and books.  i love alternative music and bands like franz ferdinand, breaking benjamin, seether, r.e.m., snow patrol, death cab for cutie, hockey (no, not the sport), tegan and sarah, incubus, unwritten law, rise against, beck, garbage, jimmy eat world, stone temple pilots, soundgarden, no doubt, ok go, weezer, yeah yeah yeahs, ...  foodwise,,, i love sushi, tempura, and fruit.  i'm a pescatarian... if you don't know what that is,,, i don't eat meat, except for fish. i would gladly date a meat-eater as long as they are okay with my eating style. i just don't like meat,,, i stopped eating it 6 years ago, didn't appeal to me anymore.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "memoirs. short stories. classics. mcsweeney's. dave eggers, david sedaris, kurt vonnegut, nietzsche, the great gatsby. a million little pieces, the perks of being a wallflower, the interpreter of maladies, water for elephants, the picture of dorian gray, american holocaust, the girl on the fridge, pride and prejudice, harry potter, and no one belongs here more than you.  most well made scientific/environmental documentaries. films featuring daniel day-lewis or tom hanks. let the right one in, harry potter, secretary, brick, dedication, shaun of the dead, green street hooligans, snatch, the diving bell and the butterfly, invincible, howl's moving castle, death at a funeral, the holiday, the jacket, everything is illuminated, good will hunting, pride and prejudice, and the departed.  sage francis, max richter, the honorary title, sufjan stevens, passion pit, the decemberists, william fitzsimmons, run on sentence, ryan adams, akron/family, as tall as lions, the album leaf, minus the bear, bob dylan, rilo kiley, thrice, claude debussy, animal collective, iron and wine, athlete, stars, john vanderslice, city and colour, philip glass, and joanna newsom.  i'll try most anything as long as it doesn't include meat. i don't eat meat but it's a personal choice and doesn't bother me in the least if you do. tofu in peanut sauce, tomato soup w/ grilled cheese, fresh green beans, rice and all sorts of fruit.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "kid's books, picture books, adult books and anything in between. where the wild things are. blink. non-fiction. cookbooks, magazines, blogs, short stories. walk across america. architectural digest. elle decor, anything design related. nyt, getting lost in the world wide web.  live, acoustic music. something to tap my feet to. piano, vocals, anything that inspires your soul. always open to learning and hearing more. in fact, my ipod needs your help.  love local, healthy and sustainable. if i can pick it or meet the grower, even better. love great desserts, wine, cold beer, homemade bread, cheese. tartine, burma superstar, moki's and cheeseboard pizza, broccoli, medium spicy, lightly salty and lots of sweet-y. i don't eat meat, and i don't mind if you do.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "books: kurt vonegut & david sedaris. short stories from the new yorker, and periodical collections like granta and paris review. vladimir nabokov, especially ada. last year's top accolades go to what is the what (eggers) and whatever it takes (tough).  i came across bukowski recently and i was smitten, especially after listening to a reading of the bluebird (check a version out on youtube). then i stumbled across a video of an interview that included him arguing with a girlfriend and he actually kicked her. like a five year old kid. i'm still in such awe i feel the need to share. i guess most great artists are atleast a bit cuckoo.  movies: sin nombre once (swell season is my favorite) spring, summer, fall, winter...and spring garden state closer  tv: i don't have a tv but do hop on the series marathon bandwagon via my laptop. the wire is the best show to ever grace a television screen. i also like californication, mad men, the office, and glee (shameful? maybe, but shunning popular culture is popular in itself.)  music: i'm not a music snob. i love to dance and sing and i appreciate music's ability to cradle my bad mood or endorphinize(?) my great mood. i really don't like to listen to angsty men whine over guitar music. i like to pretend that i can't tell the difference between neil young and bob dylan. i'm convincing. i love talking heads, paul simon, and fleetwood mac. i also listen to lil wayne more often than i like to admit.  food: i don't eat meat other than seafood, which i can't live without; maryland crabs are the best delicacy on this earth. i love fruits, especially pears, berries, nectarines, and pineapple. i eat a yogurt almost everyday. my favorite cuisine is mediterranean. for some reason, the word prawn really bothers me and i won't order them in a restaurant although i love shrimp.", 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == 'my favorite non-fiction book is the power broker. my favorite novel is in search of lost time. i read a lot but i think it\'s annoying to talk about too much, so i try not to unless specifically prompted.  my favorite movies are repo man, dirty dancing, and bull durham. i know there are other good movies that weren\'t made in the eighties because i\'ve heard people talk about them.  i don\'t watch tv, not because i\'m a dick about it but because i just don\'t. i do like to watch boxing and if you do too and have cable television we should become life partners or at least bffs.  i often think, "how can i have three weeks of music in my computer and feel like there\'s nothing to listen to, but when i was fourteen the right two tapes seemed like enough?" it might just be the curse of our age, as i also feel this way about internet dating. in any case, right now i\'m really missing miami radio.  i don\'t buy this thing everyone\'s into these days about how liking to eat food is a hobby or skill. everyone likes to eat, unless they\'ve got some kind of problem! it is not a legitimate thing to brag about, according to me, but i guess i\'m behind the times... i do not eat meat. i do eat fish. i pretty much need to eat something every forty-five minutes, so if we go out dating, you might bring some snacks.', 'diet'] = 'mostly vegetarian'

ok_cupid_df.loc[ok_cupid_df['essay4'] == "i love non fiction books, i love dramas, memoirs, and sad books (similar taste in movies). i love all kinds of food. i dont eat meat (well i do eat seafood/sushi).  books: lovely bones, intensity, stolen lives, running with scissors, a child called 'it.' magical thinking, possible side effects, are you there vodka it's me chelsea, my horizontal life, borrowed time, fish, i'm not myself these days.", 'diet'] = 'mostly vegetarian'

In [93]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("eat meat"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay4'] == essay, 'diet'] = 'anything'

In [94]:
for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains("eat meat"))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'diet'] = 'anything'

In [95]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "you have something nice to say. you're a big pile of awesomeness.. you like to dance all night. you toke the kind herb. you wanna have a good time and be treated right. you're idea of a good time doesn't necessarily mean drinking alcohol. you're a down to earth person looking for deep intimate connections with attractive like minded people. you're a tree hugger. you prefer analog vs digital. you don't wear fragrance aka phtalates on your body (kudos points if you know what i'm talking about). you don't eat meat (sorry). you're not selfish, shallow, or too materialistic..", 'diet'] = 'vegetarian'

In [96]:
essays_list = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
for e in essays_list:
    for essay in ok_cupid_df[(ok_cupid_df['diet'] == 'unknown_diet') & (ok_cupid_df[e] != np.nan) & (ok_cupid_df[e].str.contains("pescatarian"))][e].tolist():
        ok_cupid_df.loc[ok_cupid_df[e] == essay, 'diet'] = 'mostly vegetarian'

In [97]:
ok_cupid_df['diet'].value_counts()

unknown_diet           17314
mostly anything        13120
anything                5742
strictly anything       3654
mostly vegetarian       2619
mostly other             785
strictly vegetarian      699
vegetarian               564
strictly other           333
mostly vegan             266
other                    257
strictly vegan           184
vegan                    163
mostly kosher             70
mostly halal              41
strictly halal            15
strictly kosher           15
kosher                    14
halal                     11
Name: diet, dtype: int64

<span style="color:royalblue">From 18560 to 17314 unknwon_diets.</span>

#### <span style="color:royalblue">Change `unknown_diet` to `rather not say`:</span>

In [98]:
# find index of unknown_diet and replace with 'rather not say'
ok_cupid_df['diet'] = ok_cupid_df['diet'].str.replace('unknown_diet', 'rather not say')
ok_cupid_df['diet'].value_counts()

rather not say         17314
mostly anything        13120
anything                5742
strictly anything       3654
mostly vegetarian       2619
mostly other             785
strictly vegetarian      699
vegetarian               564
strictly other           333
mostly vegan             266
other                    257
strictly vegan           184
vegan                    163
mostly kosher             70
mostly halal              41
strictly halal            15
strictly kosher           15
kosher                    14
halal                     11
Name: diet, dtype: int64

In [99]:
ok_cupid_df.shape

(45866, 31)

### <span style="color:royalblue">Status:</span>

In [100]:
ok_cupid_df['status'].value_counts()

single            42697
seeing someone     1552
available          1357
married             254
unknown               6
Name: status, dtype: int64

In [101]:
# replace unknown with 'unknown_status' so it's more distinct
ok_cupid_df["status"].replace({'unknown': 'unknown_status'}, inplace=True)

In [102]:
ok_cupid_df.loc[ok_cupid_df['essay7'] == 'at home with my boy friend usually, listening to music on the i:pod in shuffle mode and painting', 'status'] = 'seeing someone'

#### <span style="color:royalblue">Drop `unknown_status`:</span>

In [103]:
index_status = ok_cupid_df[ok_cupid_df['status'] == 'unknown_status'].index
ok_cupid_df.drop(index_status, inplace = True)

In [104]:
ok_cupid_df['status'].value_counts()

single            42697
seeing someone     1552
available          1357
married             254
Name: status, dtype: int64

### <span style="color:royalblue">Body Type:</span>

In [105]:
# fill nulls in body_type with unknown_body_type
ok_cupid_df['body_type'] = ok_cupid_df['body_type'].fillna('unknown_body_type')
ok_cupid_df['body_type'].value_counts()

average              11653
fit                   9413
athletic              9116
unknown_body_type     3589
thin                  3580
curvy                 3010
a little extra        2169
skinny                1392
full figured           838
overweight             368
jacked                 309
used up                256
rather not say         167
Name: body_type, dtype: int64

#### <span style="color:royalblue">Fill `unknown_body_type` with info from essay columns (similar to diet):</span>

In [106]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'gainfully self-employed by day, amateur musician in my spare time, i play guitar and bass guitar ok, organ/ keyboards not so well. i sing and play in a band as a hobby. i read, tinker with electronics and mechanical stuff, write music and other stuff occasionally, in the little spare time my career affords me. i am fascinated with astronomy, but i\'m not an expert astronomer. i did, however, enjoy building my own 8" newtonian reflector telescope. i just like to set up and look at the moon, venus, jupiter, whatever i can find on any given night. i\'m a "blue-collar intellectual", i suppose. i can fix anything. the things that i have left blank in my details are blank because there were no suitable multiple-choice answers. for body type, i would have to say; "sturdy, stocky, strong as an ox". my income varies because i\'m self-employed. i\'ll say; "income: enough that i don\'t have to watch my pennies, but not enough to get rich quick". i have diverse interests, including astronomy, music, guitar, saxophone, organ, keyboard, electronics, antiques, camping, outdoors, commerce, and more!  i am sincere, spontaneous, and scatterbrained', 'body_type'] = 'rather not say'

In [107]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'sweet queer poly geeky kinky feminist liberal overly verbose cynical optimist with a dirty and dark sense of humor and fever dreams of polymathy iso... awesome people i haven\'t met yet, for surprising and delicious and hilarious interactions across the spectrum of possibilities. (this legitimately and honestly includes new friends -- i\'m not interested in filtering and discarding neat individuals out of my life just because there\'s no mutual relationship chemistry.)  i consider \'there\'s no way to do that\' a statement of challenge, not a concession to inevitability. i take sex both as intensely and as casually as it deserves. i don\'t like losing, but i try very hard to be gracious when i do. i\'ve got a sadly limited vocal range, but that doesn\'t stop me from singing when the mood moves me. my pronunciation is irrevocably and curiously erroneous in amusing ways. i\'m happiest when i\'ve just made someone laugh -- the harder the better. my (white, male, financial) privilege is examined, and kept under recurring surveillance. i\'ll try almost anything once, and on zero notice if my schedule is free.  (as for the body type self-summary in the details sidebar, i prefer "stocky", but that\'s not an available option on the list, so it gets put here.)', 'body_type'] = 'rather not say'

In [108]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'i\'m specifically looking to connect with those nearby* who have cultivated the ability to notice thought as it arises, and grok that thoughts aren\'t personal. (*or near bos, nyc, hnl -- places i\'ll be going to in the near future)  points of reference: rumi, yoda, tolle, ramana maharshi  gratitude, moments of noticing being both the ocean and a drop of water, easy laughter, embodied awareness/presence practices  fun-loving -- whether being deeply silent or raucously silly committed to serving the evolution of consciousness (and open to other ways of describing it) sometimes simultaneously grounded and expanded/ethereal  once in a while, i run; often dance. love to walk in forests, sit in forests, stand in forests, breathe in forests :) also have come to deeply appreciate the silence and magic of swimming with fishes, turtles, and the like. (i would say "snorkeling," but i only use a mask.)  seem to have developed a green-ish thumb, and envision gardening more in the upcoming months.  a note about children -- for a while, wanted to adopt 10 and birth a couple; life unfurled differently: no biological children, and 2 sort-of-semi-stepdaughters who are now adults.  unsure what to put down for "body type" -- kinda fit, kinda not, kinda average, kinda thin = ? when seated, am taller than one of my 6\' tall friends.', 'body_type'] = 'average'

In [109]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i would describe my self as someone loves making people laugh and loves to follow their visions. i am generally into open relationships, queerdos and newcummers too. i am open to being in a closed relationships too if it feels right. oh ya and i am kinda poor, so don't expect me to buy you gifts all the time but you can expect me to warm your little tender boygirl heart with a cup of hot chocolate with marsh mellows inside. or show you a documentary on direct actions, or videos of circus and clowns.  i love movies, nature, kink, love making, radical faeries, cage fighting (for some gay reason), thrift shopping, being supportive, booty shaking fun time, sex positivity, all body type positivity. i am big bodied, wacky, muscular, cuddly, chewy, and have tattood hair and smile lines. i am loving, strange, sweet and have luscious lips. mwahahaha!", 'body_type'] = 'curvy'

In [110]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == 'i am not afraid of working hard. but i am not defined by my work. trust me, i love to play! i have been very busy traveling over the past few years. i also like to keep myself busy by trying new activities. in the past i have trained for two century rides (100 mile bike rides) and in the past few months - since the end of 2011 - i have really gotten into crossfit (which is why i didn\'t answer the body type question. there wasn\'t an option for improving everyday-lol). i am also hoping to resume taking guitar lessons, if time allows this year. basically as the years go on, my "bucket list" keeps growing , and i keep trying to check things off of it. there doesn\'t really seem to be any good place to add this, but i also love going to watch baseball games either at the park or local bars.', 'body_type'] = 'rather not say'

In [111]:
ok_cupid_df.loc[ok_cupid_df['essay8'] == 'i didn\'t pick a body type because "fat kid with sticky fingers trapped inside" wasn\'t an option.', 'body_type'] = 'rather not say'

In [112]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("body type"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'body_type'] = 'rather not say'

In [113]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "looking for someone who resembles one of those whip-smart curvy blondes from 40s movies? my friends call me the hitchcock blonde. i am reserved at first, but deep down i'm a bit of live wire, a rule breaker, a paradigm maker, a mysterious stranger and a sometime saint. i have a kind heart, an open mind, an adventurous spirit, a wry wit, a devilish streak and a master touch. i can be elegant or earthy. i'm intelligent, passionate, content, confident, well-traveled, curious, opinionated, intense, sensual and i will make your soul burn, your heart ache and your brain spin.", 'body_type'] = 'rather not say'

In [114]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm pretty easy going, love to laugh and make others laugh. i'm passionate about life and trying to make the most of it! currently living by 2 mottos, the first is yolo (you only live once)... i swear i came up with it before drake! haha. and the second is gntl (got nothing to lose). trying to seize opportunities and go for what i want, don't want to look back with regret.  i enjoy a lot of different things: hanging with friends (and making up hand shakes with them haha), food, movies, shopping (i'm pretty into fashion), playing/listening to/composing music (music is one of my biggest passions), traveling, sports (playing and watching), snowboarding, driving (not only is it fun to go whipping around curvy roads, exploring new places, and bumpin' my favorite tunes, i also find that driving can be very therapeutic, i could drive around with no destination and just enjoy the time to think), dancing (love to dance, whether it's in a dance class, in the club, or in public haha, drumming, playing the piano, and much more...  if you want to know more, just ask :)", 'body_type'] = 'rather not say'

In [115]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("curvy"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'body_type'] = 'curvy'

In [116]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == "i'm constantly enriching my life with film, science and fine arts. i work professionally in the local entertainment industry and volunteer to help the planet.  i'm a creative curvy girl that's a foodie and not a junk hound. i also enjoy the outdoors, love hiking, visiting the ocean, wine tasting, the snow (includes the slopes) and did i mention good food? hobbies include hat making and writing. what about you?", 'body_type'] = 'curvy'

In [117]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay9'] != np.nan) & (ok_cupid_df['essay9'].str.contains("curvy"))]['essay9'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay9'] == essay, 'body_type'] = 'curvy'

In [118]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "you are interesting. you are passionate. you appreciate eccentricity. you are not extremist in your political persuasion. you are very nervous on first dates or find that quality to be strangely attractive. you have divergent interests. you are not trying to impress others nor looking to be impressed. you like short, skinny, jewish, academic types. you notice things like grammatical errors, misquotes or incorrect attributions but don't always feel the need to point them out. or when you do, it doesn't make you feel superior. you can get into an impassioned debate or discussion with someone even if you have ideological differences and maintain your composure and respect for the other. then go out for drinks afterwards. you think that intelligence is exciting and important but not everything.", 'body_type'] = 'skinny'

In [119]:
ok_cupid_df.loc[ok_cupid_df['essay3'] == 'that i have a third nipple.  no, seriously, probably my very damaged bottle-blonde hair, my ever-changing style or my never-changing (but always expanding) tattoos. some of them are pretty stupid and hilarious! you should hear my idea for the next one! totally ridiculous! also, in the past on many occasions i\'ve been called "healthy" and "thick". take your pick. i aint skinny, but i carry myself well.', 'body_type'] = 'curvy'

In [120]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay3'] != np.nan) & (ok_cupid_df['essay3'].str.contains("skinny"))]['essay3'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay3'] == essay, 'body_type'] = 'skinny'

In [121]:
ok_cupid_df.loc[ok_cupid_df['essay1'] == "i work with words and pictures and paper. my phone autocorrects 'and' to abd. i'm also pretty into running (full marathons, fast and fairly often.) i'll just come right out and say it: i'm kind of a skinny bitch who will mock you for having to train for five months to run a half-marathon. i like wasting money on antiquarian books.", 'body_type'] = 'skinny'

In [122]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "advertising and nerd don't seem to go together, but thats what i'm striving to be. a perfectionist most of the time, anti social some of the time. consider my self pretty well traveled amongst my peers, but not well versed about telling people my experiences. not short, not tall, not skinny, not fat, swim a lot, watch tv a lot, just picked up a reading habit. if you want to see some of my work, go to iambo.com  i am honest, well traveled, and patient", 'body_type'] = 'average'

In [123]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "skinny, smart, laid back guy seeks to become more muscled and socially acceptable, or at least accepted. relatively fur-less, i like fur. i gravitate toward stability. political animal of the extreme and militant middle. the kind you bring home, seeking to be less so.  found: playing starcraft, lifting weights at golds, hanging out at chaps on occasion, starting conversations with strangers on the muni.  i am laidback, kinky, and skeptical", 'body_type'] = 'skinny'

In [124]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i believe there's a somebody for everybody. i'm looking for mr. right and not mr. right now. i'd like to find someone who's interested in having a potential relationship. (i say potential because this process is by trial and error. and this doesn't have to happen tomorrow...) on that note i'm not a professional or serial dater so i hope you're not either. (and please be employed) i pretty much grew up in san francisco/daly city and can't imagine calling another place home. i'm a pastry assistant at a busy restaurant in the city. so needless to say, i love food. if you're looking for skinny-ha! that's not me but by no means am i unhealthy. i'm pretty mellow, sometimes i'm funny but mostly sarcastic. (so please have a sense of humor) lately, my favorite cuss words are mother fucker. i love anything romantic. i have several tattoos. i don't drink or 'do' coffee. (please don't ask me to meet you at a coffee shop) i try to work out but sometimes working out doesn't work out. i like doing the girly girly things but i grew up with brothers so i don't mind getting dirty. (like going camping) my brothers recently taught me how to shoot a gun. after my first shot i put the gun down and was like 'oh mother fucker. what the fuck was that!?' they got a kick out of it and were laughing their asses off. then proceeded to make me finish the clip. in the end, they said my grouping was real good for a first timer and i can go back with them. hahaha...", 'body_type'] =  'a little extra'

In [125]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i've been compared to a newfoundland puppy more often than i realize. i get to do what i love and have learned to live richly while not being rich. i've been rich and poor well off and homeless and shy and a showman and here and there. i love travelling and keeping busy. i have a wide social network with a career that enables me to see the country at the expense of missing out on the things i'd like to do at home. more to come later. i usually think of exactly what i want to say en route somewhere and have forgotten it by the time i get to a computer.  i'm a musician, biker, forever trying to get a regular exercise plan going, bicyclist, burner, actor, teacher, comic, smart ass, skinny, wide-eyed, bashful, talkative, a night owl, worried about the mundane things, cynical, jaded, yet optimistic and have hope for the world.  i love observing and analying and forming theories to debunk or prove. i was raised by analysts so i like to talk about talking about things.  i am alert, talkative, and easily distracted", 'body_type'] =  'skinny'

In [126]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm into airline magazines, worst case scenario books, b-rated action flicks and the occassional malcolm gladwell book.  i am skinny, amphibious, and perceptive  i'm married, but trying to matchmake for my wonderful friends :)", 'body_type'] =  'skinny'

In [127]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "~*~actions speak-/- im deaf to words~*~*`*  im super sweet, fun loving, i love life love love, love the thought of being in love and love being in love,,, ,, life is too short for these short term relationships that leads nowhere but heartbreak,, im ready to find that 1 and settle down,,im honest and dont like liars or people who dont keep their word,,please keep it real with me as i will always with you!!!!!! im trustworthy non judgmental and in my eyes definitely a keeper,, my b day is 9 17 86. . im a down to earth person with a very troubled past... i grew up in foster care and never really had a family..... ive been through many obstacles which has made me a stronger person.... i write poetry... im a child of god and i am a good woman to a man. i treat a man like the king he is and expect to be treated like the queen i am.... im loyal and honest.... love to cake it up when i have some1 to do so with,, p.s word of advice (for yer sake),,, when u catch a good woman, realize it, cherish her and dont let her slip from ur grasp because u never know if ull ever find another as good as her,,,, im not necessarily talking about myself but the shoe does fit,, im just merely sharing a lesson i had to learn the hard way,,, take it however it applies and if it doesn't leave it as is,,, well i can write pages but i wont for the sake of your time thanks for reading my profile and good luck to everyone finding what they are looking for on here,,,,, amanda name ; amanda..... im 23.i have a son who will be 2 on july 29th 2010,, im very down to earth easy to get along with,,,, i put god first always.... im tall ,skinny, soulful ( i was raised by all black ppl) and i have a sense of humor...anything else u wanta know just ask... i value family want to be married 1 day and have more kids,, i love animals and kids,,,  i am positive, honest, and sweet", 'body_type'] =  'skinny'

In [128]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm 49 years old and a fairly good-looking and fit white guy, 155lbs, 5'10.  i have had a very eclectic life and am broad-minded about all sorts of things. the range of experiences i have had have made me a better person, or at least interesting. i have degrees in science, arts, and humanities. i perform different kinds of music--music of bali and western classical music (in choruses). i've traveled a lot--to about three dozen countries--and have lived about five years overseas, mostly in europe and indonesia (speak fluent indonesian and some balinese). i still travel to bali often and own land there; many of my friends are balinese. it's been a gift to be exposed to so many different places and cultures, and it's a core part of who i am now.  being environmentally sensitive is an important aspect of my life. i would want environmental awareness to be important to my partner. i recently completed my ph.d. in environmental theory and increasingly try to tread lighter on earth in my everyday practices, though i'm not fanatical about it (well, ok, maybe sometimes). politically, i'm on the left, but i challenge the dogma of the left, too. i love animals and volunteer at an animal shelter. i'm pretty much vegetarian except for seafood.  i'm shy, but open up a lot when i feel accepted and supported. i'm not very detail oriented, so someone who likes, say, planning the minute details of a trip would be a particularly good counterpart for me; i'm good at the big-picture stuff. i have a wry sense of humor that some say is very funny.  i'm a sensuous person. i like to cuddle. aside from playing music, i am into hiking, traveling, occasional camping, hot springs, and various other fun things. mostly, i am into being with you, if you are the one.  i am sensitive, eclectic, and circumspect", 'body_type'] =  'fit'

In [129]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == '*** etiquette: if you\'re interested, please write an _individualized_ note that communicates personal pride and purpose (it needn\'t be long). i\'m happy to communicate and invest some time in people if they show some purpose. ***  my primary interest is the performance and recording of a large subset of the rachmaninoff and debussy solo piano corpus. i have already made one cd and am looking to make others (and should be done by the end of spring). i have also accompanied most excellent cellists in the performance of rachmaninoff\'s cello sonata. finally, i have a great appreciation for the female voice and enjoy accompaniment of talented singers, especially in schubert and rachmaninoff lieder.  by day, i\'m an applied statistician for a litigation consulting firm. after work, i revert into a distance runner who is also an advanced pianist. there are other interests, but i\'ll stick with the two interests just mentioned and give more details.  physical: i\'m an amateur distance runner (40+ mi/week) and fitness enthusiast (within reason). i\'ve run three marathon-distance runs with a maximum distance of 30 miles. also, i\'m a basketball junkie (playing, not watching). i\'ve taken very good care of myself and have no health issues whatsoever.  miscellaneous: i also enjoy classical literature, good science fiction, and excellent physical art. i have a fondness for voice, string quartets, and beauty in general. i\'d truly enjoy sharing that appreciation with somebody who can genuinely appreciate such things, and it would be excellent if she had her own artistry she could share with me.  for lighter things, i enjoy progressive rock, "lost," "seinfeld," "arrested development," observing (and often making fun of) popular culture, style, and reading.  i\'m 6\'0", 200 lbs. i\'ve never married (no children). my life is completely together with no issues, problems, things to fix, etc.  i am introspective, musical, and athletic.', 'body_type'] =  'fit'

In [130]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "my name is tristan (loupiote on okcupid). i grew up in paris and moved to san francisco more than 10 years ago. i am a software engineer by formation (now doing that part-time freelance), but i think my biggest passion is photography, and you can see a lot of my recent work at www.loupiote.com.  i'm often on-line and you can chat with me on im. i prefer using google-talk. here is where you can find me on the web:  loupiote.com - tumblr.com - tribe.net - myspace.com - facebook.com - friendster.com - orkut.com - netlog.com - twitter.com  i love traveling (usually far away, off the beaten track), connecting with people, and capturing photos of things and people that inspire me. i like fire, i like caves, i like ruins, i like to explore places where few people go. i prefer using two wheels (motorcycle or bike). i've been going to the burning man festival several times, taking amazing photos of the art and the people there. you can see them at www.playa-dust.com . i'm now considered part of the burning man official photographer's team.  i am frank, honest and truthful, and i expect the same from others. i am very loyal to my friends. some of my best friends live in paris, where i am from. i love europe, especially france, spain and italy. i can afford to take a lot of free time and i travel several weeks every years. i love traveling and adventure, especially when i can do that with someone i get along well. i also like windsurfing, skiing, spelunking / caving, urban exploration, late night dancing in underground venues, exploring san francisco on my motorbike (a klr 650), hiking and lots more. i don't generally identify as monogamous but i'm not sure to be polyamorous. i tend to be attracted by boyish, androgynous, queer women, bi or lesbian (ouch!).  you: you like to smile and are happy with life, with yourself, even when not everything is perfect. you have nice eyes. eyes are important to me. that's the first thing i look when i meet someone. i also like eyebrows and lips. you are maybe a bit tomboy-ish or androgynous, very natural, physically fit, maybe you have short hair, do not wear makeup or lipstick, share some interests with me like traveling, have the same philosophy on life and some sense of humor. you are ready for new adventures, you like to try new things. you like kissing and hugging. you enjoy life a day at a time. you are in your twenties or early thirties.  who i'd like to meet: another butterfly like me, someone who inspires me everyday and makes me feel like the happiest person in the world, someone i can trust, someone i can learn from, someone spiritual and yet someone down to earth, someone who understands the absurdity of life yet loves it and can joke about it, someone who i will never completely understand, someone who would love to explore the world with me, even if this requires some efforts, someone who feels the same as all this about me.  i prefer girls with short hair, so i always carry cisors :) just kidding!  i am adventurer, dreamer, and truthful", 'body_type'] =  'fit'

In [131]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'hi guys!  let me tell you a little about myself. i\'m a woman of many interests and talents - a jack (jill?) of all trades, as they say. i\'m an adventurer who enjoys traveling the world and exploring self and spirituality.  i\'m health conscious, not fanatical by any means, but i do try to make healthy choices. i buy organic dairy products, choose wheat over white and rarely drink alcohol. i\'m vegetarian and have been my whole life. you don\'t need to be vegetarian, but i prefer someone who is okay not having meat in the house. i know some people might freak out at this. i\'d like to recommend reading, "fast food nation" and watching the movie, "food, inc." it might change your mind. regardless, you\'ll definitely learn facts the mainstream media won\'t share with you.  i workout regularly. i\'m a dance fitness instructor. what do i teach? zumba, baby! have you heard of it? it\'s mostly the latin american (latino / hispanic) dances (salsa, merengue, etc) spiced with other global dances with a fitness twist. it\'s a big party! my passion is ethnic dancing. i\'m always up for attending a dance or theater show.  i love dances from other countries, and i love traveling to other countries too. my parents immigrated from india so i was lucky to have started traveling in my youth. in the last five years i\'ve visited mexico, india, singapore, taiwan, germany, czech republic, poland, hungary...in addition to hawaii and alaska! looking forward to more travel in asia. would love to tour and dance in egypt. an african safari would be wild!  i\'m looking for a long-term committed romantic relationship (as in marriage) built on a strong foundation of friendship, trust and respect. the first step in that though is seeing if we enjoy each other\'s conversation. conversation is the most important thing to me.  if there\'s anything in my profile that makes you think we could be a match, e-mail me! let\'s meet and see if we click. i really don\'t want to spend a lot of time on long e-mail exchanges or even talking on the phone (not looking for a pen pal or phone buddy). i prefer to meet in person. be sure you have photos and some substantial description on your profile.  you know how quickly time flies. let\'s get off this site and move on to happily ever after!', 'body_type'] =  'fit'

In [132]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "my name is david.i am 19 years young. i am new to the bay.i hail from pensacola florida.i love to play and stay fit.i am learning mma(mixed martial arts)kick boxing,ninpo tai jitsu.i am outgoing but i can be pretty laid back at times.i love to laugh and put a smile on others faces.i party like twice a month.im prety baddass when it come to throwing hands.  420 friendly  i am funny, free spirited, and hot to trot", 'body_type'] =  'fit'

In [133]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i am fit, ambitious, and odiously sarcastic", 'body_type'] =  'fit'

In [134]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "a little about me... i'm sincere, open-minded, energetic, love the world and accept life for what it is. i do believe enjoying the journey is as important as reaching the goal.  am into music, outdoor sports, n traveling. austria is my favorite. i'm into various sports.. badminton, basketball, racquetball, soccer.... hiking and try to stay fit :) i motorbike and absolutely enjoy it!  i am motorbiker, easy going, and adrenaline junkie", 'body_type'] =  'fit'

In [135]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("athletic"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'body_type'] = 'athletic'

In [136]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay1'] != np.nan) & (ok_cupid_df['essay1'].str.contains("athletic"))]['essay1'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay1'] == essay, 'body_type'] = 'athletic'

In [137]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay2'] != np.nan) & (ok_cupid_df['essay2'].str.contains("athletic"))]['essay2'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay2'] == essay, 'body_type'] = 'athletic'

In [138]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay3'] != np.nan) & (ok_cupid_df['essay3'].str.contains("athletic"))]['essay3'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay3'] == essay, 'body_type'] = 'athletic'

In [139]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay4'] != np.nan) & (ok_cupid_df['essay4'].str.contains("athletic"))]['essay4'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay3'] == essay, 'body_type'] = 'rather not say'

In [140]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay5'] != np.nan) & (ok_cupid_df['essay5'].str.contains("athletic"))]['essay5'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay5'] == essay, 'body_type'] = 'rather not say'

In [141]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay6'] != np.nan) & (ok_cupid_df['essay6'].str.contains("athletic"))]['essay6'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay6'] == essay, 'body_type'] = 'rather not say'

In [142]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay8'] != np.nan) & (ok_cupid_df['essay8'].str.contains("athletic"))]['essay8'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay8'] == essay, 'body_type'] = 'athletic'

In [143]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == 'you do your hair you do not wear athletic shoes except when you are exercising you do not leave the toilet seat up you are attracted to sarah silverman  do you love to travel? do you love to laugh? do you love love? does your taste in music define you? are you, like, laid back? are you looking for a totally chill girl? are you looking for "a partner in crime?" then i am not the one for you.', 'body_type'] =  'rather not say'

In [144]:
ok_cupid_df.loc[ok_cupid_df['essay9'] == "you like to say hello! and btw i love my curves and believe in practicing healthy living. but to those of you looking for sex object barbie, twiggy, the athletic, the less mature in age, and whatever pin up model looking dream you have in your head move on, because full figure means extra pounds, not quite bbw, but definately not barbie, twiggy, or athletic beach bunny and is not to be found here, but you will find a sexy, mature, beautiful, very sensual, full figure, educated and very real chocholate skin, ebony goddess.", 'body_type'] =  'full figured'

In [145]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay9'] != np.nan) & (ok_cupid_df['essay9'].str.contains("athletic"))]['essay9'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay9'] == essay, 'body_type'] = 'athletic'

In [146]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i'm a good egg, ut-bay ot-nay oo-tay ight-bray, if you know what i mean. i think humor belongs in music. i can and will eat anything. i'm getting better. i'm an avid picnicker and experimental chef. i'm sort of a pale, thin man/boy/child, and am wise beyond my years while bodily i still appear barely legal. i'm a recovering music addict, serving a life sentence as a fretted-stringed-instrumentalist, and although i'm currently listening to, seeing and playing less than i have in over ten years, i love it more and more, and remain deeply embroiled in and indebted to music for shaping my life.  i am kind, odd, and human", 'body_type'] =  'thin'

In [147]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == 'the shallow basics: 5\'2\'\'. thin with curves. copper/auburn haired. green eyed. ridiculously fair skinned (thank you, ancestors!).  characteristics: intelligent. compassionate. passionate. vivacious. new deal-type liberal. feminist. painfully direct. constantly questioning. considerate. cynical optimist. independent minded. realistic and idealistic. witty. sarcastic and dry sense of humor. analytical thinker, as well as one who thinks with the heart. earnest. proud. a romantic who often questions love. opinionated. sensitive. intellectualish (yes, i\'ve decided that that is in fact a real word.). redhead. largely self-aware with moments of blindness. contemplative. irreverent. intuitive. loyal. skeptical. unaffected and unpretentious. resilient. impatient and patient. caring. bold. strong-willed. somewhat of a misanthropic humanist. quality over quantity. prefers to take risks over playing it safe. seeks to be challenged.  interests: liberal politics. feminism. philosophy. art museums. indie music. various pretentious intellectual, as well as "trashy" stuff. taking a lot of spontaneous day trips/adventures with friends. soccer (currently watching euro 2012!).  random likes: vintage tortoise shell wayfarers. ballet flats. pea coats. the sunday new york times. rainy days. european style cafes. lazy weekend mornings. rothko and pollock paintings.  i am witty, irreverent, and callipygian.', 'body_type'] =  'thin'

In [148]:
ok_cupid_df.loc[ok_cupid_df['essay3'] == 'i am thin and a bit geeky, but also very easy to get along with. friends that get to know me think i am also pretty funny and a little bit silly at times.', 'body_type'] =  'thin'

In [149]:
ok_cupid_df.loc[ok_cupid_df['essay4'] == 'i havent read many books in my life. when i was young we read the hobbit and of mice and men. those books at the time seemed interesting i guess but the books ive read lately are these italian mafia books. when a mafia guy cooperates with the government he seems too wright a book and there very good usually. the movies i like are the ones that are not the norm. theres a movie called the dark backward, very good movie and id say its different. the great directors like tarantino, martin scorsese and oliver stone usually make the best movies. i started listening too classic rock music when i was a kid and i still listen to it. same songs just in a different order. i have expanded my interests a little by listening to some classical music but not much. food, wow theres so much good food in america, probably why were so heavy here. im thin with a high metabolism so i have the luxery to eat anything and as much as i want. id say chinese and italian are at the top of my list.', 'body_type'] =  'thin'

In [150]:
for essay in ok_cupid_df[(ok_cupid_df['body_type'] == 'unknown_body_type') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("full figured"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'body_type'] = 'full figured'

#### <span style="color:royalblue">Replace remaining `unknown_body_type` with `rather not say`:</span>

In [151]:
ok_cupid_df['body_type'] = ok_cupid_df['body_type'].str.replace('unknown_body_type', 'rather not say')

In [152]:
ok_cupid_df.isna().sum()/ok_cupid_df.shape[0]*100

age             0.000000
status          0.000000
sex             0.000000
orientation     0.000000
body_type       0.000000
diet            0.000000
drinks          3.517226
drugs           0.000000
education      10.928914
ethnicity       8.637157
height          0.002181
income          0.000000
job            12.812909
last_online     0.000000
location        0.000000
offspring      57.738770
pets           33.185783
religion       31.689926
sign           18.118186
smokes          5.994331
speaks          0.054514
essay0          9.498474
essay1         13.279546
essay2         16.761884
essay3         19.156127
essay4         18.587004
essay5         18.556476
essay6         23.826864
essay7         21.046664
essay8         32.810728
essay9         21.829481
dtype: float64

### <span style="color:royalblue">Education:</span>

In [153]:
# fill null with unknown education
ok_cupid_df['education'] = ok_cupid_df['education'].fillna('unknown_education')
ok_cupid_df['education'].value_counts()

graduated from college/university    17706
graduated from masters program        7022
unknown_education                     5012
working on college/university         4595
graduated from two-year college       1302
working on masters program            1234
graduated from high school            1223
graduated from ph.d program           1028
working on two-year college            921
graduated from law school              804
dropped out of college/university      778
working on ph.d program                724
college/university                     609
graduated from space camp              498
graduated from med school              370
dropped out of space camp              361
working on space camp                  304
working on law school                  190
two-year college                       186
working on med school                  173
dropped out of two-year college        154
masters program                        105
dropped out of masters program         103
dropped out

In [154]:
# drop unknown education
index_education = ok_cupid_df[ok_cupid_df['education'] == 'unknown_education'].index
ok_cupid_df.drop(index_education, inplace = True)
ok_cupid_df.isna().sum()/ok_cupid_df.shape[0]*100

age             0.000000
status          0.000000
sex             0.000000
orientation     0.000000
body_type       0.000000
diet            0.000000
drinks          1.938895
drugs           0.000000
education       0.000000
ethnicity       7.689483
height          0.000000
income          0.000000
job             8.622209
last_online     0.000000
location        0.000000
offspring      56.225519
pets           30.182628
religion       28.446925
sign           15.692323
smokes          4.205836
speaks          0.058754
essay0          8.590384
essay1         11.652957
essay2         15.158637
essay3         17.596945
essay4         16.438993
essay5         16.661770
essay6         21.942323
essay7         19.141696
essay8         30.838719
essay9         20.179691
dtype: float64

### <span style="color:royalblue">Job:</span>

In [155]:
ok_cupid_df['job'] = ok_cupid_df['job'].fillna('rather not say')
ok_cupid_df['job'].value_counts()

other                                5345
rather not say                       3852
student                              3797
science / tech / engineering         3530
computer / hardware / software       3349
sales / marketing / biz dev          3160
artistic / musical / writer          3036
medicine / health                    2725
education / academia                 2577
executive / management               1737
banking / financial / real estate    1694
entertainment / media                1474
law / legal services                  947
hospitality / travel                  944
construction / craftsmanship          692
clerical / administrative             583
political / government                541
transportation                        269
unemployed                            227
retired                               194
military                              175
Name: job, dtype: int64

### <span style="color:royalblue">Ethnicity:</span>

In [156]:
ok_cupid_df['ethnicity'] = ok_cupid_df['ethnicity'].fillna('unknown_ethnicity')
ok_cupid_df['ethnicity'].value_counts()

white                                                                              22526
asian                                                                               4679
unknown_ethnicity                                                                   3141
hispanic / latin                                                                    1911
black                                                                               1504
                                                                                   ...  
asian, black, native american, hispanic / latin, white                                 1
black, native american, pacific islander, hispanic / latin, white, other               1
asian, native american, pacific islander, hispanic / latin, white, other               1
asian, middle eastern, native american, hispanic / latin, white                        1
asian, middle eastern, black, indian, pacific islander, hispanic / latin, white        1
Name: ethnicity, Leng

In [157]:
# drop rows with unknown ethnicity:
index_ethnicity = ok_cupid_df[ok_cupid_df['ethnicity'] == 'unknown_ethnicity'].index
ok_cupid_df.drop(index_ethnicity, inplace = True)

### <span style="color:royalblue">Offspring:</span>

In [158]:
ok_cupid_df['offspring'] = ok_cupid_df['offspring'].fillna('unknown_offspring')
ok_cupid_df['offspring'].value_counts()

unknown_offspring                          21067
doesn't have kids                           4742
doesn't have kids, but might want them      2712
doesn't have kids, but wants them           2615
doesn't want kids                           2022
has kids                                    1346
has a kid                                   1279
doesn't have kids, and doesn't want any      793
has kids, but doesn't want more              342
has a kid, but doesn't want more             211
has a kid, and might want more               169
wants kids                                   144
might want kids                              114
has kids, and might want more                 79
has a kid, and wants more                     52
has kids, and wants more                      20
Name: offspring, dtype: int64

### <span style="color:royalblue">Pets:</span>

In [159]:
ok_cupid_df['pets'] = ok_cupid_df['pets'].fillna('unknown_pets')

In [160]:
index_pets = ok_cupid_df[ok_cupid_df['pets'] == 'unknown_pets'].index

# drop rows with unknown pets
ok_cupid_df.drop(index_pets, inplace = True)
ok_cupid_df.isna().sum()/ok_cupid_df.shape[0]*100

age             0.000000
status          0.000000
sex             0.000000
orientation     0.000000
body_type       0.000000
diet            0.000000
drinks          0.991915
drugs           0.000000
education       0.000000
ethnicity       0.000000
height          0.000000
income          0.000000
job             0.000000
last_online     0.000000
location        0.000000
offspring       0.000000
pets            0.000000
religion       22.443480
sign           10.768828
smokes          2.676299
speaks          0.044917
essay0          7.231622
essay1          9.645905
essay2         12.576733
essay3         14.448271
essay4         13.490043
essay5         13.478814
essay6         18.625543
essay7         16.012876
essay8         26.369966
essay9         17.169486
dtype: float64

### <span style="color:royalblue">Religion:</span>

In [161]:
ok_cupid_df['religion'] = ok_cupid_df['religion'].fillna('unknown_religion')
ok_cupid_df['religion'].value_counts()

unknown_religion                              5996
agnosticism but not too serious about it      1535
catholicism but not too serious about it      1392
agnosticism and laughing about it             1300
other                                         1255
agnosticism                                   1227
christianity but not too serious about it     1219
other and laughing about it                   1100
atheism and laughing about it                 1050
other but not too serious about it             956
christianity                                   905
atheism                                        881
judaism but not too serious about it           773
atheism but not too serious about it           674
christianity and somewhat serious about it     581
other and somewhat serious about it            498
atheism and somewhat serious about it          481
catholicism                                    441
catholicism and laughing about it              412
buddhism but not too serious ab

#### <span style="color:royalblue">Find religion from essay columns (similar to diet):</span>

In [162]:
ok_cupid_df.loc[ok_cupid_df['essay0'] == "i just have to say that internet dating is so weird. i love perusing these intimate little snapshots of all of you, and trying to imagine what it would be like to go out with you, or what your skin smells like, or how your face moves when you talk. and probably the imagination is more fun then the actual date. meaning, that undeniable chemistry that you feel when you meet someone is not something you can feel except within sight of that person. and btw i would really like to hear the scientific explanation for the way i can feel someone looking at me and raise my head and zero in on that person's face in two milliseconds without even having to look around. i already know exactly the coordinates of those eyes. what is that? but back to what i was saying, which is that this is really a poor substitute for meeting you in person. but, considering that most of the time i'm too busy to even read my damn personal email, i guess it will have to do. (update; it is much easier to keep up with email now that it is in my phone.)  i was raised atheist by scientists and now i go to church sometimes. i am dichotomous. i have been called little miss kick ass, and if there is one reason why i'm still single, i think it's probably summed up best by my chinese horoscope -- fire dragon.", 'religion'] =  'christianity but not too serious about it'

In [163]:
for essay in ok_cupid_df[(ok_cupid_df['religion'] == 'unknown_religion') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("agnostic"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'religion'] = 'agnosticism'

In [164]:
for essay in ok_cupid_df[(ok_cupid_df['religion'] == 'unknown_religion') & (ok_cupid_df['essay0'] != np.nan) & (ok_cupid_df['essay0'].str.contains("atheist"))]['essay0'].tolist():
    ok_cupid_df.loc[ok_cupid_df['essay0'] == essay, 'religion'] = 'atheism'

In [165]:
ok_cupid_df.loc[ok_cupid_df['essay6'] == "i'm an atheist. i am usually not a closeted anti-theist, but sometimes i am. i think this is it, but the notion of an afterlife, like many of the popular religions profess to, would be a pretty rad idea. unfortunately, faith alone, isn't a basis with which i wish to base this enticing concept. having said that, i'm generally a subscriber of the old adage, live life and let live. if it rains for 40 days and 40 nights i'm not going to start building an ark, but if you do, it's ok, as long as i get to laugh during construction time.", 'religion'] =  'atheism'

In [166]:
# Save half cleaned data into a dataframe
ok_cupid_df.to_csv(r'data/okcupid_profiles_half_clean.csv', index = False, header=True)